In [76]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


### AM I MISSING COMMENTS AT THE VERY BEGINNING OF GAMES? (e.g. PHI201006260)

### Test for pinch runner: CHN197807192 event 34

import numpy as np # linear algebra
import pandas # data processin g, CSV file I/O (e.g. pd.read_csv)
import os
import time
from pprint import pprint
import re
#from IPython.core.display import HTML
from IPython.display import display, HTML, FileLink

pandas.set_option("display.max_colwidth", None)

os.chdir(r'/kaggle/working')

nr_ur_rbi_fixer_re = '(\((NR|UR|RBI)\))+'

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

games_dir = '../input/baseball-events-from-retrosheetorg/'
events_by_season_dir = '../input/baseball-events-from-retrosheetorg/events_by_season/'
metadata_dir = '../input/baseball-events-from-retrosheetorg/'
rosters_dir = '../input/baseball-events-from-retrosheetorg/'


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

thisyear = 2022
#first_eventID is x+1 for x=.... 1914: -1, 1915: 102265, 1916: 206508, 1917: 274581
# ...1918: 334755, 1919: 387211, 1920: 440263, 1921: 519526, 1922: 622095, 1923: 724789
# 1924: 823555, 1925: 921267, 1926: 1018705, 1927: 1117758, 1928: 1216750, 1929: 1312653
# 1930: 1414814, 1931: 1516602, 1932: 1615338, 1933: 1712436, 1934: 1809188, 1935: 1906814
# 1936: 2003004, 1937: 2104921, 1938: 2204915, 1939: 2298485, 1940: 2395722, 1941: 2491706
# 1942: 2585275, 1943: 2677038, 1944: 2761369, 1945: 2837546, 1946: 2921271, 1947: 3012211
# 1948: 3106903, 1949: 3201387, 1950: 3295775, 1951: 3391898, 1952: 3490695, 1953: 3588623
# 1954: 3690977, 1955: 3793936, 1956: 3898064, 1957: 4002836, 1958: 4110524, 1959: 4217503
# 1960: 4325436, 1961: 4434469, 1962: 4559413, 1963: 4701869, 1964: 4841301, 1965: 4981923
# 1966: 5123464, 1967: 5263781, 1968: 5404176, 1969: 5541484, 1970: 5712489, 1971: 5885210
# 1972: 6052585, 1973: 6212806, 1974: 6382357, 1975: 6553904, 1976: 6724831, 1977: 6894767
# 1978: 7081634, 1979: 7265248, 1980: 7449893, 1981: 7636884, 1982: 7760072, 1983: 7946562
# 1984: 8133209, 1985: 8320568, 1986: 8507244, 1987: 8695545, 1988: 8885080, 1989: 9070565
# 1990: 9257258, 1991: 9445455, 1992: 9635551, 1993: 9823981
# 1994: 10028311, 1995: 10173350, 1996: 10357703, 1997: 10565781
# 1998: 10772286, 1999: 10992699, 2000: 11215272, 2001: 11437783
# 2002: 11656750, 2003: 11874879, 2004: 12093569, 2005: 12313592
# 2006: 12530707, 2007: 12750774, 2008: 12972539, 2009: 13192881
# 2010: 13412119, 2011: 13629881, 2012: 13847459, 2013: 14065048
# 2014: 14282344, 2015: 14498943, 2016: 14716355, 2017: 14934691
# 2018: 15154007, 2019: 15374352, 2020: 15596400, 2021: 15676463
# 2022: 15893539, END: 16107577

first_eventID = 15893539 + 1

if ((thisyear == 1914) | (thisyear == 1915)):
    league_list = ['F', 'A', 'N']
else:
    league_list = ['A', 'N']

new_columns = ['vis_score', 'home_score', 'outs', 'runner_1b', 'runner_2b', 'runner_3b', 'sh_flag', 'sf_flag']

final_column_order = ['gameID',
 'event_in_game',
 'inning',
 'visitor_or_home',
  'vis_score',
 'home_score',
 'outs',
 'runner_1b',
 'runner_2b',
 'runner_3b',
 'batterID',
 'batter_hand',
 'pitch_count',
 'pitch_sequence',
 'play_orig',
 'pitcherID',
 'pitcher_hand',
 'field_C_playerID',
 'field_1B_playerID',
 'field_2B_playerID',
 'field_3B_playerID',
 'field_SS_playerID',
 'field_LF_playerID',
 'field_CF_playerID',
 'field_RF_playerID',
 'bat_DH_playerID',
 'pinch_runner_playerID',
 'pinch_runner_replaced_playerID',
 'comments',
 'badj',
 'padj',
 'theyear',
 'visteam',
 'hometeam',
 'theplay',
 'baserunning',
 'play_orig',
 'sh_flag', 
 'sf_flag'
]


fielder_columns =  ['field_C_playerID',
 'field_1B_playerID',
 'field_2B_playerID',
 'field_3B_playerID',
 'field_SS_playerID',
 'field_LF_playerID',
 'field_CF_playerID',
 'field_RF_playerID']


early_end_game_list = []
early_end_game_list += ['BRF191407230', 'BRO191407270']
early_end_game_list += ['PHI191507210', 'CHN191607180', 'NY1191609080', 'NY1191609182', 'PHA191905050']
early_end_game_list += ['CHA192104210','CLE192204270', 'DET192406130', 'PHI192705140', 'SLN192710020']
early_end_game_list += ['BRO193007242', 'NY1193007240', 'PIT193009240', 'BSN193105242']
early_end_game_list += ['PHI193206150', 'PHI193305300', 'PIT193406262']
early_end_game_list += ['BOS193604210', 'CHA193708190', 'NYA193708220', 'BRO193708270']
early_end_game_list += ['BOS193809070', 'CHN193906210', 'PIT194107290', 'BRO194208220']
early_end_game_list += ['BOS194408170', 'CHA194406252', 'PHI194908212', 'PIT195107042']
early_end_game_list += ['CHA195208030', 'BRO195205190', 'BRO195208160', 'NY1195207040', 'PIT195205050']
early_end_game_list += ['NYA195407070', 'BAL195506250', 'BRO195508100', 'MLN195608300']
early_end_game_list += ['PIT195806010', 'SLN195806140', 'SLN195909260']
early_end_game_list += ['CHA196008090', 'CIN196005250', 'SLN196005290']
early_end_game_list += ['KC1196107050', 'MIN196109300', 'CHN196109300', 'PIT196108110', 'SLN196104150']
early_end_game_list += ['CLE196204130', 'MLN196305290', 'MIN196406220', 'WS2196404190']
early_end_game_list += ['BOS196508220', 'MLN196509040', 'CIN196608130', 'NYN196608160']
early_end_game_list += ['BAL196707020', 'BAL196708030', 'DET196706210', 'KC1196709030', 'NYN196708270']
early_end_game_list += ['BAL196806270', 'WS2196804270', 'ATL196805250', 'NYN196809022', 'PHI196808010']
early_end_game_list += ['KCA196905310', 'NYA196906150', 'CHN196906070', 'MON196905180']
early_end_game_list += ['BOS197004200', 'MIL197005110', 'ATL197007202', 'CHN197006010', 'CIN197006020']
early_end_game_list += ['BOS197107172', 'WS2197106020', 'WS2197108110', 'WS2197109300']
early_end_game_list += ['BOS197206090', 'KCA197204260', 'ATL197207042']
early_end_game_list += ['TEX197306030', 'ATL197305190', 'SLN197306180']
early_end_game_list += ['BAL197405170', 'MIN197408020', 'MON197406100', 'PIT197406150']
early_end_game_list += ['CHA197505250','NYN197508060']
early_end_game_list += ['DET197606150', 'MIL197607300', 'TEX197605230', 'TEX197609080', 'NYN197609270']
early_end_game_list += ['MON197707290', 'SFN197705082', 'SLN197805280', 'MIN197908220']
early_end_game_list += ['TOR198007280', 'CIN198004110', 'BAL198306060', 'KCA198305240', 'TEX198306260', 'CHN198305010', 'SDN198309280']
early_end_game_list += ['CLE198405130', 'DET198405180', 'CIN198409250', 'NYN198407050', 'SDN198404270', 'SLN198404212', 'ATL198508240']
early_end_game_list += ['BOS198706220', 'CHA198707200', 'CHN198708272', 'CHN198807160', 'PHI198809240', 'SLN198909130']
early_end_game_list += ['DET199009060', 'NYA199007120', 'ATL199008290', 'CHN199004200', 'CIN199008200', 'SLN199007210']
early_end_game_list += ['BOS199207312', 'KCA199309240', 'CHN199307180', 'DET199405070', 'CIN199404060', 'FLO199408110']
early_end_game_list += ['CHA199505160', 'FLO199509220', 'CHA199608220', 'DET199609270', 'FLO199605260', 'PIT199609220']
early_end_game_list += ['CHA199704300', 'CLE199709090', 'COL199706082', 'FLO199706030']
early_end_game_list += ['KCA199909040', 'CHN199906010', 'CHN199906110', 'COL199909190', 'MIL199906010']
early_end_game_list += ['BOS200005100', 'NYA200009020', 'CIN200004030', 'SLN200006200']
early_end_game_list += ['BAL200108310', 'CLE200106210', 'CHN200110040', 'COL200108010', 'SLN200105300']
early_end_game_list += ['ATL200208150', 'CHA200305040', 'COL200306170', 'FLO200306210', 'PHI200406140']
early_end_game_list += ['KCA200505120', 'KCA200507030', 'CLE200609270', 'DET200709260', 'SLN200709060', 'BAL200809260', 'BAL200809270']
early_end_game_list += ['BAL200905060', 'BOS200906180', 'BOS200909120', 'CLE200906300', 'CHN200905260', 'WAS200905162', 'WAS200906042', 'WAS200907230']
early_end_game_list += ['NYN201004250', 'PIT201008210', 'CHN201105140', 'CHN201105250', 'BOS201207310', 'OAK201204100', 'CHN201207180']
early_end_game_list += ['DET201308270', 'PHI201309210', 'COL201505080', 'PIT201609290', 'WAS201609280']
early_end_game_list += ['CHA201707190', 'TEX201809210', 'CIN201809080', 'WAS201809260', 'NYA201904120', 'ATL201906110']
early_end_game_list += ['WAS202007230', 'CIN202106300', 'PIT202106100', 'SLN202110030', 'BAL202210030']

special_case_events = {}
special_case_events.update({'PHI191509092': [40]})
special_case_events.update({'PHA191605170': [22], 'NY1191606210': [8]})
special_case_events.update({'PHI191710022': [25]})
special_case_events.update({'CHA191806200': [67]})
special_case_events.update({'BRO191905060': [93], 'CHA191907100': [52], 'DET191906021': [28]})
special_case_events.update({'CHN192008300': [41], 'PHI192005060': [5]})
special_case_events.update({'BOS192206241': [52], 'DET192205060': [62], 'NYA192205100': [79]})
special_case_events.update({'BRO192207290': [49], 'CHN192207070': [73], 'CIN192207160': [35], 'SLN192205120': [62]})
special_case_events.update({'WS1192308022': [29], 'NY1192309020': [66], 'PHI192304280': [68], 'PIT192305240': [59]})
special_case_events.update({'CHA192407281': [71], 'CHA192407310': [44], 'DET192409130': [54], 'NY1192408041': [7], 'SLN192405180': [20]})
special_case_events.update({'DET192506030': [66], 'DET192507081': [64], 'BRO192510030': [25], 'BSN192505280': [6]})
special_case_events.update({'CHA192708240': [33], 'CLE192707041': [16], 'NY1192705170': [15], 'PHI192707021': {15}})
special_case_events.update({'PHI192806161': [98]})
special_case_events.update({'CLE192909250': [19], 'BRO192906181': [60], 'NY1192905040': [9]})
special_case_events.update({'PHA193005090': [6], 'BOS193109211': [41]})
special_case_events.update({'BOS193205170': [41], 'BOS193207150': [29], 'NYA193204300': [14], 'PHI193204300': [42]})
special_case_events.update({'BSN193306220': [18], 'SLN193304210': [16]})
special_case_events.update({'PHA193408070': [36], 'WS1193409292': [28]})
special_case_events.update({'BOS193507071': [3], 'DET193506230': [80], 'SLN193506090': [63]})
special_case_events.update({'SLA193606270': [57], 'SLA193607030': [22], 'WS1193607130': [13]})
special_case_events.update({'PHA193705020': [14], 'BRO193708210': [29], 'CIN193708070': [50], 'SLN193705312': [9]})
special_case_events.update({'CIN193809070': [50], 'SLN193805140': [47]})
special_case_events.update({'CHA193908140': [40], 'PIT193909212': [41]})
special_case_events.update({'NYA194005040': [75], 'SLA194005190': [58], 'BRO194009152': [46], 'PIT194006080': [32]})
special_case_events.update({'BOS194106240': [20], 'BOS194109150': [27], 'SLA194104270': [65], 'WS1194109242': [63], 'SLN194109211': [62]})
special_case_events.update({'NYA194204290': [65], 'NYA194206072': [74]})
special_case_events.update({'DET194307051': [73], 'CIN194407050': [38], 'NYA194508261': [64]})
special_case_events.update({'NYA194605120': [54], 'PHI194708110': [6]})
special_case_events.update({'CHA194908250': [10], 'PHI194909042': [76]})
special_case_events.update({'CLE195108230': [18], 'CIN195106150': [25]})
special_case_events.update({'NYA195306240': [54], 'BRO195308161': [16], 'SLN195305220': [6]})
special_case_events.update({'CHN195705121': [21], 'KC1195908100': [51], 'CLE196109200': [22]})
special_case_events.update({'BAL196308220': [38], 'LAA196308080': [34], 'LAN196706040': [54]})
special_case_events.update({'MIN196805270': [21], 'HOU196808110': [81]})
special_case_events.update({'BOS196908190': [12], 'OAK196906210': [99]})
special_case_events.update({'HOU197005230': [43], 'PHI197006090': [5]})
special_case_events.update({'CHA197305050': [41], 'ATL197304250': [79], 'SFN197408230': [57]})
special_case_events.update({'BOS197504180': [36], 'DET197508290': [43], 'TEX197506280': [37], 'HOU197507250': [79], 'SLN197604190': [127]})
special_case_events.update({'CAL197704220': [26], 'KCA197708300': [37]})
special_case_events.update({'DET197809180': [104], 'KCA197804150': [5], 'KCA197807030': [51], 'CHN197807192': [26]})
special_case_events.update({'CIN197807170': [70], 'HOU197808230': [11], 'CHN197907221': [11], 'LAN197904140': [46], 'SDN197906170': [52]})
special_case_events.update({'SEA198004230': [39], 'TEX198005190': [114], 'TEX198006190': [87], 'SLN198109290': [37]})
special_case_events.update({'CLE198209070': [50], 'ATL198307180': [13], 'CIN198307090': [46], 'SEA198405010': [31], 'HOU198405110': [4]})
special_case_events.update({'NYA198504180': [54], 'SEA198709260': [26], 'ATL198706180': [26], 'MON198708210': [20], 'PHI198705060': [18]})
special_case_events.update({'OAK198804180': [48], 'LAN198808030': [94], 'SEA198908110': [42], 'TEX198909020': [75], 'MON199009130': [7]})
special_case_events.update({'CLE199107300': [19], 'TEX199110040': [54], 'CHN199106280': [29], 'CIN199107120': [58], 'DET199208100': [37], 'PIT199308110': [57], 'NYN199407200': [85], 'OAK199606220': [37]})
special_case_events.update({'LAN200209170': [40], 'SFN200205310': [86], 'TOR200407240': [45], 'LAN200507030': [91]})
special_case_events.update({'SFN201407300': [64], 'LAN201504150': [55], 'CHN201708160': [40], 'PHI201708250': [39], 'MIL202204170': [12]})

def ordinaltg(n):
    return str(n) + {1: 'st', 2: 'nd', 3: 'rd'}.get(4 if 10 <= n % 100 < 20 else n % 10, "th")
g = 0

print('ok')


ok


In [77]:
s = time.time()

intentional_walk_just_i_re = re.compile('^I(?!W)')
iphr_re = re.compile('^H(?!(R|P))+\d*/+.*(IPHR)+')
hslash_re = re.compile('^H(?!(R|P))+\d*/+')

print('Adding game context data for games in {0:.0f}...'.format(thisyear))
print('\n')
print('reading rosters...')
#rosters_df = pandas.read_csv(rosters_dir+'rosters_through_{0:.0f}.csv'.format(thisyear), low_memory=False, index_col='rosterID')
rosters_df = pandas.read_csv(rosters_dir+'rosters.csv', low_memory=False, index_col='rosterID')
print('reading games...')
games_df = pandas.read_csv(games_dir+'games.csv', low_memory=False, index_col='gameID', encoding='utf-8', converters={'visitor_linescore': lambda x: str(x), 'home_linescore': lambda x: str(x)})

print('\tfixing linescores...')
games_df.loc[games_df['visitor_linescore'] == '"nan"', 'visitor_linescore'] = np.nan
games_df.loc[games_df['home_linescore'] == '"nan"', 'home_linescore'] = np.nan
games_df.loc[:, 'visitor_linescore'] = games_df['visitor_linescore'].apply(lambda x: str(str(x)[1:-1]))
games_df.loc[:, 'home_linescore'] = games_df['home_linescore'].apply(lambda x: str(str(x)[1:-1]))
games_df = games_df.assign(has_linescores = True)
games_df.loc[(games_df['visitor_linescore'].isnull()) | (games_df['home_linescore'].isnull()), 'has_linescores'] = False

print('\tchecking for DH...')
has_dh = True
if (thisyear < 1973):
    has_dh = False

print('\tgetting fielder columns...')
fielder_columns = ['field_C_playerID', 'field_1B_playerID', 'field_2B_playerID', 'field_3B_playerID', 'field_SS_playerID']
fielder_columns += ['field_LF_playerID', 'field_CF_playerID', 'field_RF_playerID'] 
if (has_dh):
    fielder_columns += ['bat_DH_playerID']


print('\tselecting games and roster rows for {0:.0f}...'.format(thisyear))
games_df.loc[:, 'date'] = pandas.to_datetime(games_df['date'], errors='coerce')
games_df = games_df[games_df['date'].apply(lambda x: x.year == thisyear)]
rosters_df = rosters_df[rosters_df['theyear'] == thisyear]

print('Reading events for {0:.0f}...'.format(thisyear))
events_df = pandas.DataFrame()
eventfile_list = []
for thisleague in league_list:
    eventfile_list.append('{0}{1:.0f}/{1:.0f}{2:}/events_{1:.0f}{2:}.csv'.format(events_by_season_dir, thisyear, thisleague.lower()))

# print('Reading these files:')
# pprint(eventfile_list)
# print('\n')

for thisfile in eventfile_list:
    events_i_df = pandas.read_csv(thisfile, encoding='utf-8', index_col='eventID', low_memory=False)#, dtype={'pitch_count': 'str'})
    events_df = pandas.concat((events_df, events_i_df), axis=0)

events_df = events_df.reset_index(drop=True)
events_df.index.name = 'eventID'

# print('\tbacking up...')
# events_df_bk = events_df

e = time.time()
g += e-s
print('\n')
print('Read {0:,.0f} events and {1:,.0f} games from {2:.0f} in {3:,.0f} seconds!'.format(len(events_df), len(games_df), thisyear, e-s))

oas_become_nps = []
oas_become_nps += [{'NYA198508082': [68]}]
oas_become_nps += [{'ANA201405050': [55]}, {'ANA201405050': [55]}, {'BOS201404230': [17]}, {'BOS201405300': [48]}, {'CHA201404260': [73]}, {'CHA201406120': [47]}, {'CHA201407190': [27]}, {'CLE201405210': [65]}]
oas_become_nps += [{'DET201407080': [60]}, {'DET201408160': [16]}, {'KCA201408300': [10]}, {'MIN201406220': [28]}, {'MIN201407050': [20]}, {'NYA201404162': [66]}, {'NYA201404290': [82]}]
oas_become_nps += [{'OAK201407040': [43]}, {'OAK201408040': [108]}, {'TBA201404220': [47]}, {'TBA201405080': [5]}, {'TBA201405250': [78]}, {'TBA201407290': [67]}, {'TBA201409160': [79]}, {'TEX201405100': [71]}, {'TOR201409120': [68]}]
oas_become_nps += [{'CHN201406040': [22]}, {'CHN201408210': [25]}, {'CIN201404110': [71]}, {'CIN201404290': [60]}, {'CIN201407090': [45]}, {'LAN201405090': [50]}, {'LAN201409030': [52]}, {'MIA201405310': [84]}]
oas_become_nps += [{'MIL201407110': [99]}, {'NYN201408010': [16]}, {'NYN201408140': [79]}, {'PHI201405230': [40]}, {'SDN201408270': [86]}, {'SDN201408300': [85]}, {'SLN201408020': [91]}, {'WAS201408240': [22]}]
oas_become_nps += [{'ANA201605280': [46]}, {'ANA201608300': [82]}, {'ANA201609100': [108]}, {'BAL201606220': [42]}, {'BAL201606240': [18]}, {'BAL201607270': [16]}, {'BAL201608020': [55]}, {'BAL201608310': [25]}, {'BOS201608100': [84]}]
oas_become_nps += [{'CHA201606130': [80]}, {'CHA201606240': [86]}, {'CHA201606250': [74]}, {'CHA201609060': [74]}, {'CLE201609020': [78]}, {'CLE201609180': [28]}, {'DET201604090': [47]}, {'DET201606040': [37]}, {'DET201608030': [48]}, {'DET201608270': [29]}, {'DET201609140': [78]}]
oas_become_nps += [{'HOU201605020': [45]}, {'HOU201607030': [57]}, {'HOU201609120': [104]}, {'KCA201605170': [60]}, {'KCA201605270': [52]}, {'KCA201605310': [95]}, {'KCA201607090': [35]}, {'KCA201607250': [71]}, {'KCA201608290': [74]}, {'KCA201609040': [91]}, {'KCA201609190': [51]}]
oas_become_nps += [{'MIN201604250': [45]}, {'MIN201609010': [41]}, {'NYA201605060': [87]}, {'NYA201608150': [12]}, {'OAK201606220': [34]}, {'OAK201607030': [63]}, {'OAK201609200': [73]}, {'SEA201605130': [89]}, {'SEA201605270': [62]}, {'SEA201606280': [31]}, {'SEA201609160': [80]}]
oas_become_nps += [{'TBA201607060': [31]}, {'TEX201605110': [15]}, {'TEX201605150': [19,26]}, {'TEX201609200': [73]}, {'TOR201604270': [55]}, {'TOR201605180': [46]}, {'TOR201605290': [102]}, {'TOR201607290': [23]}, {'TOR201608090': [42]}, {'TOR201608130': [13]}, {'TOR201609110': [52,107]}]
oas_become_nps += [{'ARI201605280': [33]}, {'ARI201606080': [38]}, {'ARI201610020': [75]}, {'ATL201604250': [84]}, {'ATL201605120': [54]}, {'ATL201606300': [102]}, {'ATL201608180': [69]}, {'ATL201609140': [54]}, {'CHN201605050': [70]}, {'CHN201607050': [23]}, {'CHN201607270': [12]}, {'CHN201608180': [62]}, {'CHN201609190': [75]}, {'CHN201609240': [72]}]
oas_become_nps += [{'CIN201608150': [22]}, {'CIN201608160': [78]}, {'CIN201608210': [50]}, {'COL201605140': [33]}, {'COL201607200': [11]}, {'LAN201604280': [53]}, {'LAN201607060': [149]}, {'LAN201607300': [87]}]
oas_become_nps += [{'MIA201608230': [22]}, {'MIL201606280': [5]}, {'MIL201608090': [11]}, {'MIL201609060': [91]}, {'PHI201605210':  [55]}, {'PHI201608040': [66]}, {'PHI201609020': [58,80]}, {'PHI201609300': [65]}, {'PIT201605180': [38]}]
oas_become_nps += [{'PIT201605240': [6]}, {'PIT201607200': [3]}, {'PIT201607230': [104]}, {'PIT201609240': [51]}, {'PIT201609250': [33]}, {'SFN201604080': [67]}, {'SFN201604220': [30]}, {'SFN201604270': [5]}, {'SFN201606120': [68,72]}, {'SFN201607080': [16]}, {'SFN201609180': [15]}]
oas_become_nps += [{'SLN201604190': [55]}, {'SLN201606290': [60]}, {'SLN201608080': [19]}, {'WAS201605090': [86]}, {'WAS201608140': [22]}, {'WAS201609260': [50]}]



# print('getting from backup...')
# events_df = events_df_bk

print('fixing some things in games and events...')
print('\tfinding and fixing baserunning...')
events_df = events_df.rename(columns={'theplay': 'play_orig'})
events_df = events_df.assign(theplay = events_df['play_orig'].apply(lambda x: x.split('.')[0]))
events_df = events_df.assign(baserunning_orig = events_df[events_df['play_orig'].apply(lambda x: len(x.split('.')) > 1)]['play_orig'].apply(lambda x: x.split('.')[1]))
events_df = events_df.assign(baserunning = events_df['baserunning_orig'].dropna().apply(lambda x: re.sub(nr_ur_rbi_fixer_re, '', x).replace('B', '0').replace('H', '4')))
events_df.loc[:, 'baserunning'] = events_df['baserunning'].fillna('')

# print('\tfixing pitch counts so they to be properly zero-padded when needed...')
# try:
#     events_df.loc[:, 'pitch_count'] = events_df['pitch_count'].apply(lambda x: '{0:02d}'.format(x))
# except ValueError:
#     print("\t\tCAN'T GET PITCH COUNT TO BE AN INTEGER FOR SOME REASON?...")
#     events_df.loc[:, 'pitch_count'] = pandas.to_numeric(events_df['pitch_count'], errors='coerce')

print('\tchecking for weird features in game data...')
print('\t\tchecking for games where home team batted first (to reverse linescores...)')


inning_reverse_games = []
inning_reverse_games += ['SEA200709261', 'PHI201006260', 'SEA201106250', 'TBA201505020', 'TBA201505030', 'MIL201709160']
inning_reverse_games += ['BAL202008052', 'BAL202008060', 'BAL202009172', 'NYA202008282', 'OAK202009262', 'CHN202009052', 'NYN202008252', 'PIT202009182', 'SDN202009190', 'SFN202009252', 'WAS202008222']


for revgame in inning_reverse_games:
    if (int(revgame[3:7]) == thisyear):
        print(revgame)
#         pprint(events_df[(events_df['gameID'] == revgame)][
#             ['gameID', 'event_in_game', 'visteam', 'hometeam', 'visitor_or_home', 'batterID', 'theplay']
#         ].head(10))
        events_df.loc[(events_df['gameID'] == revgame), 'visitor_or_home'] = events_df[
            (events_df['gameID'] == revgame)
        ]['visitor_or_home'].apply(lambda x: 1-x)
#         pprint(events_df[(events_df['gameID'] == revgame)][
#             ['gameID', 'event_in_game', 'visteam', 'hometeam', 'visitor_or_home', 'batterID', 'theplay']
#         ])
        vl = games_df.loc[revgame]['visitor_linescore']
        hl = games_df.loc[revgame]['home_linescore']
        games_df.loc[revgame, 'visitor_linescore'] = hl
        games_df.loc[revgame, 'home_linescore'] = vl


print('\tchecking for weird features in event data...')
print('\t\tchecking for NP events wrongly recorded as OA...')
for thisoa in oas_become_nps:
    for k,v in thisoa.items():
        for w in v:
            if (int(k[3:7]) == thisyear):
                events_df.loc[(events_df['gameID'] == k) & (events_df['event_in_game'] == w), 'theplay'] = 'NP'

print('\t\tfixing weird notations that would have broken stuff...')
print('\t\t\tIntentional walks indicated by I instead of IW (n = {0:,.0f})...'.format(len(events_df[(events_df['theplay'].apply(lambda x: re.search(intentional_walk_just_i_re,x) != None))])))
events_df.loc[
    (events_df['theplay'].apply(lambda x: re.search(intentional_walk_just_i_re,x) != None)), 
    'theplay'] = events_df[
    (events_df['theplay'].apply(lambda x: re.search(intentional_walk_just_i_re,x) != None))
]['theplay'].apply(lambda x: 
                  re.sub('^I(?!W)', 'IW', x)
                  )

print('\t\t\tInside the park home runs (n = {0:,.0f})...'.format(len(events_df[(events_df['theplay'].apply(lambda x: re.search(iphr_re,x) != None))])))
events_df.loc[
    events_df['theplay'].apply(lambda x: re.search(iphr_re, x) != None), 
    'theplay'] = events_df[
        events_df['theplay'].apply(lambda x: re.search(iphr_re, x) != None)
    ]['theplay'].apply(lambda x: re.sub('^H', 'HR',x))

print('\t\t\tHome runs recorded only as hits with direction (n = {0:,.0f})...'.format(len(events_df[(events_df['theplay'].apply(lambda x: re.search(hslash_re,x) != None)) & (events_df['theplay'].apply(lambda x: re.search(iphr_re,x) == None))])))
events_df.loc[
    (events_df['theplay'].apply(lambda x: re.search(hslash_re, x) != None)),
    'theplay'] = events_df[
        (events_df['theplay'].apply(lambda x: re.search(hslash_re, x) != None))
    ]['theplay'].apply(lambda x: re.sub('^H', 'HR/',x))

print('\t\t\tHome runs recorded only as hits with direction (n = {0:,.0f})...'.format(len(events_df[(events_df['theplay'].apply(lambda x: re.search(hslash_re,x) != None)) & (events_df['theplay'].apply(lambda x: re.search(iphr_re,x) == None))])))
#first_eventID

print('Fixing eventIDs...')
events_df = events_df.reset_index()
events_df.loc[:, 'eventID'] = events_df['eventID'].apply(lambda x: x + first_eventID)
events_df = events_df.set_index('eventID')

print('\n')
print('Done')


Adding game context data for games in 2022...


reading rosters...
reading games...
	fixing linescores...
	checking for DH...
	getting fielder columns...
	selecting games and roster rows for 2022...
Reading events for 2022...


Read 214,038 events and 2,479 games from 2022 in 14 seconds!
fixing some things in games and events...
	finding and fixing baserunning...
	checking for weird features in game data...
		checking for games where home team batted first (to reverse linescores...)
	checking for weird features in event data...
		checking for NP events wrongly recorded as OA...
		fixing weird notations that would have broken stuff...
			Intentional walks indicated by I instead of IW (n = 0)...
			Inside the park home runs (n = 0)...
			Home runs recorded only as hits with direction (n = 0)...
			Home runs recorded only as hits with direction (n = 0)...
Fixing eventIDs...


Done


In [78]:
def double_digit_scores_in_multitple_innings(games_df, thisgameID, this_inning, vh, printme = False):
    if ((thisgameID == 'CHN192208250') and (vh == 1)):
        sco = score_CHN192208250(this_inning, printme)  
    if ((thisgameID == 'PHI192907062') and (vh == 0)):
        sco = score_PHI192907062(this_inning, printme)
    if ((thisgameID == 'BRO194307100') and (vh == 1)):
        sco = score_BRO194307100(this_inning, printme)
    return sco

def score_CHN192208250(this_inning, printme = False):
    if (printme):
        print('\tFixing special case of CHN192208250: Cubs score 10 runs in the 2nd and 14 runs in the 4th')
    if (this_inning == 1):
        sco = 1
    elif (this_inning in [2,3]):
        sco = 11
    elif (this_inning in [4,5]):
        sco = 25
    elif (this_inning >= 6):
        sco = 26
    return sco


def score_PHI192907062(this_inning, printme = False):
    if (printme):
        print('\tFixing special case of PHI192907062: Cardinals score 10 runs in the 1st and 10 runs in the 5th')
    if (this_inning == 1):
        sco = 10
    elif (this_inning in [2,3]):
        sco = 11
    elif (this_inning == 4):
        sco = 13
    elif (this_inning in [5,6,7]):
        sco = 23
    elif (this_inning in [8,9]):
        sco = 28
    #print('PHI inning {0:.0f} score {1:.0f}, printme = {2:}'.format(this_inning, sco, printme))
    return sco

                            
def score_BRO194307100(this_inning, printme = False):                            
    if (printme):
        print('\tFixing special case of BRO194307100: Dodgers score 10 runs in the 1st and 10 runs in the 4th')
    if (this_inning in [1,2]):
        sco = 10
    elif (this_inning == 3):
        sco = 12
    elif (this_inning == 4):
        sco = 22
    else:
        sco = 23
    return sco
        
def fix_special_case_event(gameID, eventnum, state, printme = False):
    print('\tFixing a special case in game number {0:.0f} ({1:}) at event-in-game {2:.0f}: {3:}'.format(this_game_number, thisrow['gameID'], thisrow['event_in_game'], thisrow['play_orig']))

    if ((gameID == 'PHI191509092') and (eventnum == 40)):  
        print('\t\tExplanation: batter paskd101 still out at 3rd despite fielding error\n')
        state['onbase'][3] = ''
        state['nOuts'] += 1
    elif ((gameID == 'PHA191605170') and (eventnum == 22)):  
        print('\t\tExplanation: double steal of 2nd and 3rd was recorded in wrong order\n')
        state['onbase'][3] = 'walsj108'
        state['onbase'][2] = 'pickc103'
    elif ((gameID == 'NY1191606210') and (eventnum == 8)):  
        print('\t\tExplanation: double play with baserunners, batter kaufb101 reaches 1st\n')
        state['nOuts'] += -1
    elif ((gameID == 'PHI191710022') and (eventnum == 25)):  
        print('\t\tExplanation: zimmh101 caught at home despite fielding error\n')
        state['score'][0] += -1
        state['nOuts'] += 1
    elif ((gameID == 'CHA191806200') and (eventnum == 67)):  
        print('\t\tExplanation: colle101 still out at 3rd despite fielding error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'CHA191907100') and (eventnum == 52)):  
        print('\t\tExplanation: mcmuf101 scores on fielding error\n')
        state['nOuts'] += -1
        state['score'][1] += 1
    elif ((gameID == 'DET191906021') and (eventnum == 28)):  
        print('\t\tExplanation: felsh101 scores on fielding error\n')
        state['nOuts'] += -1
        state['score'][0] += 1
    elif ((gameID == 'BRO191905060') and (eventnum == 93)):  
        print('\t\tExplanation: double play due to myerh102 out at home, batter malol101 reaches 1st\n')
        state['nOuts'] += -1
    elif ((gameID == 'CHN192008300') and (eventnum == 41)):  
        print('\t\tExplanation: Error occurred after strikeout, robed103 still out\n')
        state['nOuts'] += 1
    elif ((gameID == 'PHI192005060') and (eventnum == 5)):  
        print('\t\tExplanation: cruiw101 scored on fielding error\n')
        state['nOuts'] += -1
        state['score'][0] += 1
    elif ((gameID == 'BOS192206241') and (eventnum == 52)):  
        print('\t\tExplanation: double play due to warda101 out at home, batter hofmf101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'hofmf101'
        state['onbase'][3] = ''
    elif ((gameID == 'DET192205060') and (eventnum == 62)):
        print('\t\tExplanation: cobbt101 still out at 3rd despite fielding error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'NYA192205100') and (eventnum == 79)):
        print('\t\tExplanation: hooph101 still out at home despite fielding error\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'BRO192207290') and (eventnum == 49)):
        print('\t\tExplanation: mitcc102 still out at home despite fielding error\n')
        state['nOuts'] += 1
        state['score'][1] += -1
    elif ((gameID == 'CHN192207070') and (eventnum == 73)):
        print('\t\tExplanation: barbt101 still out at home despite fielding error\n')
        state['nOuts'] += 1
        state['score'][1] += -1
    elif ((gameID == 'CIN192207160') and (eventnum == 35)):  
        print('\t\tExplanation: double play due to hargb101 out at home, batter cavei101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'cavei101'
        state['onbase'][3] = ''
    elif ((gameID == 'SLN192205120') and (eventnum == 62)):  
        print('\t\tExplanation: double play due to fourj101 out at home, batter heatc101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'heatc101'
    elif ((gameID == 'WS1192308022') and (eventnum == 29)):  
        print('\t\tExplanation: double play due to mcmam101 out at home, batter seveh101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'seveh101'
    elif ((gameID == 'NY1192309020') and (eventnum == 66)):  
        print('\t\tExplanation: double play due to nixoa101 out at home, batter boect101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'boect101'
    elif ((gameID == 'PHI192304280') and (eventnum == 68)):
        print('\t\tExplanation: walkc101 still out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'PIT192305240') and (eventnum == 59)):
        print('\t\tExplanation: smitj106 still out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'CHA192407281') and (eventnum == 71)):
        print('\t\tExplanation: warda101 still out at home despite error\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'CHA192407310') and (eventnum == 44)):
        print('\t\tExplanation: fullc102 still out at 3rd despite error\n')
        state['nOuts'] += 1
    elif ((gameID == 'DET192409130') and (eventnum == 54)):
        print('\t\tExplanation: rices101 still out at home despite error\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'NY1192408041') and (eventnum == 7)):
        print('\t\tExplanation: oconj102 still out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'SLN192405180') and (eventnum == 20)):
        print('\t\tExplanation: dyere101 still out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'DET192506030') and (eventnum == 66)):
        print('\t\tExplanation: double play due to woodl101 out at home, batter hanef101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'hanef101'
    elif ((gameID == 'DET192507081') and (eventnum == 64)):
        print('\t\tExplanation: cobbt101 still out at home despite error\n')
        state['nOuts'] += 1
        state['score'][1] += -1
    elif ((gameID == 'BRO192510030') and (eventnum == 25)):
        print('\t\tExplanation: higha101 still out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] += ''
    elif ((gameID == 'BSN192505280') and (eventnum == 6)):     
        print('\t\tExplanation: meusi101 still out at home despite error\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'CHA192708240') and (eventnum == 33)):     
        print('\t\tExplanation: hofmf101 still out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'CLE192707041') and (eventnum == 16)):
        print('\t\tExplanation: thomt103 still out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'NY1192705170') and (eventnum == 15)):  
        print('\t\tExplanation: double play due to harpg101 out at home, batter hambj101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'hambj101'
    elif ((gameID == 'PHI192707021') and (eventnum == 15)):          
        print('\t\tExplanation: double play due to barrb104 out at 2nd, batter hargc101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'hargc101'
        state['onbase'][2] = ''
    elif ((gameID == 'PHI192806161') and (eventnum == 98)):
        print('\t\tExplanation: whitp103 still out at home despite error\n')
        state['nOuts'] += 1
        state['score'][1] += -1
    elif ((gameID == 'CLE192909250') and (eventnum == 19)):
        print('\t\tExplanation: sewej101 still out at home despite error\n')
        state['nOuts'] += 1
        state['score'][1] += -1
    elif ((gameID == 'BRO192906181') and (eventnum == 60)):
        print('\t\tExplanation: rouse101 still out at home despite error\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'NY1192905040') and (eventnum == 9)):        
        print('\t\tExplanation: ott-m101 still out at home despite error\n')
        state['nOuts'] += 1
        state['score'][1] += -1
    elif ((gameID == 'PHA193005090') and (eventnum == 6)):        
        print('\t\tExplanation: jamic101 still out at home despite error\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'BOS193109211') and (eventnum == 41)):
        print('\t\tExplanation: schuf101 still out at home despite error\n')
        state['nOuts'] += 1
        state['onbase'][1] = ''
        state['score'][0] += -1        
    elif ((gameID == 'BOS193205170') and (eventnum == 41)):          
        print('\t\tExplanation: double play due to tateb101 out at 3nd, batter olsom101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'olsom101'
        state['onbase'][3] = ''
    elif ((gameID == 'BOS193207150') and (eventnum == 29)):        
        print('\t\tExplanation: kresr101 out at home despite error\n')
        state['nOuts'] += 1
        state['onbase'][2] = ''
        state['score'][0] += -1
    elif ((gameID == 'NYA193204300') and (eventnum == 14)):        
        print('\t\tExplanation: crosf103 out at home despite error\n')
        state['nOuts'] += 1
        state['onbase'][1] = ''
        state['score'][1] += -1
    elif ((gameID == 'PHI193204300') and (eventnum == 42)):      
        print('\t\tExplanation: lopea102 out at home despite error\n')
        state['nOuts'] += 1
        state['onbase'][1] = ''
        state['score'][0] += -1
    elif ((gameID == 'BSN193306220') and (eventnum == 18)):            
        print('\t\tExplanation: urbab101 out at home despite error\n')        
        state['nOuts'] += 1
        state['onbase'][1] = ''
        state['score'][1] += -1
    elif ((gameID == 'SLN193304210') and (eventnum == 16)):
        print('\t\tExplanation: martp103 out at home despite error\n')  
        state['nOuts'] += 1
        state['onbase'][1] = ''
        state['score'][1] += -1
    elif ((gameID == 'PHA193408070') and (eventnum == 36)):
        print('\t\tExplanation: double play due to cookd102 out at home, batter portd101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'portd101'
        state['onbase'][3] = ''
    elif ((gameID == 'WS1193409292') and (eventnum == 28)):
        print('\t\tExplanation: batter hoagm101 out at 3rd despite error\n')
        state['nOuts'] += 1
    elif ((gameID == 'BOS193507071') and (eventnum == 3)):
        print('\t\tExplanation: double play due to mosew101 out at home, batter johnb102 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'johnb102'
        state['onbase'][3] = ''
    elif ((gameID == 'DET193506230') and (eventnum == 80)):
        print('\t\tExplanation: double play due to schuf101 out at home, batter kuhej101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'kuhej101'
        state['onbase'][2] = ''
    elif ((gameID == 'SLN193506090') and (eventnum == 63)):
        print('\t\tExplanation: double play due to medwj101 out at home, batter delab102 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'delab102'
        state['onbase'][3] = ''
    elif ((gameID == 'SLA193606270') and (eventnum == 57)):
        print('\t\tExplanation: double play due to lazzt101 out at home, batter crosf103 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'crosf103'
        state['onbase'][3] = ''
    elif ((gameID == 'SLA193607030') and (eventnum == 22)):
        print('\t\tExplanation: double play due to hemsr101 out at 3rd, tietl101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'tietl101'
        state['onbase'][2] = ''
    elif ((gameID == 'WS1193607130') and (eventnum == 13)):
        print('\t\tExplanation: bottj101 out at home despite error\n')
        state['nOuts'] += -1
        state['onbase'][2] = ''
        state['score'][0] += 1
    elif ((gameID == 'PHA193705020') and (eventnum == 14)):
        print('\t\tExplanation: double play due to blueo102 out at 3rd, cromh101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'cromh101'
        state['onbase'][3] = ''
    elif ((gameID == 'BRO193708210') and (eventnum == 29)):
        print('\t\tExplanation: double play due to manuh101 out at home on runner interference, batter phelb101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'phelb101'
        state['onbase'][3] = ''
    elif ((gameID == 'CIN193708070') and (eventnum == 50)):
        print('\t\tExplanation: strikeout error was on baserunning, batter winst101 still out\n')
        state['nOuts'] += 1
    elif ((gameID == 'SLN193705312') and (eventnum == 9)):
        print('\t\tExplanation: double play due to guttd101 out at home, batter martp103 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'martp103'
        state['onbase'][2] = ''
    elif ((gameID == 'CIN193809070') and (eventnum == 50)):
        print('\t\tExplanation: strikeout error was on baserunning, batter rizzj101 still out\n')
        state['nOuts'] += 1
    elif ((gameID == 'SLN193805140') and (eventnum == 47)):
        print('\t\tExplanation: The ball struck the facing of the upper deck and caromed back onto the field; it was ruled in play and Dusty Cooke. Cooke reached 3B for a triple; the Reds protested the game, which was upheld; Cooke was awarded a home run by NL President Ford Frick, the game was replayed in its entirety on August\n')
        state['score'][0] += 1
    elif ((gameID == 'CHA193908140') and (eventnum == 40)):
        print('\t\tExplanation: kreem101 out at home on stolen base attempt\n')
        state['nOuts'] += 1
    elif ((gameID == 'PIT193909212') and (eventnum == 41)):
        print('\t\tExplanation: muelr101 still out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'NYA194005040') and (eventnum == 75)):
        print('\t\tExplanation: crosf103 safe due to error on pickoff attempt\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'crosf103'
    elif ((gameID == 'SLA194005190') and (eventnum == 58)):
        print('\t\tExplanation: second out of DP was siebd102 out at home, batter chaps101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'chaps101'
        state['onbase'][3] = ''
    elif ((gameID == 'BRO194009152') and (eventnum == 46)):
        print('\t\tExplanation: batter freyl101 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'PIT194006080') and (eventnum == 32)):
        print('\t\tExplanation: may-p101 scores on error at home\n')
        state['nOuts'] += -1
        state['score'][0] += 1
    elif ((gameID == 'BOS194106240') and (eventnum == 20)):
        print('\t\tExplanation: pytlf101 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['score'][1] += -1
    elif ((gameID == 'BOS194109150') and (eventnum == 27)):
        print('\t\tExplanation: second out of DP was finnl101 out at home, batter willt103 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'willt103'
    elif ((gameID == 'SLA194104270') and (eventnum == 65)):
        print('\t\tExplanation: soltm101 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'WS1194109242') and (eventnum == 63)):
        print('\t\tExplanation: batter laynh101 out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'SLN194109211') and (eventnum == 62)):
        print('\t\tExplanation: second out of DP was sturb101 out at home, batter hacks101 reaches 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'hacks101'
    elif ((gameID == 'NYA194204290') and (eventnum == 65)):
        print('\t\tExplanation: batter swifb102 out at home despite error\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'NYA194206072') and (eventnum == 74)):
        print('\t\tExplanation: henrt101 safe at 1st after pickoff error\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'henrt101'
    elif ((gameID == 'DET194307051') and (eventnum == 73)):
        print('\t\tExplanation: batter waked101 out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'CIN194407050') and (eventnum == 38)):
        print('\t\tExplanation: batter marsm102 out at home despite error\n')
        state['nOuts'] += 1
        state['score'][1] += -1
    elif ((gameID == 'NYA194508261') and (eventnum == 64)):
        print('\t\tExplanation: second out of DP was stirs101 out at home, batter stait101 advances to 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'stait101'
        state['onbase'][3] = ''
    elif ((gameID == 'NYA194605120') and (eventnum == 54)):
        print('\t\tExplanation: second out of DP was peskj101 out at home, batter doerb101 advances to 1st\n')
        state['nOuts'] += -1
        state['onbase'][1] = 'doerb101'
        state['onbase'][3] = ''
    elif ((gameID == 'PHI194708110') and (eventnum == 6)):        
        print('\t\tExplanation: batter marsw101 out at home despite error\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'CHA194908250') and (eventnum == 10)):   
        print('\t\tExplanation: ostrj102 out at home despite error\n')
        state['nOuts'] += 1
        state['onbase'][1] = ''
        state['score'][1] += -1
    elif ((gameID == 'PHI194909042') and (eventnum == 76)):   
        print('\t\tExplanation: batter sisld101 out at home despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'CLE195108230') and (eventnum == 18)):   
        print('\t\tExplanation: rizzp101 out on failed advance to 3rd, baueh101 stays at 3rd, collj101 reaches 1st on FC \n')
        state['nOuts'] += 1
        state['onbase'][3] = 'baueh101'
        state['onbase'][1] = 'collj101'
    elif ((gameID == 'CIN195106150') and (eventnum == 25)):
        print('\t\tExplanation: wyroj101 out on failed advance to 3rd, adamb103 stays at 3rd, batter klust101 reaches 1st on FC \n')
        state['nOuts'] += 1
        state['onbase'][3] = 'adamb103'
        state['onbase'][1] = 'klust101'
    elif ((gameID == 'NYA195306240') and (eventnum == 54)):
        print('\t\tExplanation: batter minom101 out at home despite error\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'BRO195308161') and (eventnum == 16)):
        print('\t\tExplanation: campr102 picked off at 1st, robij103 stays at 2nd\n')
        state['nOuts'] += 1
        state['onbase'][2] = 'robij103'
        state['onbase'][1] = ''
    elif ((gameID == 'SLN195305220') and (eventnum == 6)):
        print('\t\tExplanation: batter musis101 out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'CHN195705121') and (eventnum == 21)):
        print('\t\tExplanation: batter robif103 out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'KC1195908100') and (eventnum == 51)):  
        print('\t\tExplanation: colar101 scores due to obstruction by catcher housf103\n')
        state['score'][0] += 1
    elif ((gameID == 'CLE196109200') and (eventnum == 22)):
        print('\t\tExplanation: batter kubij101 out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'BAL196308220') and (eventnum == 38)):
        print('\t\tExplanation: batter snydr102 out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'LAA196308080') and (eventnum == 34)):
        print('\t\tExplanation: batter morab101 out at 2nd despite error\n')
        state['nOuts'] += 1
        state['onbase'][2] = ''
    elif ((gameID == 'LAN196706040') and (eventnum == 54)):
        print('\t\tExplanation: bailb103 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['onbase'][1] = ''
        state['score'][1] += -1
    elif ((gameID == 'MIN196805270') and (eventnum == 21)):
        print('\t\tExplanation: tovac101 steals home and scores despite error\n')
        state['score'][1] += 1
    elif ((gameID == 'HOU196808110') and (eventnum == 81)):
        print('\t\tExplanation: menkd101 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['score'][1] += -1
    elif ((gameID == 'BOS196908190') and (eventnum == 12)):
        print('\t\tExplanation: perrj102 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'OAK196906210') and (eventnum == 99)):
        print('\t\tExplanation: olivt102 scores despite error\n')
        #state['nOuts'] += 1
        state['onbase'][1] = state['onbase'][2] #'tovac101'
        state['onbase'][2] = ''
        state['score'][0] += 1
    elif ((gameID == 'HOU197005230') and (eventnum == 43)):
        print('\t\tExplanation: may-l101 hits single but tries to advance to 3rd and is thrown out\n')
        state['nOuts'] += 1
    elif ((gameID == 'PHI197006090') and (eventnum == 5)):
        print('\t\tExplanation: doyld102 out at 3rd despite error')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'CHA197305050') and (eventnum == 41)):
        print('\t\tExplanation: jetej101 hits single but tries to advance to 3rd and is thrown out\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'ATL197304250') and (eventnum == 79)):
        print('\t\tExplanation: baked002 scores and DP is baked002 out at home and johnd105 out at 2nd\n')
        state['nOuts'] += 1
        state['score'][1] += -1
    elif ((gameID == 'SFN197408230') and (eventnum == 57)):
        print('\t\tExplanation: maddg001 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['score'][1] += -1
    elif ((gameID == 'BOS197504180') and (eventnum == 36)):
        print('\t\tExplanation: lynnf001 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['score'][1] += -1
    elif ((gameID == 'DET197508290') and (eventnum == 43)):
        print('\t\tExplanation: batter leflr101 out at 3rd despite error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'TEX197506280') and (eventnum == 37)):
        print('\t\tExplanation: hower001 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['score'][1] += -1
    elif ((gameID == 'HOU197507250') and (eventnum == 79)):
        print('\t\tExplanation: hower001 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'SLN197604190') and (eventnum == 127)):
        print('\t\tExplanation: batter millf105 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'CAL197704220') and (eventnum == 26)):
        print('\t\tExplanation: batter coopc001 out at 2nd despite error\n')
        state['nOuts'] += 1
        state['onbase'][2] = ''
    elif ((gameID == 'KCA197708300') and (eventnum == 37)):
        print('\t\tExplanation: batter patef101 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['score'][1] += -1        
    elif ((gameID == 'DET197809180') and (eventnum == 104)):
        print('\t\tExplanation: burlr001 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'KCA197804150') and (eventnum == 5)):
        print('\t\tExplanation: batter thora001 out at 2nd despite error\n')
        state['nOuts'] += 1
        state['onbase'][2] = ''
    elif ((gameID == 'KCA197807030') and (eventnum == 51)):
        print('\t\tExplanation: andej001 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['score'][0] += -1
    elif ((gameID == 'CHN197807192') and (eventnum == 26)):
        print('\t\tExplanation: grosg001 steals home despite error (and scores)\n')
        #state['onbase'][2] = ''
        state['score'][1] += 1
    elif ((gameID == 'CIN197807170') and (eventnum == 70)):
        print('\t\tExplanation: bencj101 out at home despite error (and does not score)\n')
        state['nOuts'] += 1
        state['score'][1] += -1
    elif ((gameID == 'HOU197808230') and (eventnum == 11)):
        print('\t\tExplanation: kingd001 out at 3rd despite error\n')
        state['nOuts'] += 1
    elif ((gameID == 'CHN197907221') and (eventnum == 11)):
        print('\t\tExplanation: martj001 out at 3rd despite error\n')
        state['nOuts'] += 1
    elif ((gameID == 'LAN197904140') and (eventnum == 46)):
        print('\t\tExplanation: thomg101 scored despite error\n')
        state['score'][1] += 1
    elif ((gameID == 'SDN197906170') and (eventnum == 52)):
        print('\t\tExplanation: sizet101 out at home despite error (and does not score)\n')
        state['score'][0] += -1
        state['nOuts'] += 1
    elif ((gameID == 'SEA198004230') and (eventnum == 39)):
        print('\t\tExplanation: hendr001 out at 3rd despite error\n')
        state['nOuts'] += 1
    elif ((gameID == 'TEX198005190') and (eventnum == 114)):
        print('\t\tExplanation: cruzt001 out at home despite error (and does not score)\n')
        state['score'][0] += -1
        state['nOuts'] += 1
    elif ((gameID == 'TEX198006190') and (eventnum == 87)):
        print('\t\tExplanation: oglib001 out at home despite error (and does not score)\n')
        state['score'][0] += -1
        state['nOuts'] += 1
    elif ((gameID == 'SLN198109290') and (eventnum == 37)):
        print('\t\tExplanation: portd001 out at home despite error (and does not score)\n')
        state['score'][1] += -1
        state['nOuts'] += 1        
    elif ((gameID == 'CLE198209070') and (eventnum == 50)):
        print('\t\tExplanation: harrt001 out at home despite error (and does not score)\n')
        state['score'][1] += -1
        state['nOuts'] += 1        
    elif ((gameID == 'ATL198307180') and (eventnum == 13)):
        print('\t\tExplanation: wilsm001 out at 3rd despite error\n')
        state['nOuts'] += 1   
    elif ((gameID == 'CIN198307090') and (eventnum == 46)):
        print('\t\tExplanation: hayev001 caught stealing home, out not recorded for some reason\n')
        state['nOuts'] += 1
    elif ((gameID == 'SEA198405010') and (eventnum == 31)):
        print('\t\tExplanation: hatcm001 out at home despite error (and does not score)\n')
        state['score'][0] += -1
        state['nOuts'] += 1
    elif ((gameID == 'HOU198405110') and (eventnum == 4)):
        print('\t\tExplanation: bassk001 out at 3rd despite error\n')
        state['nOuts'] += 1
    elif ((gameID == 'NYA198504180') and (eventnum == 54)):
        print('\t\tExplanation: kittr001 out at 3rd despite error\n')
        state['nOuts'] += 1
    elif ((gameID == 'SEA198709260') and (eventnum == 26)):
        print('\t\tExplanation: bradp001 out at 3rd despite error\n')
        state['nOuts'] += 1
    elif ((gameID == 'ATL198706180') and (eventnum == 26)):
        print('\t\tExplanation: parkd001 out at home despite error (and does not score)\n')
        state['score'][0] += -1
        state['nOuts'] += 1
    elif ((gameID == 'MON198708210') and (eventnum == 20)):
        print('\t\tExplanation: nichr001 out at 3rd despite error\n')
        state['nOuts'] += 1   
    elif ((gameID == 'PHI198705060') and (eventnum == 18)):
        print('\t\tExplanation: grosg001 out at 3rd despite error\n')
        state['nOuts'] += 1           
    elif ((gameID == 'OAK198804180') and (eventnum == 48)):
        print('\t\tExplanation: ray-j001 out at home despite error (and does not score)\n')
        state['score'][0] += -1
        state['nOuts'] += 1        
    elif ((gameID == 'LAN198808030') and (eventnum == 94)):
        print('\t\tExplanation: reedj001 out at home despite error (and does not score)\n')
        state['score'][0] += -1
        state['nOuts'] += 1       
    elif ((gameID == 'SEA198908110') and (eventnum == 42)):
        print('\t\tExplanation: caldi001 safe at home despite error (and scores)\n')
        state['score'][0] += 1
        state['nOuts'] += 1       
    elif ((gameID == 'TEX198909020') and (eventnum == 75)):
        print('\t\tExplanation: franj002 out at home despite error (and does not score)\n')
        state['score'][1] += -1
        state['nOuts'] += 1     
    elif ((gameID == 'MON199009130') and (eventnum == 7)):
        print('\t\tExplanation: josef001 out at home despite error (and does not score)\n')
        state['score'][0] += -1
        state['nOuts'] += 1
    elif ((gameID == 'CLE199107300') and (eventnum == 19)):
        print('\t\tExplanation: sojol001 out at home despite error (and does not score)\n')
        state['score'][0] += -1
        state['nOuts'] += 1
    elif ((gameID == 'TEX199110040') and (eventnum == 54)):
        print('\t\tExplanation: pettg001 save at home despite error (and scores)\n')
        state['score'][1] += 1
        state['nOuts'] += -1    
    elif ((gameID == 'CHN199106280') and (eventnum == 29)):
        print('\t\tExplanation: vizcj001 out at home despite error (and does not score)\n')
        state['score'][1] += -1
        state['nOuts'] += 1    
    elif ((gameID == 'CIN199107120') and (eventnum == 58)):
        print('\t\tExplanation: saboc001 out at home despite error (but somehow score got recorded correctly?)\n')
        state['nOuts'] += 1
    elif ((gameID == 'DET199208100') and (eventnum == 37)):
        print('\t\tExplanation: brogr001 out at 3rd despite error\n')
        state['nOuts'] += 1        
    elif ((gameID == 'PIT199308110') and (eventnum == 57)):
        print('\t\tExplanation: minob001 out at home despite error (and does not score)\n')
        state['score'][1] += -1
        state['nOuts'] += 1            
    elif ((gameID == 'NYN199407200') and (eventnum == 85)):
        print('\t\tExplanation: thomr004 out at home despite error (and does not score)\n')
        state['score'][1] += -1
        state['nOuts'] += 1
    elif ((gameID == 'OAK199606220') and (eventnum == 37)):
        print('\t\tExplanation: lovut001 out at home despite error (and does not score)\n')
        state['score'][1] += -1
        state['nOuts'] += 1
    elif ((gameID == 'LAN200209170') and (eventnum == 40)):
        print('\t\tExplanation: belta001 out at 3rd despite error (and does not score)\n')
        state['nOuts'] += 1
    elif ((gameID == 'SFN200205310') and (eventnum == 86)):
        print('\t\tExplanation: hollt001 out at home despite error (and does not score)\n')
        state['score'][0] += -1
        state['nOuts'] += 1        
    elif ((gameID == 'TOR200407240') and (eventnum == 45)):
        print('\t\tExplanation: crawc002 safe at home despite error (and scores)\n')
        state['score'][0] += 1
        state['nOuts'] += -1                
    elif ((gameID == 'LAN200507030') and (eventnum == 91)):
        print('\t\tExplanation: cruzj004 out at 3rd despite error\n')
        state['nOuts'] += 1
    elif ((gameID == 'SFN201407300') and (eventnum == 64)):
        print('\t\tExplanation: Weird thing, snidt001 caught stealing at 3rd and sancg001 picked off returning to 3rd...\n')
        print('\t\t\tExplanation: on ball four, Travis Snider walked towards 3B, evidently thinking there was a runner on 1B and he was forced to move; when the Giants went after Snider, Gaby Sanchez ran down the 3B line\n')
        state['onbase'][2] = ''
        state['onbase'][3] = ''
        state['nOuts'] += 2
    elif ((gameID == 'LAN201504150') and (eventnum == 55)):
        print('\t\tExplanation: canor001 caught stealing home on walk\n')
        state['onbase'][3] = ''
        state['nOuts'] += 1
    elif ((gameID == 'CHN201708160') and (eventnum == 40)):
        print('\t\tExplanation: lackj001 caught stealing 3rd\n')
        state['onbase'][2] = ''
        state['nOuts'] += 1
    elif ((gameID == 'PHI201708250') and (eventnum == 39)):
        print('\t\tExplanation: galvf001 caught stealing home\n')
        state['onbase'][3] = ''
        state['nOuts'] += 1        
    elif ((gameID == 'MIL202204170') and (eventnum == 12)):
        print('\t\tExplanation: yelic001 safe at home despite error\n')
        state['onbase'][3] = ''
        state['score'][1] += 1

    return state

print('ok')

ok


In [79]:
# #baserunning_fielder_error_re = re.compile('(\d|B|H)\(\d*(E|PB)\d*\)')  # had to add the possibility of (NR)(*E)
baserunning_fielder_error_re = re.compile('(?<!\))\(\d*E\d*\)')

def advance_runner(start_base, end_base, state, printme):
#     if (printme):
#         print('\t\t\t^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
#         print('\t\t\tAdvancing runner from {0:} to {1:}!'.format(ordinaltg(start_base), ordinaltg(end_base)))
#         print('\t\t\t\tBases were: {0:}'.format(state['onbase']))
    if (end_base == 4):
        state['score'][vh] += 1
    else:
        if (start_base == 0):
            state['onbase'][end_base] = state['batterID']
            state['manual_batter_advance'] = True
        else:
            state['onbase'][end_base] = state['onbase'][start_base]
    if (start_base == 0):
        state['manual_batter_advance'] = True
    elif (start_base == end_base): # don't clear base if runner ended at same base
        pass
    else:
        state['onbase'][start_base] = ''
#     if (printme):
#         print('\t\t\t\tBases are now: {0:}'.format(state['onbase']))    
#         print('\t\t\t^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
        
    return state

def runner_out(start_base, end_base, state, printme):
#     if (printme):
#         print('\t\t\t^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
#         pprint(state)
#         print('\n')
#         print('\t\t\t Runner coming from {0:} is out at {1:}!'.format(ordinaltg(start_base), ordinaltg(end_base)))
#         print('\t\t\t\tBases were: {0:}'.format(state['onbase']))
    state['nOuts'] += 1
    if (end_base < 4):
        state['onbase'][end_base] = ''
    if (start_base == 0):
        state['manual_batter_advance'] = True
    else:
        state['onbase'][start_base] = ''
        
#     if (printme):
#         print('\n')
#         print('\t\t\t\tBases are now: {0:}'.format(state['onbase']))
#         print('\t\t\t^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')

    return state

def parse_baserunning(baserunning_str, printme = False):
    
    baseruns = []
    
    baserunning_str_list = sorted(baserunning_str.split(';'), reverse=True)
#     if (printme):
#         print('\tSTATE OF BASES BEFORE: {0:}'.format(state['onbase']))
#         print('\tBaserunning: {0:} | {1:}'.format(baserunning_str, baserunning_str_list))
    
    for this_baserun_str in baserunning_str_list:
        this_baserun = {}
#         if (printme):
#             print('xxxxxxxxxxxxxxxxxxxxxx')
#             print('this_baserun_str = ',this_baserun_str)
        this_baserun['start_base'] = 0 if this_baserun_str[0] == 'B' else int(this_baserun_str[0])
        
        this_baserun['success'] = True if this_baserun_str[1] == '-' else False
        if (re.search(baserunning_fielder_error_re, this_baserun_str) != None):
            this_baserun['success'] = True

        this_baserun['end_base'] = 4 if this_baserun_str[2] == 'H' else int(this_baserun_str[2])
#         if (printme):
#             print('this_baserun = ',this_baserun)
        baseruns.append(this_baserun)
        
    baseruns = sorted(baseruns, key=lambda d: d['start_base'], reverse=True)
    
#     if (printme):
#         print('`````````````````````')
#         print(baseruns)
#         print('`````````````````````')
    
    return baseruns

def baserunning(baserunning_str, state, printme = False):
    baseruns = parse_baserunning(baserunning_str, printme)
#     if (printme):
#         print(baseruns)
    for this_baserun in baseruns:
#         if (printme):
#             print('/////////////////////')
#             print(this_baserun)
        if (this_baserun['success']):
            state = advance_runner(this_baserun['start_base'], this_baserun['end_base'], state, printme)
        else:
            state = runner_out(this_baserun['start_base'], this_baserun['end_base'], state, printme)
#             if (printme):
#                 print('\n')
#                 pprint(state)
#                 print('/////////////////////')
    return state


def steal_bases(stealstring, state, printme = False, baserunning_str = ''):
#     if (printme):
#         print('\tggggggggggggggggggggg')
#         print('\t\tstate of bases after this steal: {0:}'.format(state['onbase']))
        
    steal_list = sorted(stealstring.split(';'), reverse=True)

#     if (printme):
#         print('\tBases: {0:} | {1:}'.format(steal_list, stealstring))
    
    baseruns = []
    if (baserunning_str != ''):
        baseruns = parse_baserunning(baserunning_str, printme)
#         if (printme):
#             print('\t\tBaseruns: {0:} |{1:}'.format(baserunning_str, baseruns))
    
    highest_base_stolen = np.max([int(x[2].replace('B','0').replace('H','4')) for x in steal_list])
    lowest_base_stolen = np.min([int(x[2].replace('B','0').replace('H','4')) for x in steal_list])
#     if (printme):
#         print('Highest stolen base: {0:}, lowest stolen base: {1:}'.format(ordinaltg(highest_base_stolen), ordinaltg(lowest_base_stolen)))
    
    baseruns_beyond = [x for x in baseruns if x['start_base'] > highest_base_stolen]
#     if ((printme) & (len(baseruns_beyond) > 0)):
#         print('Baserunning beyond highest stolen base of {0:}: {1:}'.format(ordinaltg(highest_base_stolen), baseruns_beyond))
    for x in baseruns_beyond: 
        if (x['success']):
            state = advance_runner(x['start_base'], x['end_base'], state, printme)
        else:
            state = runner_out(x['start_base'], x['end_base'], state, printme)

    for this_steal in steal_list:
        base_stolen = int(this_steal[2].replace('B','0').replace('H','4'))
        source = -3
        for b in range(base_stolen-1, 0, -1):
            if (state['onbase'][b] != ''):
                source = b
                break
        if (source == -3): # if all prior bases are empty, batter steals
            source = 0
#         if (printme):
#             print('{0:} base is stolen |{1:}|'.format(ordinaltg(base_stolen), this_steal))
#             print('Source was {0:} (onbase = {1:})'.format(ordinaltg(source), state['onbase']))
        
        baseruns_here = [x for x in baseruns if x['start_base'] == source]
        if (len(baseruns_here) > 0):
#             if (printme):
#                 print('WE HAVE EXPLICIT BASERUNNING for the runner who started on {0:}:'.format(ordinaltg(source)))
            for this_explicit_baserun in baseruns_here:
#                 if (printme):
#                     print('\t\t<<<<<{0:}>>>>>'.format(this_explicit_baserun))
                if (this_explicit_baserun['success']):
                    state = advance_runner(this_explicit_baserun['start_base'], this_explicit_baserun['end_base'], state, printme)
                else:
                    state = runner_out(this_explicit_baserun['start_base'], this_explicit_baserun['end_base'], state, printme)
        else:
            state = advance_runner(source, base_stolen, state, printme)
    
    baseruns_before = [x for x in baseruns if x['start_base'] < lowest_base_stolen-1]
#     if ((printme) & (len(baseruns_before) > 0)):
#         print('Baserunning prior to lowest stolen base of {0:}: {1:}'.format(ordinaltg(lowest_base_stolen), baseruns_before))
    for x in baseruns_before: 
        if (x['success']):
            state = advance_runner(x['start_base'], x['end_base'], state, printme)
        else:
            state = runner_out(x['start_base'], x['end_base'], state, printme)
            
#     if (printme):
#         print('\t\tstate of bases after this steal: {0:}'.format(state['onbase']))
#         print('\tggggggggggggggggggggg')
    return state

def catch_stealing(catchstring, state, printme = False, baserunning_str = ''):
#     if (printme):
#         print('\thhhhhhhhhhhhhhhhhhh')
#         print('\tEvent: {0:}'.format(catchstring))
    caught_list = sorted(catchstring.split(';'), reverse=True)

#     if (printme):
#         print('\Caught stealing list: {0:} | {1:}'.format(caught_list, catchstring))
    
    baseruns = []
    if (baserunning_str != ''):
        baseruns = parse_baserunning(baserunning_str, printme)
#         if (printme):
#             print('\t\tBaseruns: {0:} |{1:}'.format(baserunning_str, baseruns))
    
    highest_base_caught_at = np.max([int(x[2].replace('B','0').replace('H','4')) for x in caught_list])
    lowest_base_caught_at = np.min([int(x[2].replace('B','0').replace('H','4')) for x in caught_list])
    
    baseruns_beyond = [x for x in baseruns if x['start_base'] > highest_base_caught_at-1]
    if (len(baseruns_beyond) > 0):
#         if (printme):
#             print('Baserunning from at or beyond highest base caught at of {0:}: {1:}'.format(ordinaltg(highest_base_caught_at), baseruns_beyond))
        for x in baseruns_beyond: 
            if (x['success']):
                state = advance_runner(x['start_base'], x['end_base'], state, printme)
            else:
                state = runner_out(x['start_base'], x['end_base'], state, printme)

    for this_caught in caught_list:
        caught_at = int(this_caught[2].replace('B','0').replace('H','4'))

        source = -3
        for b in range(caught_at-1, 0, -1):
            if (state['onbase'][b] != ''):
                source = b
                break
        if (source == -3): # if all prior bases are empty, batter steals
            source = 0

        baseruns_here = [x for x in baseruns if x['start_base'] == source]
        if (len(baseruns_here) > 0):
#             if (printme):
#                 print('WE HAVE EXPLICIT BASERUNNING for the runner who started on {0:}:'.format(ordinaltg(source)))
            for this_explicit_baserun in baseruns_here:
#                 if (printme):
#                     print('\t\t<<<<<{0:}>>>>>'.format(this_explicit_baserun))
                if (this_explicit_baserun['success']):
                    state = advance_runner(this_explicit_baserun['start_base'], this_explicit_baserun['end_base'], state, printme)
                else:
                    state = runner_out(this_explicit_baserun['start_base'], this_explicit_baserun['end_base'], state, printme)
        else:
            if (re.search(baserunning_fielder_error_re, this_caught) != None):
#                 if (printme):
#                     print('Failed caught stealing at {0:} from source at {1:} |{2:}|'.format(ordinaltg(caught_at), source, this_caught))
                state = advance_runner(source, caught_at, state, printme)
            else:
#                 if (printme):
#                     print('Runner was caught at {0:} from source at {1:} |{2:}|'.format(ordinaltg(caught_at), source, this_caught))
                state = runner_out(source, caught_at, state, printme)
    
    baseruns_before = [x for x in baseruns if x['start_base'] < lowest_base_caught_at-1]
#    if ((printme) & (len(baseruns_before) > 0)):
    if (len(baseruns_before) > 0):
#         if (printme):
#             print('Baserunning prior to lowest base caught at of {0:}: {1:}'.format(ordinaltg(lowest_base_caught_at), baseruns_before))
        for x in baseruns_before: 
            if (x['success']):
                state = advance_runner(x['start_base'], x['end_base'], state, printme)
            else:
                state = runner_out(x['start_base'], x['end_base'], state, printme)
                        
#     if (printme):
#         print('\thhhhhhhhhhhhhhhhhhh')
    return state

def pickoff(pickoff_str, state, printme = False, baserunning_str = ''):
#     if (printme):
#         print('wfe')
    pickoff_list = sorted(pickoff_str.split(';'), reverse = True)

#     if (printme):
#         print('\t\tPickoffs: {0:} |{1:}'.format(pickoff_str, pickoff_list))
    
    baseruns = []
    if (baserunning_str != ''):
        baseruns = parse_baserunning(baserunning_str, printme)
#         if (printme):
#             print('\t\tBaseruns: {0:} |{1:}'.format(baserunning_str, baseruns))
    for this_pickoff in pickoff_list:
        base_picked_off_at = int(this_pickoff[2].replace('B', '0').replace('H', '4'))
#         if (printme):
#             print('\t\t\tPicked off at {0:.0f} |{1:}|'.format(base_picked_off_at, this_pickoff))
        baseruns_beyond = [x for x in baseruns if x['start_base'] > base_picked_off_at]
        for x in baseruns_beyond:
            if (x['success']):
                state = advance_runner(x['start_base'], x['end_base'], state, printme)
            else:
                state = runner_out(x['start_base'], x['end_base'], state, printme)
        baseruns_here = [x for x in baseruns if x['start_base'] == base_picked_off_at]
        if (len(baseruns_here) > 0):  # explicit baserunning from pickoff base
            for x in baseruns_here:
                if (x['success']):
                    state = advance_runner(x['start_base'], x['end_base'], state, printme)
                else:
                    state = runner_out(x['start_base'], x['end_base'], state, printme)
        else: # no baserunning from pickoff base, eval pickoff only - if error, DON'T ADVANCE RUNNER
            if (not(re.search(baserunning_fielder_error_re, this_pickoff))):
                state = runner_out(base_picked_off_at, base_picked_off_at+1, state, printme)
#             if (re.search(baserunning_fielder_error_re, this_pickoff)):
#                 state = advance_runner(base_picked_off_at, base_picked_off_at+1, state, printme)
#             else:
#                 state = runner_out(base_picked_off_at, base_picked_off_at+1, state, printme)

        baseruns_before = [x for x in baseruns if x['start_base'] < base_picked_off_at]
        for x in baseruns_before:
            if (x['success']):
                state = advance_runner(x['start_base'], x['end_base'], state, printme)
            else:
                state = runner_out(x['start_base'], x['end_base'], state, printme)            
    return state

print('ok')



ok


In [80]:
#def noplay(thisrow, nextrow, prevrow, state, printme = False):
def noplay(noplay_df, state, printme = False):
    
    noplay_skips = {}
    
    # two or more players stayed in as defensive replacements, parsed in previous row 
    noplay_skips.update({'BUF191406270': [68], 'SLF191406100': [84], 'CHA191406180': [81], 'CHA191409222': [52], 'BSN191407270': [27], 'BSN191409190': [65], 'CIN191404140': [88,89,90], 'PIT191407150': [84]}) 
    noplay_skips.update({'CHF191504270': [76], 'CHF191505160': [41], 'BSN191505070': [55], 'CHN191504140': [61], 'CHN191504170': [88], 'CHN191506120': [86], 'CIN191508151': [65], 'BSN191609131': [53], 'SLN191607011': [74]})
    noplay_skips.update({'BOS191709041': [51], 'CLE191707220': [76], 'BRO191706261': [85,86], 'BRO191708210': [108], 'CIN191705310': [55], 'PHI191706201': [59], 'CLE191808120': [63], 'PHI191804300': [79], 'PIT191808100': [54], 'WS1191906040': [46]})
    noplay_skips.update({'BOS192008232': [66], 'CLE192006010': [91], 'SLA192008100': [50], 'SLA192009170': [39], 'PHI192009200': [80]})                     
    noplay_skips.update({'BOS192106030': [72], 'BOS192108050': [78], 'CHA192107071': [104], 'CLE192104220': [69], 'CLE192104250': [12], 'CLE192105110': [40,41], 'CLE192105250': [51], 'CLE192106260': [56], 'CLE192106280': [85,86], 'CLE192106290': [81], 'CLE192106300': [67], 'CLE192107050': [52], 'CLE192107101': [55], 'CLE192107160': [63], 'CLE192108260': [55], 'DET192105150': [87], 'DET192105160': [103], 'DET192109010': [101], 'NYA192107041': [81,82], 'SLA192104140': [79], 'SLA192104150': [71], 'SLA192105280': [29], 'SLA192107171': [56], 'WS1192106070': [77], 'BSN192104190': [79], 'BSN192105302': [99], 'BSN192109280': [83,84], 'NY1192105120': [77], 'NY1192107130': [73], 'PHI192109032': [84], 'PIT192109150': [71], 'SLN192106040': [93]})
    noplay_skips.update({'WS1192208180': [62], 'NY1192207270': [69], 'NY1192207282': [69], 'NY1192208050': [65,66], 'PIT192207060': [75], 'PIT192207170': [53], 'SLN192205200': [55], 'SLN192206240': [46], 'SLN192207080': [78]})
    noplay_skips.update({'CHA192310020': [73], 'CLE192306240': [52], 'DET192305030': [62], 'DET192309270': [59], 'NYA192306200': [77], 'NYA192306300': [70], 'PHA192308060': [81], 'SLA192307151': [62], 'BSN192304191': [89], 'BSN192305160': [99], 'CHN192309032': [54], 'NY1192308231': [100], 'NY1192309290': [40], 'PHI192307050': [57], 'SLN192306170': [81], 'SLN192308090': [95]})
    noplay_skips.update({'CHA192409032': [56], 'CHA192409130': [85], 'CLE192405040': [56], 'CLE192409130': [76], 'DET192408020': [60], 'PHA192405140': [81], 'SLA192409111': [93], 'WS1192405130': [60], 'WS1192407140': [74], 'BSN192407010': [61], 'BSN192409202': [73,74], 'CIN192407162': [92], 'CIN192408180': [86], 'NY1192408080': [79], 'NY1192408090': [73], 'PHI192408070': [68]})
    noplay_skips.update({'BOS192506200': [88], 'CHA192505100': [77], 'CHA192507160': [69], 'CHA192508150': [54], 'CLE192505230': [71], 'CLE192506300': [78], 'DET192505301': [57], 'DET192508250': [51], 'DET192509071': [90], 'DET192509102': [66], 'SLA192504180': [85], 'BSN192507160': [103], 'CIN192509060': [57], 'PHI192505070': [71], 'PHI192508200': [40], 'PHI192509040': [37], 'SLN192506140': [87,88]})
    noplay_skips.update({'CHA192608072': [66], 'CLE192606150': [74], 'DET192607310': [71], 'NYA192605300': [87], 'PHA192604220': [82], 'PHA192607100': [49], 'PHA192608171': [74], 'SLA192609070': [79,80], 'BSN192604220': [67], 'CHN192604290': [75], 'PHI192606100': [92], 'PHI192609182': [61], 'PHI192609210': [61]})
    noplay_skips.update({'BOS192704260': [73], 'BOS192709240': [59], 'CLE192704130': [85], 'CLE192705170': [76], 'CLE192707081': [86,87,88], 'CLE192707092': [60], 'CLE192708290': [60], 'DET192708170': [81], 'PHA192705050': [72], 'BSN192707160': [72], 'NY1192705200': [90], 'PHI192705272': [84]})
    noplay_skips.update({'BRO192805290': [81], 'NY1192806020': [90], 'NY1192806090': [27], 'NY1192806140': [84], 'CHA192905060': [73], 'CLE192907050': [61], 'CLE192907210': [85], 'NYA192906060': [92,93], 'WS1192906221': [55], 'BSN192910051': [52,70], 'CIN192907282': [79], 'NY1192904240': [47], 'PHI192909281': [58], 'PIT192909200': [65]})
    noplay_skips.update({'BOS193005050': [64], 'CHA193006040': [86], 'CLE193006130': [72], 'CLE193007300': [78], 'DET193007260': [57], 'NYA193006232': [60], 'PHA193007030': [59], 'BRO193009060': [81], 'CIN193005050': [75], 'CIN193008171': [71]})
    noplay_skips.update({'CHA193107122': [93], 'SLA193105140': [48], 'SLA193105242': [65], 'BSN193105121': [52], 'PIT193108052': [77], 'NYA193204230': [71], 'WS1193304130': [72], 'CHN193307070': [96], 'CHN193307140': [77], 'CIN193306111': [45], 'CIN193310012': [23], 'PIT193309200': [66]})
    noplay_skips.update({'NYA193408180': [70], 'PHA193406250': [44], 'BRO193409120': [56], 'CIN193409231': [88], 'BOS193509022': [93], 'CLE193507280': [73], 'BRO193506210': [37], 'CHN193508110': [71], 'CLE193607291': [50], 'DET193605210': [42], 'NYA193606301': [82], 'WS1193609230': [71], 'BRO193607280': [74], 'CIN193607052': [98], 'CIN193607140': [97], 'SLN193606020': [61]})
    noplay_skips.update({'CHN193805050': [97], 'CIN193805060': [81], 'PHI193807290': [58], 'PIT193807202': [74], 'NYA193904300': [82], 'NYA193905271': [40], 'WS1193905302': [74,75], 'BSN193909070': [74], 'NY1193906060': [74], 'BRO194006270': [76], 'BRO194008012': [43], 'BSN194006171': [33], 'BSN194009140': [95], 'CIN194009022': [65], 'CIN194009030': [51], 'PHI194009290': [55], 'SLN194008220': [83], 'PHA194109090': [76], 'SLA194104270': [74], 'BRO194109071': [70], 'BSN194109071': [86,87,101], 'BSN194109072': [78], 'CHN194109101': [84], 'CIN194109230': [41], 'NY1194106081': [89], 'PHI194109201': [65], 'SLN194107260': [67], 'SLN194108031': [74], 'BRO194204190': [74], 'BRO194205311': [64], 'PIT194206060': [62], 'PIT194209072': [61], 'PIT194209201': [83], 'SLN194207181': [66]})
    noplay_skips.update({'NYA194309052': [62], 'SLA194306230': [82], 'WS1194305091': [61], 'BRO194307030': [48], 'BSN194308152': [84], 'CHN194307281': [116], 'DET194407180': [76], 'DET194407231': [76,77], 'PHA194408160': [69], 'PHA194408202': [81], 'SLA194406030': [83], 'BSN194407261': [73]})
    noplay_skips.update({'SLA194507012': [95], 'WS1194509151': [62], 'BSN194508052': [87], 'CIN194509292': [68], 'PIT194505301': [57], 'PIT194507152': [83], 'SLA194609022': [109], 'BRO194606230': [76], 'BRO194607010': [22], 'BSN194609152': [69], 'BSN194609170': [71,72], 'CHN194606110': [76], 'CHN194607180': [53,54], 'PHI194605240': [37], 'SLN194608270': [80], 'BOS194704250': [72], 'DET194708090': [77], 'DET194708230': [76], 'NYA194705171': [87], 'BSN194706112': [77], 'BSN194707101': [75], 'CHN194706130': [75], 'CIN194706170': [32,47], 'CIN194708012': [88,89], 'CIN194709100': [63], 'PHI194704272': [76], 'PHI194707030': [61], 'PHI194708140': [75], 'SLN194706230': [36], 'CLE194808081': [80], 'CLE194809112': [81,82,83], 'CLE194809180': [82,83], 'NYA194805152': [54], 'BRO194805190': [22], 'BRO194808211': [94], 'BRO194808230': [116], 'BSN194805231': [46], 'CIN194806122': [96], 'CIN194807190': [91], 'PHI194807312': [62]})
    noplay_skips.update({'BOS194905301': [73,74], 'BOS194908110': [67], 'CLE194907160': [58], 'DET194904300': [78], 'DET194907190': [83,104], 'NYA194906161': [39], 'NYA194910010': [78], 'WS1194907030': [51], 'WS1194907280': [77,86], 'WS1194907300': [99], 'CIN194905170': [76], 'CIN194907270': [76], 'PIT194909182': [54]})
    noplay_skips.update({'DET195006252': [69], 'NYA195004210': [87], 'NY1195006240': [77], 'PIT195008140': [40], 'BOS195106172': [73], 'CHA195106260': [50], 'CLE195104222': [77], 'NYA195109150': [55], 'PHA195107291': [76], 'WS1195105150': [64], 'CLE195206170': [69], 'DET195207292':  [67], 'SLA195204180': [82], 'SLA195205252': [71], 'SLA195207280': [88], 'BOS195305100': [35], 'BOS195306180': [88],  'BOS195308010': [79], 'BOS195308080': [67], 'CHA195309030': [67], 'DET195306100': [65], 'DET195308270': [76,77], 'DET195309220': [86], 'NYA195306181': [59], 'PHA195308171': [75], 'SLA195305250': [71], 'CHN195305230': [84], 'CIN195305270': [63], 'MLN195306190': [87], 'MLN195308220': [57]})
    noplay_skips.update({'BAL195405281': [66], 'BAL195408040': [89], 'CHA195406180': [62], 'DET195406250': [76], 'NYA195406090': [57], 'NYA195406100': [16], 'NYA195408130': [72], 'PHA195406112': [8], 'WS1195409180': [67], 'CIN195406290': [44], 'NY1195406200': [84], 'KC1195506020': [74], 'KC1195507160': [80], 'NYA195509020': [71,72], 'NYA195509111': [72], 'WS1195508070': [65], 'MLN195506220': [67], 'NY1195505301': [75], 'NY1195509051': [95]})
    noplay_skips.update({'KC1195606300': [68], 'WS1195607030': [102], 'CHN195608261': [70], 'PHI195607250': [68], 'BAL195706260': [56], 'BAL195708090': [69], 'BAL195709021': [43], 'BOS195705220': [69], 'CHA195707180': [84], 'CLE195707120': [80], 'CLE195708100': [70], 'CLE195708112': [70], 'NYA195707250': [60], 'NYA195709251': [71], 'WS1195704212': [53], 'BRO195708240': [60], 'PHI195704240': [84]})
    noplay_skips.update({'BAL195805030': [56], 'CLE195805270': [88], 'DET195807290': [62], 'NYA195804190': [67], 'CHN195805020': [56], 'CHN195807162': [67], 'CIN195807240': [64], 'LAN195808130': [82], 'LAN195808270': [96], 'LAN195809261': [82], 'MLN195804150': [61], 'SLN195805040': [67,68]})
    noplay_skips.update({'BAL195905101': [87], 'BAL195905140': [61], 'BAL195907190': [55], 'BOS195907100': [81], 'CLE195907282': [66], 'DET195909220': [78,79], 'KC1195904220': [54], 'CHN195904120': [81], 'CHN195909230': [36], 'CHN195909250': [82], 'CIN195905020': [80], 'LAN195905190': [78], 'LAN195906290': [92], 'MLN195907110': [49], 'PHI195906110': [64], 'SLN195905140': [55], 'SLN195909220': [62]})
    noplay_skips.update({'BAL196005020': [60], 'KC1196005250': [87], 'KC1196006240': [93], 'NYA196008142': [75], 'WS1196006100': [84], 'CHN196007010': [29], 'CHN196007020': [51], 'CHN196007270': [55], 'CIN196005230': [85], 'CIN196008130': [75], 'LAN196006050': [62], 'PHI196005302': [85], 'PHI196006020': [82], 'PHI196006230': [54], 'PHI196007030': [81], 'PHI196008022': [81], 'PIT196005250': [59], 'SLN196004220': [92], 'SLN196005210': [84], 'SLN196006280': [48], 'SLN196008071': [81,82], 'SLN196009160': [50], 'BAL196108100': [52], 'KC1196108201': [67], 'MIN196109260': [68], 'CHN196106270': [81,82], 'CHN196107260': [77], 'CHN196107290': [81], 'CHN196110010': [69], 'CIN196106230': [65], 'PHI196106291': [24], 'PIT196105050': [81], 'SFN196105190': [93], 'SFN196109180': [71], 'SLN196104260': [56], 'SLN196106260': [51], 'SLN196107190': [84]})
    noplay_skips.update({'BAL196208251': [85], 'CHA196205230': [67], 'CLE196204280': [27], 'DET196206210': [76], 'KC1196207060': [75], 'CHN196205132': [68], 'CHN196208220': [77], 'CIN196207270': [88], 'NYN196209182': [88], 'PHI196206221': [49], 'PHI196207290': [45], 'PHI196209210': [55], 'PHI196209220': [60], 'PIT196209180': [99]})
    noplay_skips.update({'BAL196309250': [69], 'WS2196306282': [85], 'CIN196308110': [82], 'CIN196308150': [65], 'CIN196309092': [86], 'CIN196309180': [84], 'HOU196307030': [50], 'MLN196306161': [47], 'NYN196306092': [66], 'SFN196304270': [97], 'SLN196305180': [68], 'SLN196308190': [56]})
    noplay_skips.update({'BAL196405250': [58], 'CLE196405230': [61], 'DET196405311': [51], 'DET196407040': [74], 'DET196409220': [68], 'KC1196407131': [76], 'KC1196407140': [90], 'KC1196407151': [73], 'NYA196405220': [39], 'NYA196409301': [86], 'WS2196405062': [85], 'WS2196405101': [76], 'WS2196406100': [73], 'WS2196406260': [95], 'WS2196408210': [58], 'HOU196409150': [50], 'LAN196405240': [58], 'NYN196408231': [56,57], 'PHI196408180': [68]})
    noplay_skips.update({'BAL196505130': [57], 'BOS196505312': [57], 'CAL196509221': [75], 'MIN196508010': [49], 'NYA196505231': [53], 'NYA196508152': [74], 'WS2196509251': [62], 'WS2196509252': [88], 'CHN196506050': [64], 'CHN196509150': [96,97], 'CHN196509210': [69], 'CIN196507090': [72], 'MLN196508230': [78], 'NYN196507030': [58], 'NYN196509140': [92], 'PIT196505310': [89], 'SFN196508150': [32], 'SFN196509121': [76], 'SLN196506272': [93]})
    noplay_skips.update({'BOS196608141': [67], 'KC1196604240': [87], 'KC1196609020': [55], 'KC1196609210': [107], 'MIN196606160': [78], 'CHN196604300': [49], 'CIN196607242': [79], 'CIN196608272': [105], 'LAN196606140': [64], 'NYN196606132': [80], 'NYN196609040': [74], 'PHI196608200': [77], 'PHI196609100': [77], 'PHI196609260': [66], 'PIT196608220': [60], 'SFN196604130': [70], 'SFN196605050': [64], 'SFN196607010': [91], 'SFN196608170': [72], 'KC1196705200': [46], 'KC1196708090': [51], 'MIN196709250': [46], 'ATL196706250': [71], 'CHN196706130': [71], 'NYN196707010': [76], 'NYN196707021': [72], 'NYN196708080': [87], 'PHI196705190': [77], 'PHI196706050': [55], 'PHI196709060': [69], 'PIT196708040': [75], 'SFN196706222': [75], 'SLN196708160': [73]})
    noplay_skips.update({'BOS196806291': [59], 'BOS196808022': [77], 'CHA196806180': [89,90], 'CLE196806231': [16], 'CLE196807050': [49], 'OAK196806161': [75], 'OAK196808201': [83], 'WS2196809200': [82], 'CIN196807312': [68], 'CIN196809062': [74], 'LAN196809150': [84], 'NYN196806010': [87], 'NYN196808160': [104], 'PHI196805050': [64], 'PHI196809030': [80], 'PHI196809170': [85]})
    noplay_skips.update({'BAL196906110': [51], 'BAL196906250': [64], 'BAL196908050': [91], 'BOS196906211': [79], 'BOS196907200': [67], 'CAL196908030': [77], 'CHA196908060': [89], 'CHA196908160': [58], 'CLE196906082': [53], 'MIN196908260': [94,95], 'MIN196909010': [73,74], 'NYA196904290': [77], 'CHN196907260': [68], 'LAN196906300': [53], 'PHI196904190': [64], 'PHI196908200': [84], 'SLN196907040': [87]})
    noplay_skips.update({'WS1196007290': [50], 'NYA196208270': [62], 'BOS196304200': [100], 'KC1196304120': [75], 'LAA196304292': [28], 'MIN196309180': [58,59], 'MIN196309190': [61], 'PIT196307120': [65], 'CHN196409240': [57], 'MIN196508070': [19], 'CIN196505220': [53], 'CIN196506230': [43]})
    noplay_skips.update({'CLE196606250': [83], 'HOU196605210': [107], 'PHI196608060': [49], 'SFN196608230': [48], 'BOS196704230': [45], 'CAL196709020': [89], 'CAL196809030': [72], 'CLE196807072': [51], 'NYA196806030': [66], 'NYA196808232': [78], 'NYN196808010': [81], 'KCA196904300': [69]})
    noplay_skips.update({'BAL197005270': [77], 'BAL197007020': [74], 'BAL197008282': [80], 'CAL197006200': [68], 'CAL197009040': [58], 'CAL197009130': [69], 'CAL197009260': [59], 'CHA197006141': [90], 'CLE197007242': [57], 'DET197006090': [70], 'DET197006100': [89], 'DET197009110': [80], 'KCA197004240': [82], 'KCA197005010': [96], 'KCA197005220': [86], 'KCA197006140': [75], 'MIN197005250': [75], 'MIN197008031': [70], 'NYA197005032': [64], 'NYA197007192': [61], 'NYA197009020': [75], 'OAK197005172': [105], 'OAK197007310': [57], 'OAK197008021': [33], 'WS2197008092': [70], 'WS2197008110': [77], 'WS2197009030': [81], 'WS2197009040': [70], 'CIN197007090': [109], 'HOU197008130': [74,75], 'LAN197005130': [90], 'LAN197007220': [85], 'LAN197009230': [103], 'MON197007290': [65], 'MON197009250': [79], 'NYN197007250': [84], 'PHI197005220': [84], 'PHI197007050': [86], 'PHI197009230': [70], 'PHI197009300': [45], 'SDN197005250': [63], 'SDN197010010': [25], 'SFN197007192': [86], 'SFN197009110': [73], 'SLN197007202': [66]})
    noplay_skips.update({'BOS197105312': [69], 'BOS197106290': [78], 'CAL197106150': [75,76], 'CHA197105122': [70], 'CHA197108100': [65], 'CHA197109061': [74], 'CLE197105231': [48], 'CLE197107020': [84], 'CLE197108260': [88], 'CLE197109142': [79], 'DET197107252': [80], 'DET197107290': [85], 'MIL197105250': [73], 'MIL197107252': [91], 'MIN197109200': [73], 'NYA197106040': [79], 'NYA197106271': [43], 'OAK197104280': [68], 'OAK197108040': [78], 'WS2197104250': [101], 'WS2197106010': [62], 'ATL197108120': [56], 'NYN197104182': [80], 'NYN197106090': [83], 'NYN197107310': [74], 'PHI197104130': [85], 'PHI197108200': [56], 'PIT197108310': [84], 'SDN197104210': [94], 'SLN197104300': [78], 'SLN197107152': [42]})
    noplay_skips.update({'BAL197204190': [63], 'CHA197204260': [81,92], 'CHA197208220': [55], 'CLE197208061': [61], 'CLE197209241': [54], 'KCA197206240': [61], 'MIL197207232': [83], 'MIN197208260': [95], 'NYA197205260': [90], 'NYA197205280': [56], 'NYA197206290': [74], 'NYA197208260': [80], 'NYA197208291': [75,76], 'TEX197208190': [72], 'HOU197208260': [68], 'MON197206300': [72], 'MON197208190': [65], 'MON197210010': [33], 'NYN197206100': [62], 'PHI197204210': [87], 'PHI197205202': [76], 'PHI197209100': [74], 'SDN197205150': [85]})
    noplay_skips.update({'BAL197307300': [55], 'BOS197305100': [67], 'CAL197304060': [77], 'CAL197305110': [76], 'CAL197306070': [64], 'CHA197306200': [74], 'CHA197306220': [79], 'CHA197306230': [70], 'CHA197308262': [60], 'CHA197309020': [79], 'CHA197309150': [91], 'CLE197305061': [64], 'DET197307040': [63,85,86], 'DET197307260': [73], 'DET197309010': [62], 'DET197309020': [69], 'DET197309240': [106], 'KCA197304190': [81,113], 'KCA197307060': [79,80,81], 'KCA197307292': [78], 'MIL197307090': [50], 'MIL197307130': [98], 'MIN197304130': [78], 'MIN197305230': [87], 'NYA197305020': [67], 'NYA197305131': [73], 'NYA197305180': [89], 'OAK197304220': [99], 'OAK197308020': [69], 'OAK197308210': [86], 'OAK197309280': [88], 'TEX197307042': [67], 'TEX197308220': [90], 'CHN197308220': [82], 'CIN197307132': [86], 'CIN197309300': [70], 'HOU197305120': [76], 'MON197307050': [73], 'MON197309160': [79], 'MON197309231': [77], 'NYN197304222': [91], 'NYN197307310': [57], 'NYN197309032': [82]})
    noplay_skips.update({'BAL197406240': [78], 'BAL197408170': [72], 'BOS197407032': [69], 'BOS197407072': [60], 'BOS197407110': [80], 'BOS197409170': [73], 'CAL197404130': [70], 'CHA197404070': [102], 'CHA197406280': [84], 'CHA197406301': [58], 'CHA197407020': [62], 'CHA197407040': [89], 'CHA197408080': [66], 'CLE197404280': [66], 'CLE197407150': [29], 'CLE197407170': [82,83], 'DET197406090': [75], 'KCA197405190': [67], 'KCA197407140': [93], 'KCA197408041': [69], 'KCA197409290': [64], 'MIL197406050': [83], 'MIL197406060': [63], 'MIL197407072': [60], 'MIL197408061': [38], 'MIL197408231': [80], 'MIL197408300': [40], 'MIL197409100': [96], 'MIL197409130': [31], 'MIL197409150': [59], 'MIN197407140': [59], 'NYA197405240': [62], 'OAK197405040': [59], 'OAK197406150': [71], 'OAK197409020': [52], 'TEX197406020': [80], 'TEX197408080': [86]})    
    noplay_skips.update({'ATL197406080': [74], 'CHN197407052': [71], 'MON197409290': [56], 'PHI197407300': [79], 'PHI197409110': [87], 'PIT197404110': [78], 'SFN197408060': [77], 'SLN197407200': [96]})
    noplay_skips.update({'BAL197508151': [70], 'BAL197509022': [79], 'BAL197509190': [70], 'BOS197507010': [89], 'BOS197507110': [91], 'BOS197507170': [57], 'BOS197507180': [60], 'BOS197507280': [66], 'BOS197508040': [96], 'BOS197509140': [82], 'CAL197505260': [44], 'CLE197505210': [78], 'DET197509160': [61], 'MIL197505010': [100], 'MIL197508060': [87], 'MIN197505150': [83], 'MIN197509140': [98], 'MIN197509270': [72], 'NYA197509131': [66], 'OAK197505130': [64], 'TEX197506170': [91], 'TEX197508240': [56], 'TEX197509010': [75], 'TEX197509162': [72]})
    noplay_skips.update({'CHN197506160': [66], 'CIN197505210': [86], 'HOU197504250': [75], 'LAN197506100': [37,38], 'LAN197508010': [77], 'MON197506171': [72], 'MON197506172': [76,77], 'MON197508170': [77], 'MON197509052': [75], 'PHI197508040': [61], 'PHI197509030': [66], 'SLN197507070': [76]})
    noplay_skips.update({'BOS197609120': [72], 'CAL197609210': [68], 'CAL197609230': [84], 'CHA197606010': [68], 'CHA197606050': [91,92], 'CHA197607251': [66], 'CHA197608081': [94], 'CLE197606110': [83], 'CLE197607240': [87], 'CLE197609050': [62], 'DET197605280': [70], 'DET197606011': [86], 'DET197606120': [56], 'DET197608082': [47], 'KCA197606051': [62],  'KCA197607220': [77], 'MIL197607112': [71], 'MIN197609100': [55], 'NYA197605210': [83], 'OAK197605310': [77], 'TEX197608080': [95], 'TEX197609260': [59], 'CIN197605302': [92], 'CIN197606132': [94], 'HOU197606250': [88], 'MON197607240': [81], 'MON197609181': [80], 'NYN197604110': [42], 'NYN197607310': [60], 'SDN197606132': [69]})
    noplay_skips.update({'BAL197707242': [70], 'BAL197708150': [92], 'CAL197707010': [73], 'CHA197707032': [85], 'DET197708051': [78,94], 'DET197708180': [80], 'KCA197704190': [63], 'KCA197706030': [84,85], 'KCA197706170': [76], 'KCA197706190': [111], 'KCA197708200': [64], 'MIL197710010': [53], 'MIN197706210': [72], 'MIN197707090': [62], 'NYA197708290': [83], 'NYA197708310': [83], 'OAK197706050': [59], 'OAK197706240': [70], 'OAK197708242': [83], 'TEX197710010': [77], 'ATL197704240': [110], 'ATL197704250': [105], 'CHN197705170': [107], 'CHN197705250': [85], 'CIN197705200': [78], 'HOU197704190': [93], 'SDN197707240': [76], 'SLN197709160': [90]})
    noplay_skips.update({'BAL197807130': [77], 'BAL197808090': [62], 'CAL197804120': [84,85], 'CAL197805060': [71], 'CAL197805280': [82], 'CHA197805270': [73], 'CHA197806252': [95], 'CLE197805160': [69], 'DET197804090': [34], 'DET197804231': [51], 'DET197804232': [96], 'DET197805120': [61,100], 'DET197806120': [69], 'DET197807250': [85], 'DET197809090': [56], 'DET197809180': [84], 'KCA197806120': [91], 'MIL197805100': [61], 'MIL197805180': [86,87], 'MIL197808221': [73], 'MIL197809170': [80], 'MIN197805140': [84], 'MIN197806200': [90], 'MIN197808200': [75,76], 'MIN197809010': [72], 'NYA197807301': [74], 'OAK197807080': [53], 'OAK197807130': [78], 'SEA197804280': [49], 'TOR197806210': [40]})
    noplay_skips.update({'ATL197804280': [71,72], 'ATL197807081': [69], 'CHN197805250': [61], 'CHN197807162': [73], 'CIN197805240': [85], 'NYN197805292': [75], 'PIT197806190': [76], 'SFN197804170': [81], 'SLN197806090': [73], 'SLN197809042': [98,99]})
    noplay_skips.update({'BOS197906220': [61,62], 'CAL197904100': [89,90], 'CAL197909040': [68,69], 'CHA197904100': [95,96], 'CHA197906190': [61], 'CHA197906210': [82], 'CLE197904250': [66], 'CLE197905272': [82], 'CLE197906160': [89], 'DET197904280': [68], 'DET197907010': [82], 'DET197907020': [101], 'KCA197904250': [66], 'KCA197907130': [71], 'MIN197905050': [56], 'MIN197905280': [64], 'MIN197907010': [74], 'MIN197909290': [62], 'NYA197909160': [59,76], 'OAK197907140': [99], 'SEA197904240': [96], 'SEA197906220': [65], 'SEA197908060': [72,93], 'SEA197908070': [39], 'SEA197908170': [88,89], 'TEX197907191': [67,68], 'TEX197907270': [81], 'TOR197904160': [65], 'TOR197905020': [78], 'ATL197907062': [89], 'MON197904270': [99], 'PIT197904150': [56], 'PIT197905130': [69], 'SLN197905110': [60], 'SLN197907030': [70]})
    noplay_skips.update({'BAL198005070': [92], 'BAL198009190': [62], 'CAL198008220': [81], 'CHA198004160': [64], 'CHA198005200': [65], 'CHA198006060': [100], 'CLE198006010': [79], 'CLE198006230': [69], 'CLE198009070': [77], 'DET198005200': [62], 'DET198006110': [75], 'DET198009021': [78], 'MIL198005060': [79], 'MIL198009172': [63], 'MIL198009181': [74,75], 'MIN198004280': [86], 'MIN198004290': [71], 'MIN198005030': [84], 'MIN198005160': [76], 'MIN198005280': [65], 'MIN198006200': [76], 'MIN198007230': [92], 'MIN198007251': [35,36], 'MIN198007280': [73], 'MIN198007300': [62], 'MIN198009100': [69], 'MIN198009211': [81], 'MIN198009280': [72], 'NYA198005130': [63], 'NYA198006170': [49], 'NYA198006210': [72], 'NYA198007221': [65], 'NYA198008050': [69], 'NYA198009170': [70], 'SEA198008250': [77], 'SEA198009150': [90], 'SEA198009240': [73], 'TEX198005070': [86], 'TEX198005110': [75], 'TEX198007130': [78], 'TEX198008200': [100], 'TEX198010040': [91]})
    noplay_skips.update({'HOU198008230': [123], 'MON198006081': [74], 'MON198006110': [71], 'MON198006120': [57], 'PHI198009122': [78], 'PIT198007202': [71], 'SDN198005090': [86], 'SDN198007260': [102]})
    noplay_skips.update({'BAL198104280': [88], 'BAL198105260': [62], 'BAL198108140': [58], 'BOS198105030': [70], 'BOS198109050': [89], 'BOS198109060': [82], 'CHA198106100': [68], 'CHA198106110': [71], 'CLE198105180': [79], 'CLE198109051': [62], 'DET198105241': [63], 'DET198106060': [77], 'DET198108210': [54], 'DET198108230': [67], 'KCA198106080': [68], 'KCA198109040': [66], 'MIN198104280': [71], 'MIN198104290': [45], 'MIN198105150': [55], 'MIN198109020': [82], 'NYA198105140': [80], 'OAK198109090': [78], 'SEA198104140': [80], 'SEA198105030': [79], 'SEA198105050': [60], 'SEA198105050': [60,82], 'SEA198105100': [74], 'SEA198105240': [85], 'TEX198105060': [62], 'TEX198106050': [76], 'TEX198108150': [81], 'TOR198104150': [64], 'TOR198108210': [78]})
    noplay_skips.update({'ATL198109080': [73], 'ATL198109130': [84], 'CHN198108210': [85], 'HOU198108280': [59], 'LAN198109050': [77], 'PHI198109200': [77]})
    noplay_skips.update({'BAL198206270': [54], 'BAL198207170': [91], 'BAL198207240': [77], 'BAL198210011': [81], 'BOS198207212': [68,69], 'BOS198209050': [68], 'CAL198207270': [99], 'CAL198208010': [81], 'CAL198208170': [66,77], 'CHA198209050': [87]})
    noplay_skips.update({'DET198204200': [67], 'DET198208082': [79,80], 'DET198209030': [58], 'DET198209050': [33], 'KCA198205010': [87], 'KCA198205020': [69,70], 'KCA198205100': [73], 'KCA198205110': [67,68], 'MIL198209240': [93], 'MIN198207040': [74], 'MIN198208180': [103], 'NYA198208200': [70], 'NYA198208220': [69], 'SEA198206220': [54], 'TEX198205140': [84], 'TEX198207110': [69], 'TEX198208130': [65], 'TOR198204120': [48], 'TOR198205210': [77], 'TOR198205310': [59,60], 'TOR198206010': [75,95], 'TOR198206270': [71], 'TOR198206290': [70], 'TOR198207010': [90], 'TOR198207200': [53], 'TOR198207210': [92], 'TOR198210030': [81,82]})
    noplay_skips.update({'ATL198207272': [92], 'CIN198209060': [61], 'HOU198206200': [59], 'MON198205160': [68], 'SLN198206190': [104], 'SLN198206240': [82]})
    noplay_skips.update({'BAL198305070': [51], 'BAL198305090': [53,83], 'BAL198306030': [70], 'BAL198308070': [71], 'BAL198309050': [68], 'BAL198309160': [47,48,60], 'BAL198309170': [71,86], 'BAL198309190': [42,43], 'BAL198309301': [85], 'BOS198308190': [67,68], 'BOS198309150': [81], 'CAL198307211': [79,80,81], 'CAL198307220': [66], 'CAL198307270': [97], 'CHA198309090': [73], 'CLE198304100': [78], 'DET198307010': [68], 'DET198307130': [62,63], 'DET198307260': [80], 'DET198308280': [71], 'KCA198309191': [83], 'MIL198305270': [81], 'MIL198306250': [76], 'MIL198309250': [41], 'MIN198309160': [58], 'MIN198309170': [85], 'NYA198304150': [35], 'NYA198308100': [79], 'NYA198309101': [93], 'NYA198309102': [55], 'OAK198306080': [73,74], 'OAK198307200': [72], 'OAK198307240': [71], 'SEA198307210': [91], 'TEX198306150': [77], 'TEX198309300': [73], 'TOR198304290': [78], 'TOR198305230': [72], 'TOR198305260': [67,68], 'TOR198307020': [64,65], 'TOR198307080': [50], 'TOR198307200': [93], 'TOR198308292': [87], 'TOR198308300': [76], 'TOR198309010': [60], 'TOR198309022': [90], 'TOR198309100': [52,66]})
    noplay_skips.update({'CHN198306280': [81], 'CHN198307190': [107], 'CHN198308100': [69], 'CHN198309120': [58,80], 'CIN198306100': [83], 'CIN198308280': [71], 'HOU198308270': [81], 'MON198306212': [72], 'MON198307282': [96], 'MON198309192': [50], 'NYN198305230': [74], 'PIT198306220': [66], 'PIT198309100': [89], 'SDN198306140': [85], 'SDN198308180': [85], 'SDN198310010': [93], 'SLN198306290': [63], 'SLN198309051': [51,52], 'SLN198309052': [77,78]})
    noplay_skips.update({'BAL198405020': [59], 'BAL198405120': [88], 'BAL198406080': [75,76,85], 'BAL198406101': [79], 'BAL198407130': [90], 'BAL198407150': [91], 'BAL198409070': [63], 'BOS198406230': [93,94], 'BOS198406280': [79], 'BOS198408072': [75], 'BOS198409090': [77], 'BOS198409250': [92], 'CAL198404080': [64], 'CAL198409010': [74], 'CHA198404080': [52], 'CHA198407070': [74], 'CHA198409080': [65], 'CLE198404150': [95,96], 'CLE198407240': [91], 'CLE198409090': [68], 'CLE198409280': [78], 'DET198404220': [50], 'DET198405160': [89], 'DET198406020': [57,69], 'DET198406292': [64], 'DET198407271': [67], 'DET198408141': [91], 'DET198408150': [71], 'DET198409150': [68], 'DET198409230': [62,84], 'KCA198404150': [70], 'KCA198407250': [124], 'KCA198408100': [79], 'KCA198408110': [65], 'KCA198409040': [62], 'KCA198409250': [112], 'MIL198406140': [64], 'MIL198407022': [61], 'MIL198407240': [84], 'MIL198408300': [67], 'MIL198409250': [81]})
    noplay_skips.update({'MIN198404080': [84], 'MIN198405150': [57], 'MIN198405270': [82], 'MIN198407040': [57], 'MIN198407150': [62], 'MIN198408010': [86], 'MIN198408140': [60], 'MIN198408250': [85], 'NYA198406080': [92,93,94], 'NYA198406160': [62], 'NYA198406250': [81], 'NYA198406260': [110], 'NYA198408020': [74], 'NYA198409280': [88], 'NYA198409290': [79], 'OAK198408270': [76], 'OAK198409150': [67,68], 'SEA198404050': [35], 'SEA198407210': [68,74], 'SEA198408280': [70], 'TEX198404060': [84], 'TEX198405050': [61], 'TEX198407080': [74,75], 'TOR198404170': [64,70], 'TOR198404180': [59], 'TOR198405020': [73], 'TOR198405040': [83], 'TOR198405220': [87], 'TOR198406010': [90], 'TOR198406110': [68], 'TOR198406120': [76,82], 'TOR198407050': [78], 'TOR198407080': [69], 'TOR198407310': [80], 'TOR198408220': [106], 'TOR198408290': [86], 'TOR198409180': [75], 'TOR198409190': [79]})    
    noplay_skips.update({'ATL198406160': [70], 'CHN198405090': [88], 'CHN198405260': [106], 'CIN198406240': [79,80], 'CIN198409110': [59], 'HOU198404130': [62], 'HOU198408230': [95], 'HOU198409220': [105,106], 'LAN198404030': [50], 'LAN198404130': [73], 'LAN198404240': [64], 'LAN198405160': [74], 'LAN198406060': [104], 'LAN198406280': [87], 'LAN198407060': [84], 'LAN198408170': [77], 'LAN198408210': [71], 'LAN198409040': [58], 'MON198404270': [90], 'MON198406092': [80], 'MON198408242': [70], 'NYN198407240': [85], 'NYN198408290': [90], 'NYN198409130': [74,75], 'NYN198409220': [66], 'PHI198406081': [75,76,81], 'PHI198407250': [76], 'PHI198408090': [75], 'PIT198405050': [66], 'PIT198406230': [68], 'PIT198407270': [80], 'PIT198409050': [59], 'SDN198405130': [85], 'SDN198408210': [61], 'SFN198409242': [97], 'SLN198406140': [54], 'SLN198408050': [76]})
    noplay_skips.update({'BAL198504130': [93], 'BAL198504270': [73], 'BAL198504300': [81,89,90], 'BAL198506050': [67], 'BAL198506170': [34,35], 'BAL198507110': [83], 'BAL198507190': [88], 'BAL198508140': [62], 'BAL198510032': [103], 'BAL198510050': [80,81]})
    noplay_skips.update({'BOS198504280': [81], 'BOS198506150': [77], 'BOS198507220': [64]})
    noplay_skips.update({'CAL198505270': [62], 'CAL198509240': [79], 'CHA198505290': [76], 'CHA198507210': [68,79], 'CHA198508090': [70], 'CHA198508270': [66], 'CHA198508290': [58], 'CLE198505230': [56], 'CLE198508261': [62]})
    noplay_skips.update({'DET198504210': [99], 'DET198506100': [93], 'DET198506200': [97], 'DET198509070': [97], 'DET198509130': [76], 'DET198509190': [87,94]})
    noplay_skips.update({'KCA198504100': [88,89], 'KCA198508100': [66], 'MIL198509290': [109], 'MIN198505050': [94,105], 'MIN198507020': [66], 'MIN198508300': [85], 'MIN198510010': [78]})
    noplay_skips.update({'NYA198504160': [91], 'NYA198505310': [61], 'NYA198508030': [79], 'NYA198508040': [73], 'NYA198508050': [54], 'NYA198509010': [80], 'NYA198509240': [82], 'NYA198509292': [54]})
    noplay_skips.update({'OAK198505030': [75], 'OAK198505260': [52], 'SEA198506120': [51], 'SEA198507050': [101], 'SEA198507080': [72], 'SEA198507290': [83], 'SEA198508130': [71]})
    noplay_skips.update({'TEX198506180': [65], 'TOR198505100': [58], 'TOR198505220': [66], 'TOR198505310': [64], 'TOR198506080': [81], 'TOR198507020': [66], 'TOR198507200': [65], 'TOR198507220': [44], 'TOR198508081': [66], 'TOR198509010': [41], 'TOR198509250': [115]})
    noplay_skips.update({'ATL198508210': [86], 'ATL198508220': [85], 'ATL198509120': [63,84], 'CHN198507080': [89], 'CHN198509130': [60]})
    noplay_skips.update({'CIN198505030': [73], 'CIN198505050': [80,81], 'CIN198506212': [80], 'CIN198507210': [79,80], 'CIN198508040': [73]})
    noplay_skips.update({'HOU198505150': [82], 'HOU198506150': [65], 'HOU198508210': [94], 'HOU198509070': [82], 'LAN198506010': [90], 'LAN198506050': [70], 'LAN198506190': [63,70], 'LAN198510010': [93]})
    noplay_skips.update({'MON198505070': [62], 'MON198508240': [65], 'MON198509132': [68], 'NYN198505080': [57], 'NYN198506091': [68], 'NYN198507240': [79]})
    noplay_skips.update({'PHI198505220': [82,91], 'PHI198506130': [61], 'PHI198508230': [65], 'PHI198510052': [85], 'PIT198504170': [57]})
    noplay_skips.update({'SDN198505080': [76], 'SDN198506200': [75], 'SDN198509240': [67], 'SFN198505110': [81], 'SFN198508110': [68], 'SFN198509180': [85]})
    noplay_skips.update({'SLN198506050': [76], 'SLN198506300': [85], 'SLN198508170': [96], 'SLN198509060': [61], 'SLN198509070': [79]})
    noplay_skips.update({'BAL198606170': [59], 'BAL198606190': [82], 'BAL198607200': [55], 'BAL198608230': [79,80], 'BAL198609080': [102], 'BAL198609220': [82]})
    noplay_skips.update({'CAL198606140': [81,82], 'CAL198607190': [67,68], 'CAL198607210': [70], 'CAL198608170': [62], 'CAL198609170': [79], 'CAL198609180': [91], 'CAL198609210': [70]})
    noplay_skips.update({'CHA198607060': [61], 'CHA198609090': [64], 'CHA198609130': [119], 'CLE198606140': [87], 'CLE198608260': [91]})
    noplay_skips.update({'DET198606291': [74], 'DET198607260': [79], 'DET198608051': [92], 'DET198608100': [85], 'DET198609210': [59,60], 'KCA198606210': [81], 'KCA198608140': [97], 'KCA198608210': [78]})
    noplay_skips.update({'MIL198608230': [89], 'MIL198608240': [74], 'MIN198606200': [86], 'MIN198607130': [67], 'MIN198608230': [63], 'MIN198608240': [109], 'MIN198609010': [80,81,93], 'MIN198609230': [82]})
    noplay_skips.update({'NYA198606270': [67], 'NYA198607030': [68], 'NYA198607180': [84,85], 'NYA198607210': [86], 'NYA198607230': [82], 'NYA198609270': [79]})
    noplay_skips.update({'OAK198607270': [89], 'OAK198607300': [74,75], 'OAK198608300': [75], 'OAK198609010': [85], 'OAK198609160': [63], 'OAK198609230': [70]})
    noplay_skips.update({'SEA198608020': [78], 'SEA198609280': [106], 'TEX198604110': [68], 'TEX198607300': [62], 'TOR198606150': [70], 'TOR198606200': [100], 'TOR198607060': [82], 'TOR198607130': [49], 'TOR198608310': [62], 'TOR198609160': [67]})
    noplay_skips.update({'ATL198609010': [69], 'CHN198606160': [83], 'CHN198607010': [69], 'CHN198607220': [88], 'CHN198609190': [68], 'CIN198605050': [79]})
    noplay_skips.update({'CIN198607210': [45], 'CIN198607240': [90], 'HOU198604260': [55], 'HOU198605160': [73], 'HOU198607230': [79], 'HOU198608080': [63], 'HOU198608090': [75], 'HOU198609050': [74], 'HOU198609060': [66,80], 'HOU198609230': [74]})
    noplay_skips.update({'LAN198604100': [81], 'LAN198604270': [88], 'LAN198605180': [78], 'LAN198605220': [73], 'LAN198606070': [53], 'LAN198607040': [65], 'LAN198609100': [61,62], 'LAN198609252': [65,66]})
    noplay_skips.update({'MON198605110': [69], 'MON198606220': [68], 'MON198608302': [70,93,94], 'MON198609171': [69]})
    noplay_skips.update({'NYN198607300': [94], 'NYN198608172': [70], 'PHI198609170': [53], 'PIT198604270': [73], 'PIT198608020': [87]})
    noplay_skips.update({'SDN198604160': [73,74], 'SDN198605170': [85], 'SDN198607080': [68], 'SDN198608030': [71], 'SDN198608040': [73], 'SDN198610010': [60]})
    noplay_skips.update({'SFN198604200': [61], 'SFN198604220': [88], 'SFN198607100': [76], 'SFN198608170': [72], 'SFN198610020': [87]})
    noplay_skips.update({'SLN198605110': [72], 'SLN198609280': [61]})
    noplay_skips.update({'BAL198709020': [89], 'BAL198709050': [78], 'BOS198706190': [82,83], 'CAL198707110': [76], 'CAL198708200': [93], 'CAL198709230': [77]})
    noplay_skips.update({'CHA198704240': [62], 'CHA198704250': [79], 'CHA198708050': [77], 'CHA198709020': [69], 'CLE198706030': [98], 'CLE198708211': [44]})
    noplay_skips.update({'DET198705050': [59], 'DET198705150': [83], 'DET198706100': [85], 'DET198708030': [78], 'DET198708280': [81,82], 'KCA198708140': [84], 'MIL198709221': [83], 'MIL198709230': [83]})
    noplay_skips.update({'NYA198707050': [98], 'NYA198709150': [41], 'OAK198705010': [80], 'SEA198705250': [86], 'SEA198706120': [66]})
    noplay_skips.update({'TEX198705180': [40], 'TEX198705230': [62], 'TEX198709290': [56], 'TEX198710020': [58]})
    noplay_skips.update({'TOR198704180': [76], 'TOR198705010': [80], 'TOR198707240': [81], 'TOR198707250': [90], 'TOR198707290': [86,92], 'TOR198709120': [51]})
    noplay_skips.update({'ATL198708190': [65,66], 'ATL198709130': [74], 'ATL198709230': [81], 'ATL198709240': [88], 'CHN198707100': [97], 'CHN198709080': [71], 'CHN198709240': [76]})
    noplay_skips.update({'CIN198707010': [79], 'CIN198708300': [71], 'CIN198709240': [77], 'HOU198705110': [68], 'HOU198708050': [91], 'HOU198709050': [56], 'HOU198709080': [94], 'HOU198709180': [89]})
    noplay_skips.update({'LAN198704130': [67], 'LAN198706240': [73], 'LAN198706270': [85], 'LAN198707280': [78], 'LAN198708290': [100], 'LAN198709190': [65]})
    noplay_skips.update({'MON198704240': [84], 'MON198704250': [65], 'MON198706100': [84], 'MON198708140': [96], 'MON198709170': [77], 'MON198710030': [70]})
    noplay_skips.update({'NYN198705160': [66], 'NYN198705240': [64], 'NYN198708060': [66,67], 'NYN198708260': [71], 'NYN198709140': [67,68,75], 'NYN198709250': [75]})
    noplay_skips.update({'PHI198707020': [65], 'PHI198709260': [67], 'PIT198706280': [66], 'PIT198707050': [72], 'PIT198707080': [86], 'PIT198709140': [114], 'PIT198709302': [69]})
    noplay_skips.update({'SDN198706250': [64], 'SDN198707220': [83], 'SDN198709300': [60], 'SFN198705060': [89], 'SFN198707200': [71], 'SFN198709010': [97], 'SFN198710020': [68]})
    noplay_skips.update({'SLN198706180': [81], 'SLN198706210': [62], 'SLN198706240': [57], 'SLN198707071': [80], 'SLN198708150': [72], 'SLN198709060': [73]})
    noplay_skips.update({'BAL198805080': [88], 'CAL198806120': [71], 'CAL198807170': [67,68], 'CAL198809180': [92], 'CHA198805140': [78], 'CHA198805150': [84], 'CHA198805200': [53], 'CHA198806200': [74]})
    noplay_skips.update({'CLE198805030': [94], 'CLE198806110': [79], 'CLE198808110': [64], 'CLE198809041': [74], 'CLE198809042': [70], 'KCA198808170': [65], 'NYA198807180': [79]})
    noplay_skips.update({'OAK198805050': [74], 'SEA198804170': [92], 'SEA198804180': [67], 'SEA198807200': [79], 'SEA198808110': [83], 'SEA198808290': [79], 'SEA198809170': [72], 'SEA198810020': [81]})
    noplay_skips.update({'TEX198808080': [82], 'TEX198808220': [74], 'TOR198807300': [68], 'TOR198809140': [63]})
    noplay_skips.update({'ATL198806150': [84], 'CIN198804190': [62], 'CIN198805210': [75], 'CIN198808140': [60,61], 'CIN198808280': [83]})
    noplay_skips.update({'HOU198805110': [73], 'HOU198806100': [68], 'HOU198808130': [63], 'HOU198809180': [61]})
    noplay_skips.update({'LAN198806191': [69], 'LAN198809212': [101], 'LAN198809300': [80], 'MON198805070': [78,79], 'MON198808310': [77], 'MON198809120': [63,64,65], 'MON198809150': [62]})
    noplay_skips.update({'NYN198805240': [70], 'PIT198805250': [67], 'SDN198809250': [82], 'SFN198807262': [101], 'SFN198808240': [57], 'SFN198809090': [65]})
    noplay_skips.update({'BAL198905120': [87], 'BAL198908200': [77], 'CAL198904240': [76], 'CAL198904260': [72], 'CAL198907220': [113]})
    noplay_skips.update({'CHA198908060': [73], 'CLE198904220': [79], 'CLE198904230': [52], 'CLE198906210': [64], 'CLE198909240': [85], 'DET198905200': [86], 'DET198905210': [62,63], 'DET198908030': [69], 'DET198908071': [83]})
    noplay_skips.update({'KCA198904110': [78], 'KCA198905280': [56], 'KCA198909221': [71], 'MIL198907250': [54], 'MIN198904050': [89], 'MIN198909230': [81], 'NYA198906080': [99], 'NYA198908242': [60], 'OAK198904250': [73]})
    noplay_skips.update({'OAK198905210': [80], 'OAK198907090': [91], 'OAK198909060': [84], 'OAK198909280': [62], 'SEA198905020': [72], 'SEA198906020': [75], 'SEA198906200': [79]})
    noplay_skips.update({'TEX198907280': [80,97], 'TEX198909071': [78,79], 'TEX198909240': [93]})
    noplay_skips.update({'ATL198907300': [43,53], 'CHN198905090': [67], 'CHN198905100': [84], 'CHN198908080': [62], 'CHN198908090': [63], 'CHN198908210': [76]})
    noplay_skips.update({'CIN198904050': [78], 'CIN198905100': [92], 'HOU198905160': [87], 'HOU198905220': [48], 'HOU198905230': [69], 'HOU198907040': [75], 'HOU198907300': [76], 'HOU198908120': [83], 'HOU198909270': [56]})
    noplay_skips.update({'LAN198908280': [66], 'MON198908150': [70], 'MON198908200': [76], 'NYN198905170': [75], 'NYN198905190': [75], 'NYN198906200': [97], 'NYN198908050': [78], 'NYN198908101': [82], 'PHI198906040': [67], 'PHI198908160': [89,90], 'PHI198909172': [76], 'PIT198904150': [88], 'PIT198907040': [66], 'PIT198909150': [61]})
    noplay_skips.update({'SDN198906230': [56,57], 'SDN198907180': [95], 'SFN198904140': [81], 'SFN198904150': [62], 'SFN198906300': [63], 'SFN198907130': [69], 'SFN198908070': [89], 'SLN198906270': [67]})
    noplay_skips.update({'BAL199006270': [76], 'BOS199009050': [84], 'CAL199009190': [90], 'CHA199005020': [62], 'CHA199006160': [94], 'CHA199006200': [74], 'CHA199007020': [85], 'CHA199007030': [74]})
    noplay_skips.update({'CLE199006160': [62], 'CLE199008090': [86], 'DET199005010': [73,74], 'DET199006200': [87], 'DET199007080': [89], 'DET199009250': [69]})
    noplay_skips.update({'KCA199009210': [78], 'MIL199009040': [92], 'MIL199009070': [69], 'MIN199006080': [76], 'MIN199007220': [78], 'NYA199007150': [69], 'NYA199008250': [74]})
    noplay_skips.update({'OAK199009170': [52], 'OAK199009190': [86], 'SEA199004130': [81,93,94], 'SEA199006110': [79], 'SEA199008140': [58], 'SEA199009210': [70]})
    noplay_skips.update({'TEX199004180': [82], 'TEX199006220': [72], 'TOR199004200': [74], 'TOR199006030': [76], 'TOR199007290': [56], 'TOR199008290': [64], 'TOR199009210': [56,90]})
    noplay_skips.update({'ATL199004112': [86], 'ATL199006240': [63], 'CIN199010010': [78], 'HOU199004150': [76], 'HOU199005230': [80,87], 'HOU199007170': [94], 'HOU199007200': [99], 'LAN199006010': [79], 'LAN199006020': [77], 'LAN199007260': [85], 'LAN199009040': [97]})
    noplay_skips.update({'MON199005240': [95], 'MON199009030': [77], 'PHI199005110': [53], 'PHI199006030': [78], 'PHI199007030': [53,54], 'PHI199009100': [85], 'PIT199004140': [66], 'PIT199005130': [69], 'PIT199006170': [88], 'PIT199009090': [73]})
    noplay_skips.update({'SDN199004280': [79], 'SDN199008150': [79], 'SFN199005310': [72], 'SFN199008030': [81], 'SFN199009050': [86], 'SLN199005270': [88], 'SLN199008040': [72]})
    noplay_skips.update({'BAL199104270': [72], 'BAL199105160': [84], 'BAL199105250': [62], 'BAL199107270': [69], 'BOS199105070': [78], 'BOS199105090': [87], 'BOS199105110': [72], 'BOS199110030': [81]})
    noplay_skips.update({'CAL199108040': [84], 'CHA199104180': [70], 'CHA199104230': [77], 'CLE199106250': [111], 'CLE199108180': [79], 'CLE199109300': [73], 'CLE199110030': [83]})
    noplay_skips.update({'DET199104080': [60], 'DET199104170': [84], 'DET199105040': [54], 'DET199105050': [67], 'DET199105240': [95], 'DET199106040': [104]})
    noplay_skips.update({'KCA199104130': [82], 'KCA199104300': [100], 'KCA199106010': [89,90], 'KCA199109150': [74], 'MIL199104150': [57], 'MIL199107130': [65], 'MIL199107150': [73], 'MIL199109140': [84,85], 'MIL199109270': [72,73]})
    noplay_skips.update({'MIN199104300': [82], 'MIN199105120': [71], 'NYA199104200': [79], 'NYA199105140': [77], 'NYA199107270': [102,103], 'NYA199108112': [70], 'NYA199108140': [80], 'NYA199109010': [61], 'NYA199109150': [77], 'NYA199110050': [87], 'NYA199110050': [87,88]})
    noplay_skips.update({'OAK199105230': [87], 'OAK199108050': [82], 'OAK199108060': [57], 'OAK199108310': [73], 'SEA199106100': [85,86], 'SEA199106150': [89], 'SEA199107060': [79,97], 'SEA199108150': [69], 'TEX199106120': [72], 'TEX199109191': [61], 'TOR199104280': [86], 'TOR199105070': [84], 'TOR199105290': [90], 'TOR199106280': [66], 'TOR199107260': [88], 'TOR199110010': [79]})
    noplay_skips.update({'ATL199104100': [79], 'ATL199105080': [99], 'CIN199105240': [99], 'HOU199109280': [70], 'LAN199105290': [88], 'LAN199107250': [76], 'MON199105010': [80], 'NYN199107110': [85], 'PHI199107170': [80], 'PHI199108220': [106], 'SDN199108110': [77], 'SDN199109250': [79], 'SFN199105120': [79], 'SFN199107050': [65], 'SLN199104300': [72,80], 'SLN199106010': [94], 'SLN199106250': [70]})    
    noplay_skips.update({'BOS199204261': [65], 'BOS199207260': [87], 'BOS199207270': [71], 'BOS199207290': [79], 'BOS199208200': [63]})
    noplay_skips.update({'CAL199209200': [99], 'CAL199209280': [98], 'CHA199205020': [77], 'CHA199209081': [74], 'CHA199209230': [86], 'CLE199205040': [85], 'DET199204110': [75], 'DET199207290': [55], 'DET199209200': [82,83], 'DET199209250': [86]})
    noplay_skips.update({'KCA199205170': [75], 'KCA199207051': [88], 'KCA199207052': [70], 'KCA199210040': [89], 'MIL199204180': [61]})
    noplay_skips.update({'MIN199204110': [102], 'NYA199205190': [64], 'NYA199209270': [82], 'OAK199206170': [78], 'SEA199204210': [87], 'TEX199206200': [76,77], 'TEX199206240': [70], 'TEX199208150': [78], 'TOR199207280': [78], 'TOR199209180': [49,50,75]})
    noplay_skips.update({'ATL199206160': [71], 'ATL199206180': [76], 'CHN199209140': [114], 'CHN199209200': [116,117], 'CIN199206150': [57], 'HOU199206140': [91]})
    noplay_skips.update({'LAN199204170': [78,79], 'LAN199204180': [85], 'LAN199204190': [72], 'LAN199204220': [73], 'LAN199204270': [97], 'LAN199206040': [72], 'LAN199207081': [93,94], 'LAN199208220': [80], 'LAN199209130': [94]})
    noplay_skips.update({'MON199206051': [69], 'NYN199207220': [85], 'PHI199204080': [95,96], 'PHI199209240': [96], 'PIT199207180': [62], 'SDN199204120': [90], 'SFN199204260': [90], 'SFN199209090': [53], 'SLN199209300': [95]})
    noplay_skips.update({'BAL199304300': [81], 'BAL199306240': [63], 'BAL199309070': [70], 'BOS199305220': [64], 'BOS199307200': [84,85], 'BOS199309140': [85,86], 'BOS199309281': [85], 'BOS199309282': [88]})
    noplay_skips.update({'CHA199305090': [76], 'CLE199306180': [79], 'CLE199308290': [87], 'DET199304200': [72], 'DET199305070': [109], 'DET199305090': [63], 'DET199307030': [55], 'DET199307220': [80]})
    noplay_skips.update({'KCA199304090': [82,83], 'KCA199304270': [66], 'KCA199306060': [66], 'KCA199308090': [92], 'KCA199308230': [79], 'MIL199308271': [85], 'MIN199304160': [74], 'MIN199306010': [89,90], 'MIN199307220': [81], 'MIN199308070': [80]})
    noplay_skips.update({'NYA199307220': [77], 'OAK199308150': [74], 'SEA199305100': [78], 'SEA199305240': [85,86], 'SEA199307090': [72], 'TEX199304100': [89], 'TEX199308250': [71], 'TOR199304260': [67], 'TOR199309100': [87,88]})
    noplay_skips.update({'ATL199306020': [77], 'ATL199306030': [90], 'ATL199309140': [87], 'ATL199309190': [98,99,100,101], 'ATL199309290': [84], 'CHN199305310': [67], 'CHN199306010': [76], 'CHN199307110': [86], 'CHN199309210': [80], 'CIN199308050': [71]})
    noplay_skips.update({'COL199304250': [84], 'COL199309050': [74], 'COL199309170': [67], 'COL199309220': [62], 'HOU199304060': [70], 'HOU199304160': [92,93], 'HOU199306190': [89], 'HOU199308040': [82], 'HOU199308150': [56], 'LAN199308080': [49], 'MON199304140': [78], 'MON199309210': [71]})
    noplay_skips.update({'NYN199306220': [70], 'NYN199306250': [72], 'NYN199309240': [72,73], 'NYN199309290': [87], 'PHI199306040': [77], 'PHI199307090': [104], 'PHI199309250': [80], 'SDN199306120': [93], 'SLN199308060': [45]})
    noplay_skips.update({'BOS199407220': [81], 'CAL199406210': [67], 'CAL199408070': [120], 'CHA199405030': [91], 'DET199405210': [92], 'DET199405250': [90], 'MIL199404070': [87], 'MIN199406300': [75], 'MIN199408070': [66], 'SEA199405150': [93,94], 'SEA199406050': [82], 'TEX199405130': [91], 'TEX199407070': [79], 'TEX199408100': [80]})
    noplay_skips.update({'ATL199406140': [68], 'CHN199406030': [75], 'CIN199404040': [102], 'CIN199406240': [90,91], 'FLO199404140': [57], 'HOU199405010': [92], 'MON199404090': [84], 'MON199405280': [84], 'PIT199406060': [80], 'SDN199405300': [70], 'SFN199406120': [69], 'SFN199406160': [70]})
    noplay_skips.update({'BAL199505010': [80], 'BAL199506040': [92], 'BAL199506250': [85], 'BAL199507030': [80], 'BOS199504300': [102,103], 'BOS199507020': [85], 'BOS199507160': [78]})
    noplay_skips.update({'CAL199509090': [68], 'CAL199509100': [101], 'CAL199509130': [72], 'CAL199509160': [74], 'CAL199509170': [71,72], 'CHA199509010': [81], 'CLE199506130': [81], 'CLE199509300': [84], 'CLE199510010': [83,84], 'DET199505030': [90], 'DET199505250': [37], 'DET199506140': [78]})
    noplay_skips.update({'KCA199504280': [73], 'KCA199505200': [78,79], 'KCA199507210': [60], 'MIL199505110': [96], 'MIL199506130': [90], 'MIL199507200': [110], 'MIN199504280': [68], 'MIN199505030': [88], 'MIN199506090': [83]})
    noplay_skips.update({'NYA199508030': [73], 'NYA199509080': [81,82,83], 'OAK199508230': [59], 'SEA199505310': [81], 'SEA199509120': [87,88], 'TEX199506270': [83], 'TEX199509030': [61], 'TOR199505020': [96], 'TOR199507020': [95], 'TOR199507290': [51], 'TOR199508220': [91], 'TOR199508250': [68], 'TOR199509130': [79]})
    noplay_skips.update({'CHN199507240': [71], 'CIN199505200': [90], 'CIN199506160': [82], 'CIN199508020': [64], 'COL199505060': [57], 'COL199509080': [88], 'HOU199508160': [90], 'HOU199509190': [94], 'HOU199509250': [76], 'LAN199504290': [61], 'LAN199505170': [87]})
    noplay_skips.update({'NYN199505140': [85], 'NYN199509120': [67], 'PIT199505050': [63], 'PIT199509080': [72], 'SFN199505010': [68], 'SLN199505030': [80]})
    noplay_skips.update({'BAL199604110': [88], 'BAL199605170': [107], 'BAL199605190': [75], 'BAL199607270': [72], 'BOS199604130': [95], 'BOS199605150': [83], 'BOS199607220': [71], 'BOS199608240': [149], 'BOS199608240': [64], 'BOS199609290': [87]})
    noplay_skips.update({'CAL199604020': [90], 'CAL199604030': [97], 'CAL199605250': [84], 'CHA199605110': [67], 'CHA199606210': [78]})
    noplay_skips.update({'CLE199606040': [77,101], 'CLE199606050': [74], 'CLE199606190': [78], 'CLE199607310': [75], 'CLE199609120': [84], 'CLE199609141': [85], 'CLE199609230': [66,67], 'DET199605050': [73], 'DET199607010': [76], 'DET199607050': [84]})
    noplay_skips.update({'KCA199604240': [76], 'KCA199604250': [70], 'KCA199608200': [76], 'MIL199604250': [68], 'MIL199607180': [81,82,83], 'MIN199604210': [75], 'MIN199605210': [72], 'MIN199607250': [72,73], 'MIN199608210': [95], 'MIN199609080': [68], 'MIN199609120': [82]})
    noplay_skips.update({'NYA199604090': [71], 'NYA199605040': [85], 'OAK199608130': [70], 'OAK199609040': [77], 'OAK199609080': [69]})
    noplay_skips.update({'SEA199605070': [69], 'SEA199605080': [86], 'SEA199605310': [76], 'SEA199606160': [94], 'SEA199606180': [53], 'SEA199606280': [93], 'SEA199607240': [82], 'SEA199608070': [87], 'SEA199608300': [69], 'SEA199609200': [90,107]})
    noplay_skips.update({'TEX199604170': [74], 'TEX199606210': [85], 'TEX199607040': [84], 'TOR199604280': [77], 'TOR199606280': [67], 'TOR199607010': [70], 'TOR199607190': [62], 'TOR199609110': [76,77], 'TOR199609280': [80]})
    noplay_skips.update({'CHN199605150': [58], 'COL199606270': [98], 'COL199606290': [80], 'COL199607270': [73], 'FLO199609240': [76], 'HOU199604070': [78], 'LAN199606230': [82], 'LAN199608180': [73], 'LAN199608250': [85], 'NYN199606070': [87], 'PHI199605140': [76], 'PHI199609190': [72]})
    noplay_skips.update({'PIT199608300': [96], 'PIT199609190': [86], 'PIT199609220': [68], 'SDN199608040': [66], 'SLN199609020': [110], 'SLN199609280': [80,81]})
    noplay_skips.update({'ANA199708060': [77], 'ANA199709170': [83], 'ANA199709280': [70], 'BAL199709120': [96], 'BOS199705270': [88], 'BOS199708160': [89], 'BOS199709070': [58], 'CHA199706230': [69], 'CLE199704180': [79], 'DET199706050': [79], 'MIL199706240': [63], 'MIL199709280': [88]})
    noplay_skips.update({'MIN199705260': [97], 'MIN199707130': [96], 'MIN199707260': [82], 'MIN199708060': [77], 'MIN199708310': [70], 'MIN199709070': [74], 'MIN199709220': [65]})
    noplay_skips.update({'NYA199705050': [69], 'OAK199706120': [58], 'OAK199707120': [72], 'OAK199707150': [89], 'SEA199704050': [64], 'SEA199704060': [69], 'SEA199704200': [92], 'SEA199705030': [93,106], 'SEA199706230': [61], 'SEA199707160': [93], 'SEA199707190': [80], 'TOR199704250': [87,88], 'TOR199705260': [77]})
    noplay_skips.update({'ATL199708130': [65], 'CIN199706150': [76,77], 'HOU199709150': [86], 'MON199706090': [79], 'PIT199708070': [84], 'SDN199705100': [92], 'SDN199708140': [89], 'SLN199708150': [85]})
    noplay_skips.update({'ANA199805090': [23], 'BAL199805170': [80], 'BOS199807030': [88], 'BOS199808280': [85], 'BOS199809130': [58], 'CHA199804090': [61,87], 'CLE199807110': [73], 'DET199805180': [85], 'DET199809040': [82]})
    noplay_skips.update({'KCA199805300': [92], 'KCA199809140': [111,112], 'MIN199808070': [104], 'MIN199809150': [92], 'NYA199809210': [64], 'SEA199803310': [91], 'SEA199809250': [77,78], 'TEX199804110': [102], 'TEX199805250': [61]})
    noplay_skips.update({'ARI199809060': [84], 'ATL199806180': [77], 'ATL199808040': [70], 'ATL199809250': [76], 'CHN199806170': [70], 'CHN199809100': [50], 'CHN199809280': [66], 'CIN199807190': [99], 'CIN199809120': [71], 'CIN199809160': [64]})                        
    noplay_skips.update({'COL199808260': [87], 'HOU199809080': [88], 'LAN199808270': [81], 'MIL199807110': [100], 'MON199805250': [67], 'MON199808230': [75], 'MON199809080': [71], 'NYN199807120': [70], 'NYN199807290': [81], 'NYN199808211': [95], 'NYN199809070': [51]})
    noplay_skips.update({'PHI199805040': [86], 'PIT199804180': [64], 'PIT199805090': [64], 'PIT199808210': [78], 'SDN199805142': [82], 'SFN199805160': [83], 'SFN199805280': [89], 'SFN199806080': [84], 'SLN199807110': [102], 'SLN199808090': [71]})
    noplay_skips.update({'ANA199904070': [90], 'ANA199908020': [67], 'BAL199907240': [91], 'BAL199909302': [50], 'BOS199906060': [54], 'CHA199907270': [64], 'CHA199910020': [71], 'CLE199904140': [73], 'CLE199909300': [75], 'DET199904140': [53], 'DET199905110': [74], 'KCA199906230': [80], 'MIN199909080': [84], 'NYA199904090': [68], 'NYA199908290': [83]})
    noplay_skips.update({'OAK199904300': [85], 'OAK199907160': [71], 'OAK199908060': [81], 'OAK199908250': [90], 'TBA199908100': [88], 'TEX199908020': [61], 'TOR199905290': [81], 'TOR199907310': [83]})
    noplay_skips.update({'ARI199905070': [95], 'ARI199905240': [88], 'ATL199907270': [80], 'ATL199909210': [86], 'CHN199909240': [85], 'COL199906040': [109], 'COL199907080': [93], 'FLO199906280': [105], 'HOU199909140': [74], 'LAN199907110': [70], 'MIL199905270': [78], 'MIL199909220': [87]})
    noplay_skips.update({'MON199905300': [86], 'MON199906300': [75], 'MON199907010': [85], 'MON199909240': [85], 'NYN199905202': [70], 'NYN199906070': [92,93], 'NYN199907020': [80], 'PHI199905050': [80]})
    noplay_skips.update({'PIT199904090': [58], 'SDN199908180': [80], 'SDN199909200': [95], 'SFN199908150': [96]})
    noplay_skips.update({'ANA200009090': [75], 'BAL200005040': [82], 'BAL200007140': [77], 'BOS200009212': [100,101], 'CHA200004180': [102], 'CHA200007130': [86], 'CLE200009260': [75], 'DET200007221': [86], 'DET200007260': [78], 'KCA200004100': [65], 'KCA200007180': [84], 'KCA200009240': [79,80], 'MIN200007230': [61,62]})
    noplay_skips.update({'OAK200004120': [62], 'OAK200005210': [77], 'SEA200005310': [78,79], 'TBA200009150': [86,87]})
    noplay_skips.update({'ARI200008210': [116], 'CHN200005110': [37], 'CHN200008010': [72], 'CHN200009090': [90], 'CIN200006250': [85], 'COL200006180': [76], 'COL200007171': [62], 'COL200008070': [77], 'HOU200008080': [82], 'HOU200009300': [83], 'LAN200008200': [98]})
    noplay_skips.update({'MON200009240': [89], 'NYN200006070': [101], 'NYN200008110': [62], 'NYN200008140': [77], 'PIT200009082': [83], 'PIT200009300': [75], 'SFN200004130': [83], 'SFN200009231': [89]})
    noplay_skips.update({'ANA200107140': [91], 'BAL200104170': [66,84], 'BAL200108160': [89], 'BOS200109050': [70], 'CHA200106030': [96], 'CLE200107250': [80], 'DET200107190': [80], 'KCA200107200':[82], 'MIN200107200': [71], 'NYA200106020': [85], 'NYA200109080': [82], 'TEX200109050': [80], 'TOR200107280': [88]})
    noplay_skips.update({'ARI200104260': [76], 'ATL200107120': [80], 'CIN200109290': [71], 'CIN200109300': [81], 'COL200105210': [65], 'COL200106010': [81], 'LAN200108050': [78], 'MON200104070': [62], 'MON200109050': [73], 'PIT200108090': [85], 'SDN200105130': [61], 'SDN200109230': [91], 'SLN200105070': [80]})
    noplay_skips.update({'ANA200205090': [86], 'CLE200204090': [67], 'CLE200206290': [73,74], 'CLE200209150': [71], 'DET200207200': [88], 'DET200208230': [87], 'DET200209150': [73], 'KCA200204060': [93,94], 'KCA200205160': [87], 'KCA200207151': [70], 'MIN200208040': [93], 'NYA200207180': [84]})
    noplay_skips.update({'OAK200207060': [70], 'OAK200209220': [73,98], 'TBA200204240': [61], 'TBA200207130': [85], 'TBA200207310': [82], 'TEX200204250': [88], 'TEX200208020': [86], 'TOR200205040': [74]})
    noplay_skips.update({'ATL200207270': [51], 'ATL200208010': [71], 'CIN200207060': [69], 'CIN200207070': [73], 'CIN200208290': [73], 'COL200209060': [71,72], 'COL200209140': [86,87], 'FLO200209170': [129], 'LAN200209280': [101], 'MIL200205230': [71]})
    noplay_skips.update({'MON200209240': [85], 'NYN200206200': [57], 'NYN200209290': [84], 'PHI200209080': [92], 'PIT200206080': [91], 'SDN200205020': [62], 'SDN200205310': [96], 'SFN200206020': [78], 'SLN200207040': [63], 'SLN200209220': [85], 'SLN200209230': [91]})
    noplay_skips.update({'ANA200305070': [78,79], 'ANA200305100': [70], 'ANA200307280': [60], 'BAL200309030': [76], 'BAL200309150': [91,92], 'BOS200305190': [87], 'BOS200306110': [81], 'BOS200309230': [100], 'BOS200309250': [82]})
    noplay_skips.update({'CHA200304060': [54], 'CLE200307121': [62], 'KCA200305300': [74], 'MIN200307050': [76], 'MIN200308020': [74], 'OAK200304020': [80], 'OAK200308140': [63], 'OAK200309110': [92], 'TEX200306140': [72], 'TOR200308090': [84]})
    noplay_skips.update({'ATL200306210': [80,81], 'ATL200309210': [77,78,79], 'CHN200309010': [79], 'CIN200304260': [84], 'COL200308190': [82], 'FLO200307010': [63], 'MIL200304160': [64], 'MON200309060': [115], 'NYN200306220': [86], 'PHI200307040': [76], 'PIT200309192': [78], 'SFN200308080': [56], 'SLN200306030': [79]})
    noplay_skips.update({'ANA200407310': [116], 'ANA200409110': [106], 'BAL200404230': [87], 'BAL200404250': [94], 'BOS200407060': [61], 'BOS200408150': [82], 'BOS200408310': [78,79], 'BOS200409260': [87,88,89]})
    noplay_skips.update({'CHA200404150': [87], 'CHA200407250': [73], 'CLE200404170': [88], 'CLE200405170': [86], 'CLE200406140': [66], 'CLE200407080': [77], 'CLE200409040': [70], 'CLE200409140': [93]})
    noplay_skips.update({'MIN200405230': [102], 'MIN200408050': [53], 'OAK200407270': [72], 'TEX200406130': [83], 'TEX200409110': [86], 'TOR200404050': [86], 'TOR200404160': [90], 'TOR200409160': [91]})
    noplay_skips.update({'ARI200407250': [78,79], 'ARI200409140': [79], 'ATL200406180': [76], 'ATL200408140': [91], 'CHN200409102': [74,89,90], 'CHN200409270': [85], 'CHN200409300': [73]})
    noplay_skips.update({'COL200406060': [90], 'COL200409232': [71], 'MON200404090': [63], 'PIT200404210': [77], 'PIT200405282': [71], 'PIT200409091': [65], 'SFN200408170': [86], 'SLN200405120': [77]})    
    noplay_skips.update({'BAL200504170': [84], 'BAL200509070': [88], 'BOS200504180': [84], 'CHA200509100': [78,79,80], 'CLE200509110': [85], 'DET200506170': [66], 'DET200509100': [68]})
    noplay_skips.update({'KCA200507250': [104], 'KCA200509240': [94,95], 'NYA200504230': [95], 'NYA200505240': [59], 'NYA200505280': [67,68], 'NYA200508220': [80], 'OAK200506040': [67], 'OAK200506260': [63], 'OAK200509020': [86]})
    noplay_skips.update({'SEA200508150': [79], 'TOR200504080': [68], 'TOR200504090': [70], 'TOR200504100': [86], 'TOR200504230': [73,74], 'TOR200504260': [60], 'TOR200506140': [76], 'TOR200506190': [62], 'TOR200507140': [76], 'TOR200508080': [87]})
    noplay_skips.update({'ARI200507040': [79], 'CHN200506140': [84], 'CHN200507150': [70], 'CHN200507270': [87], 'COL200504180': [88], 'COL200507280': [90], 'COL200509200': [76], 'NYN200504230': [66], 'PHI200504190': [83], 'SDN200509280': [88,89], 'SFN200504230': [81], 'WAS200504160': [80]})
    noplay_skips.update({'ANA200609300': [97], 'BOS200608182': [92], 'CHA200605220': [82], 'CHA200609090': [87], 'CLE200604270': [89], 'CLE200605290': [66], 'CLE200606020': [89], 'CLE200609250': [67], 'DET200609150': [89], 'DET200609270': [70]})
    noplay_skips.update({'MIN200609100': [94], 'NYA200609010': [78], 'NYA200609120': [82,83], 'OAK200605040': [91], 'OAK200608150': [96], 'SEA200604240': [82], 'SEA200608220': [95]})
    noplay_skips.update({'TBA200605160': [95,96], 'TBA200605180': [78], 'TOR200604050': [90,91], 'TOR200604070': [63]})
    noplay_skips.update({'ARI200609280': [106], 'CHN200604290': [74], 'CHN200605260': [74], 'CIN200608070': [85], 'CIN200609120': [66], 'FLO200609110': [98], 'HOU200604090': [85], 'PHI200609022': [47]})
    noplay_skips.update({'SDN200607190': [62], 'SDN200609200': [87], 'SLN200605090': [81], 'SLN200606160': [78], 'SLN200609150': [74,75], 'WAS200609190': [83]})
    noplay_skips.update({'ANA200706040': [91], 'ANA200706260': [89], 'ANA200708210': [84], 'BAL200707130': [60], 'BAL200708221': [101], 'BAL200708250': [86], 'BAL200709270': [83]})
    noplay_skips.update({'BOS200704100': [59], 'BOS200705110': [80], 'CHA200708241': [81], 'CHA200709300': [82]})
    noplay_skips.update({'CLE200706110': [89], 'CLE200708110': [83], 'CLE200708150': [73], 'CLE200708310': [89], 'DET200704180': [80], 'KCA200705220': [61], 'KCA200709290': [64]})
    noplay_skips.update({'MIN200705060': [82,83], 'NYA200704190': [70], 'NYA200707080': [83], 'NYA200709040': [90], 'NYA200709180': [81], 'SEA200705010': [63]})
    noplay_skips.update({'TBA200707130': [65], 'TBA200708180': [72], 'TOR200704120': [85]})
    noplay_skips.update({'ATL200706240': [53], 'FLO200704080': [72], 'FLO200705050': [66], 'FLO200706130': [79], 'LAN200708170': [71], 'LAN200709150': [79], 'MIL200709280': [66], 'NYN200709290': [96,97]})    
    noplay_skips.update({'PHI200706170': [69], 'SLN200704260': [63], 'WAS200709210': [57]})
    noplay_skips.update({'BAL200808250': [66], 'BOS200809131': [87], 'CLE200808110': [81], 'CLE200809190': [71], 'DET200805200': [84,85]})
    noplay_skips.update({'KCA200809180': [78], 'MIN200805150': [111], 'NYA200804040': [88], 'NYA200805210': [77], 'NYA200807300': [79,80,86], 'OAK200808140': [49], 'SEA200808040': [88]})
    noplay_skips.update({'TBA200808310': [87], 'TBA200809150': [57,58], 'TBA200809190': [70], 'TEX200806030': [85], 'TEX200806190': [87], 'TOR200804040': [75], 'TOR200809170': [85]})
    noplay_skips.update({'ATL200804200': [73], 'ATL200806080': [78], 'CHN200806260': [75], 'CIN200809220': [74], 'FLO200808160': [73], 'PIT200807260': [61], 'SDN200804170': [83], 'SDN200808300': [79], 'SLN200806130': [71]})
    noplay_skips.update({'ANA200906130': [74], 'ANA200906220': [82], 'ANA200909270': [76], 'BAL200909020': [94], 'BOS200908220': [71], 'BOS200909080': [69], 'CHA200904100': [89], 'CHA200904110': [85], 'CHA200906250': [76], 'CHA200909080': [77]})
    noplay_skips.update({'DET200905050': [64], 'DET200909140': [72], 'KCA200906190': [51], 'KCA200907040': [75], 'MIN200905280': [54], 'NYA200906180': [65], 'SEA200908090': [69], 'TBA200905310': [82], 'TBA200908160': [83], 'TBA200909220': [60]})
    noplay_skips.update({'CHN200909150': [98,99], 'CIN200909010': [87], 'CIN200910010': [73], 'CIN200910040': [85], 'FLO200904080': [61], 'FLO200905220': [65,66], 'FLO200906230': [92], 'HOU200909250': [68], 'LAN200909190': [91], 'MIL200908110': [84], 'NYN200908010': [95], 'SFN200907030': [74]})
    noplay_skips.update({'ANA201004210': [82], 'BAL201006260': [83], 'CHA201009120': [116], 'CLE201007290': [88,89,98], 'CLE201009270': [87], 'DET201005310': [61], 'KCA201009140': [45], 'MIN201007010': [78], 'NYA201004300': [80], 'NYA201005310': [82]})
    noplay_skips.update({'OAK201005070': [79], 'OAK201009100': [75,76], 'OAK201009260': [100], 'TBA201008010': [75,76], 'TBA201008030': [68], 'TBA201008310': [61], 'TBA201009260': [62], 'TEX201006040': [64], 'TEX201006270': [76], 'TEX201009150': [89], 'TOR201006220': [68], 'TOR201009110': [87,99]})
    noplay_skips.update({'CHN201005260': [78], 'CHN201007020': [87], 'CIN201009300': [75], 'COL201005151': [60], 'COL201009020': [116], 'FLO201009130': [87], 'HOU201008160': [87], 'MIL201008070': [77], 'NYN201008010': [81], 'PIT201009220': [83], 'SDN201009260': [87], 'SFN201006120': [87], 'SLN201009130': [72,91]})
    noplay_skips.update({'ANA201109020': [90], 'BAL201108310': [80], 'BOS201106180': [82], 'CHA201104070': [71], 'CHA201105010': [84], 'DET201104230': [78], 'DET201107010': [78], 'DET201109010': [70], 'KCA201106250': [82], 'MIN201105280': [56], 'MIN201107060': [76]})
    noplay_skips.update({'OAK201105170': [66], 'SEA201104080': [79], 'SEA201106150': [70], 'SEA201108160': [100], 'TBA201107190': [67,68], 'TBA201109110': [83], 'TEX201106220': [88]})
    noplay_skips.update({'ARI201107230': [84,85], 'ARI201109210': [94], 'CHN201108220': [85], 'CIN201109170': [92], 'CIN201109180': [87], 'COL201108170': [65], 'LAN201105010': [81], 'LAN201109220': [77], 'MIL201104100': [81], 'MIL201109100': [76]})
    noplay_skips.update({'PHI201109220': [58], 'PIT201105100': [92], 'SDN201109040': [83], 'SFN201105070': [92], 'SLN201107050': [77], 'WAS201105160': [70], 'WAS201106220': [60]})
    noplay_skips.update({'ANA201207080': [67], 'ANA201209180': [92], 'BAL201209260': [83], 'BOS201204170': [87], 'BOS201206080': [74], 'BOS201206090': [64], 'BOS201207170': [85], 'BOS201209230': [73], 'CHA201205130': [87], 'CHA201206180': [82], 'CHA201209290': [102], 'CLE201209290': [89]})
    noplay_skips.update({'DET201204110': [73], 'DET201205050': [64], 'DET201208080': [82], 'MIN201204240': [82,83], 'NYA201206200': [81], 'NYA201208160': [97], 'NYA201209220': [97], 'NYA201210010': [83], 'OAK201204110': [102], 'OAK201206220': [65], 'OAK201208030': [102,115]})
    noplay_skips.update({'SEA201205270': [71], 'TBA201204300': [80], 'TBA201207200': [86], 'TEX201205260': [133], 'TOR201209040': [91]})
    noplay_skips.update({'ARI201204230': [85], 'ATL201207290': [57], 'CHN201204220': [97], 'COL201209270': [83], 'HOU201205060': [91], 'HOU201206250': [97], 'HOU201209110': [60], 'HOU201209230': [49], 'LAN201204110': [73]})
    noplay_skips.update({'MIA201209290': [59], 'MIL201209290': [82], 'MIL201210020': [60], 'NYN201204100': [70], 'NYN201204240': [63], 'PHI201204280': [80], 'SDN201205060': [85], 'SLN201205030': [88], 'SLN201209030': [82], 'WAS201205040': [65], 'WAS201209060': [88], 'WAS201209220': [97]})
    noplay_skips.update({'ANA201306170': [80], 'BAL201309270': [97], 'CHA201308250': [77], 'CLE201304090': [80], 'CLE201308250': [81], 'DET201305110': [78], 'DET201307280': [81], 'DET201308161': [71], 'HOU201305040': [75], 'HOU201305050': [76], 'HOU201305110': [56], 'HOU201305270': [94], 'HOU201309170': [72]})
    noplay_skips.update({'KCA201304290': [75], 'KCA201305280': [64], 'KCA201306110': [57,58], 'MIN201309230': [104], 'OAK201304290': [67], 'OAK201305130': [70], 'OAK201305140': [66], 'OAK201305150': [75], 'OAK201305190': [75], 'OAK201308210': [79], 'OAK201309190': [72]})
    noplay_skips.update({'SEA201306290': [77], 'SEA201309080': [60], 'SEA201309270': [61], 'SEA201309280': [57], 'TBA201306290': [90], 'TEX201305170': [70], 'TEX201306020': [78], 'TEX201306110': [77], 'TEX201306170': [67], 'TEX201308170': [99], 'TEX201309150': [58], 'TOR201309170': [73]})
    noplay_skips.update({'ARI201305120': [89], 'CHN201305040': [81], 'CHN201306020': [97], 'CIN201304050': [79,80,81], 'COL201308090': [85], 'LAN201306280': [74], 'LAN201307310': [79], 'MIA201306010': [74], 'MIL201305050': [87], 'PIT201306300': [82]})
    noplay_skips.update({'SFN201307220': [62], 'SFN201309080': [91], 'SLN201305120': [82], 'SLN201306050': [92], 'SLN201308150': [57], 'SLN201309140': [67], 'WAS201305100': [71]})
    noplay_skips.update({'ANA201407090': [61], 'ANA201408070': [80], 'ANA201408090': [134], 'BOS201407290': [70], 'CLE201404230': [86], 'CLE201405160': [79], 'CLE201408220': [68], 'CLE201409010': [101], 'DET201409270': [97,98,108]})
    noplay_skips.update({'HOU201404250': [80,81,91], 'HOU201405140': [65], 'HOU201405160': [88], 'KCA201406040': [104], 'KCA201408120': [83,84], 'KCA201409190': [89], 'MIN201409030': [86], 'OAK201404030': [78], 'OAK201404210': [92], 'OAK201406140': [67], 'OAK201409200': [73]})
    noplay_skips.update({'SEA201408110': [61], 'SEA201408260': [70], 'TEX201409270': [43], 'TOR201409100': [91]})
    noplay_skips.update({'CHN201409060': [52], 'CHN201409190': [102], 'CIN201408280': [91,92], 'COL201409070': [70], 'LAN201408190': [83], 'MIL201408080': [77], 'MIL201408190': [73], 'MIL201409040': [60]})
    noplay_skips.update({'NYN201407050': [77], 'PHI201406281': [65], 'PIT201405230': [83], 'SFN201405250': [77], 'SLN201409210': [84]})
    noplay_skips.update({'ANA201505280': [80], 'BAL201510031': [72], 'CHA201504240': [59], 'CHA201506190': [73], 'CHA201509150': [99], 'CLE201504110': [69], 'CLE201508280': [66], 'CLE201508280': [66,67], 'DET201507050': [87], 'HOU201504170': [67], 'HOU201504170': [66], 'HOU201508310': [74], 'HOU201509020': [97], 'KCA201509030': [102]})
    noplay_skips.update({'MIN201508020': [61], 'NYA201509270': [65], 'OAK201509020': [87], 'SEA201504250': [85,86], 'SEA201505080': [79], 'SEA201506030': [67], 'TBA201509120': [79], 'TEX201508150': [79], 'TOR201505240': [80]})
    noplay_skips.update({'ARI201507240': [85], 'CIN201507300': [68], 'CIN201509250': [73,101], 'PIT201508080': [72], 'SLN201506270': [82], 'SLN201509090': [64]})
    noplay_skips.update({'BAL201607100': [64], 'BOS201609130': [81], 'CLE201609080': [92], 'MIN201606090': [70], 'NYA201604060': [93,103], 'NYA201604160': [82], 'NYA201604210': [69,70], 'NYA201608270': [108], 'OAK201607030': [68], 'OAK201609050': [88]})
    noplay_skips.update({'SEA201604090': [65], 'SEA201604110': [69], 'SEA201605130': [79], 'SEA201605240': [84], 'SEA201605250': [78], 'SEA201608090': [103], 'SEA201609060': [106], 'SEA201609180': [78], 'TBA201606150': [73], 'TEX201604050': [65], 'TEX201606050': [62], 'TEX201607090': [85], 'TEX201608300': [66], 'TOR201605170': [93]})
    noplay_skips.update({'ARI201609100': [99,100], 'ATL201604210': [72], 'ATL201606010': [84], 'ATL201608180': [69], 'ATL201609110': [88], 'CIN201607160': [81], 'COL201608060': [92], 'LAN201609040': [57], 'LAN201609060': [78], 'NYN201607030': [91], 'SDN201608070': [65], 'SFN201606240': [89], 'SFN201607090': [67], 'SFN201609300': [66], 'WAS201606300': [56]})
    noplay_skips.update({'ANA201706020': [81,90], 'ANA201708280': [62], 'ANA201709130': [78], 'BAL201706010': [65], 'BAL201706210': [63], 'BOS201705130': [65], 'BOS201707161': [68], 'BOS201708150': [60]})
    noplay_skips.update({'CLE201706100': [71], 'CLE201707070': [89], 'CLE201709140': [91], 'DET201707160': [67], 'KCA201706040': [61], 'KCA201708070': [79], 'KCA201709110': [91], 'KCA201709130': [78]})
    noplay_skips.update({'MIN201709140': [84], 'MIN201709160': [70], 'NYA201708260': [78], 'NYA201709200': [84,85], 'OAK201704150': [57], 'SEA201707040': [71], 'SEA201707070': [62]})
    noplay_skips.update({'TBA201709060': [76], 'TEX201705120': [62], 'TEX201709100': [96]})
    noplay_skips.update({'ARI201705230': [77], 'CHN201706020': [55], 'CHN201709090': [59], 'COL201706210': [86], 'LAN201709060': [73], 'LAN201709070': [107], 'MIA201708150': [85]})
    noplay_skips.update({'MIL201709210': [71], 'NYN201709252': [60], 'PIT201709230': [75], 'SLN201709250': [95,104], 'WAS201707080': [79]})
    noplay_skips.update({'BAL201805121': [72], 'BAL201807270': [99], 'BAL201809120': [72,88], 'BOS201808290': [81], 'BOS201809090': [91,92], 'BOS201809130': [81], 'BOS201809140': [83], 'CLE201805130': [67], 'CLE201806230': [61], 'CLE201808060': [80], 'KCA201807230': [76], 'KCA201809120': [60]})
    noplay_skips.update({'NYA201804030': [57], 'NYA201807282': [97], 'OAK201803300': [65], 'OAK201806290': [67], 'OAK201807040': [61], 'OAK201809200': [83], 'SEA201804190':  [74], 'SEA201807240': [88], 'SEA201809270': [73], 'TBA201809070': [78], 'TEX201804080': [90]})
    noplay_skips.update({'ARI201809180': [99], 'ATL201809050': [99], 'LAN201807300': [63], 'MIA201804110': [70], 'MIL201809080': [79], 'NYN201807070': [68], 'NYN201809070': [50], 'PHI201809112': [50], 'PHI201809140': [73], 'SDN201805040': [49], 'SLN201809150': [104]})
    noplay_skips.update({'BAL201905230': [85], 'BOS201906100': [84], 'BOS201907280': [89], 'BOS201908160': [58], 'CLE201909220': [76], 'HOU201906070': [82], 'HOU201907230': [91], 'HOU201908220': [73], 'MIN201907200': [94], 'MIN201909060': [115], 'TBA201908170': [89], 'TOR201904140': [86]})
    noplay_skips.update({'ARI201905190': [76], 'ATL201906110': [82], 'CHN201907150': [81], 'COL201909290': [57], 'LAN201909060': [46,47], 'PHI201905290': [82], 'PHI201907300': [74], 'PHI201909090': [76], 'SDN201907020': [76], 'SFN201907180': [74], 'SLN201904220': [93]})
    noplay_skips.update({'ANA202007290': [106], 'CHA202009260': [79], 'HOU202008110': [68], 'MIN202008180': [85], 'MIN202009120': [76], 'TBA202007270': [86], 'TEX202009240': [91], 'TOR202008150': [74]})
    noplay_skips.update({'ATL202008040': [73], 'CHN202009190': [62], 'COL202009160': [66], 'MIL202008250': [60], 'SFN202008220': [66]})    
    noplay_skips.update({'BAL202106210': [75], 'BAL202108080': [81], 'BAL202109120': [88], 'BOS202108210': [97], 'CHA202104130': [81], 'CHA202108150': [74]})
    noplay_skips.update({'CLE202104050': [69], 'CLE202104110': [81], 'DET202106110': [64], 'HOU202104280': [87], 'HOU202106170': [78], 'KCA202109280': [77]})
    noplay_skips.update({'MIN202105060': [78], 'MIN202106130': [86], 'NYA202107041': [78,79], 'NYA202108070': [62], 'NYA202108172': [57], 'NYA202109040': [73], 'OAK202105080': [73], 'OAK202105270': [57]})
    noplay_skips.update({'SEA202107230': [60], 'TEX202106090': [56], 'TEX202109160': [72]})
    noplay_skips.update({'ATL202107212': [75], 'CHN202109241': [74], 'CIN202106090': [87], 'COL202105121': [71], 'MIA202104160': [50], 'NYN202106130': [63]})
    noplay_skips.update({'PIT202108280': [91,92], 'SFN202104130': [58], 'SLN202104130': [71], 'WAS202105292': [74,75]})
    noplay_skips.update({'ANA202204080': [86], 'ANA202204120': [74], 'ANA202208020': [56], 'ANA202209280': [72,73], 'BAL202204170': [87], 'BAL202205190': [65], 'BAL202207240': [83], 'BAL202208050': [66]})
    noplay_skips.update({'BOS202209140': [66], 'CLE202206240': [70], 'DET202206100': [84], 'DET202206130': [87], 'DET202209290': [94], 'HOU202208120': [69], 'HOU202208260': [69]})
    noplay_skips.update({'KCA202209240': [67], 'KCA202209250': [57], 'NYA202207310': [99], 'OAK202206030': [83], 'OAK202207220': [69], 'OAK202208240': [66], 'OAK202210030': [80]})
    noplay_skips.update({'SEA202205090': [82], 'SEA202206182': [64], 'SEA202207260': [58], 'TBA202204080': [83], 'TBA202204270': [75], 'TBA202208240': [64], 'TEX202206020': [67], 'TEX202206050': [73], 'TEX202208140': [80], 'TOR202210010': [75,76]})
    noplay_skips.update({'ARI202208310': [91], 'ARI202209010': [70], 'ATL202208160': [70], 'CHN202205301': [75], 'LAN202208210': [87], 'LAN202209020': [86]})
    noplay_skips.update({'MIA202207160': [74], 'MIL202204200': [70], 'MIL202207240': [66], 'MIL202208270': [59], 'MIL202208300': [60], 'MIL202209170': [68], 'MIL202210010': [45], 'MIL202210030': [59], 'NYN202208050': [64]})
    noplay_skips.update({'PHI202205310': [47], 'PHI202206110': [72], 'PHI202207230': [65], 'PIT202206230': [78], 'SDN202208030': [81], 'SFN202205230': [67], 'SFN202206140': [75], 'SFN202207130': [82], 'SFN202207300': [71], 'SLN202204090': [77]})
    
    
    # players return to game after having a courtesy runner run for them due to injury
    noplay_skips.update({'BOS191508090': [35], 'NYA191608260': [37], 'CLE192104240': [52,68], 'SLN192107260': [50,51,73,74], 'DET192206090': [19], 'NYA192208220': [56], 'BOS192309250': [30], 'CLE192308120': [48], 'PHA192306080': [19]})
    noplay_skips.update({'CHA192406140': [28], 'DET192404270': [54], 'DET192406200': [73], 'DET192407060': [34], 'SLA192406130': [16], 'CIN192405170': [13], 'DET192507210': [28,55], 'PHI192604170': [37,38], 'PHI192906221': [26]})
    noplay_skips.update({'WS1193005050': [90], 'CHA193407041': [30], 'WS1193408270': [11], 'NY1193409181': [39], 'NY1193608140': [23], 'PIT193609072': [11], 'CHA193808041': [13], 'CLE194104150': [16]})
    
    # no play for unknown reason, nothing seems to have happened
    noplay_skips.update({'CHA191406160': [7], 'CHN191407100': [33], 'SLA191507170': [59], 'BRO191508190': [70], 'BRO191508200': [21], 'BOS191607111': [13], 'PHI192105190': [96]})
    noplay_skips.update({'CHA191706210': [22], 'CLE191905030': [36], 'PHI191907240': [98], 'CLE192106190': [20], 'SLA192304210': [78], 'DET192406070': [35], 'BOS192606290': [71], 'CIN192605160': [67], 'DET192708260': [58], 'NYA192704170': [80], 'PHA192706222': [43], 'WS1192706290': [32], 'CIN192906220': [70]})
    noplay_skips.update({'CHN193107180': [14], 'BOS193405020': [50], 'PHA193405110': [85], 'BRO193405210': [87], 'SLN193406190': [95], 'PHA193507271': [122, 124], 'PHA193509081': [65], 'CIN193609072': [19]})
    noplay_skips.update({'CHN193807211': [61], 'NYA193909200': [56], 'PHI193906111': [71], 'SLA194009170': [24], 'BRO194009210': [54], 'NYA194108110': [80], 'BRO194109280': [50], 'CHN194107250': [16], 'SLA194205200': [36], 'PIT194405190': [57], 'PIT194406222': [72], 'CHN194504291': [65,85], 'DET194709070': [69], 'PHA194705160': [17], 'BRO194704170': [88], 'BRO194706260': [42], 'CHN194709280': [33], 'PIT194706250': [53], 'NYA194807212': [108], 'NYA194808280': [77], 'BRO194807070': [58], 'NY1194808070': [51]})
    noplay_skips.update({'PIT194906200': [72], 'SLA195107042': [136], 'CHA195209280': [54], 'CLE195204180': [78], 'CIN195206291': [92]})                     
    noplay_skips.update({'BOS195308090': [80], 'CLE195306120': [68], 'DET195305230': [69], 'DET195306020': [117,118], 'DET195309010': [52], 'SLA195307230': [82], 'SLN195308020': [19], 'CLE195404180': [45], 'CIN195404240': [55], 'MLN195406062': [102], 'NY1195408221': [98]})
    noplay_skips.update({'WS1195704211': [114], 'PHI195806012': [127], 'KC1195907020': [36]})
    noplay_skips.update({'CHA197006300': [50], 'CLE197006211': [84], 'MIN197008180': [98], 'NYA197005200': [60], 'KCA197307280': [18], 'CIN197309280': [30], 'SFN197305050': [86], 'NYA197406110': [59], 'TEX197505020': [80], 'CHN197508230': [77], 'BAL197606220': [146], 'CAL197608270': [61], 'MIN197608150': [82], 'SFN197605210': [9], 'SLN197605160': [84], 'CAL197704190': [45], 'MIN197708210': [40], 'MON197708170': [83]})
    noplay_skips.update({'BAL197805180': [43], 'MON197909020': [75], 'SDN197906190': [77]})
    noplay_skips.update({'DET198009280': [60], 'CHN198008110': [138], 'BAL198105300': [62], 'DET198104270': [38], 'KCA198104210': [62], 'LAN198108280': [60], 'LAN198108310': [61], 'LAN198110040': [88]})
    noplay_skips.update({'CAL198204230': [30], 'CHA198309212': [64,76], 'DET198407280': [62], 'TEX198509060': [79], 'NYN198504280': [66,76]})
    noplay_skips.update({'CHA199204160': [15], 'TOR199306080': [84], 'TOR199309260': [19], 'COL199405190': [50], 'COL199406030': [69]})
    noplay_skips.update({'BAL199506060': [83], 'BAL199507270': [74], 'CLE199507050': [20], 'DET199608020': [63,64], 'KCA199609110': [63], 'MON199604230': [33], 'SFN199607240': [12]})
    noplay_skips.update({'BOS199809010': [55], 'SEA199805010': [87], 'SEA199809070': [84,89,90,92], 'ATL199806250': [18], 'MON199809100': [54,56,57,58,60,61,68,70,72,74,75,76,77], 'MON199809110': [56], 'OAK199910030': [54]})
    noplay_skips.update({'BAL200110052': [82], 'ATL200110070': [17], 'TEX200308170': [33]})
    noplay_skips.update({'SEA201609200': [94,97]})
            
    # other reasons
    noplay_skips.update({'PIT192004230': [13]})  # hamie101 replaced carlh101 with a 2-2 count on mchea101
    noplay_skips.update({'CLE192208030': [113]})  # mcmin101 stayed in at CF in bottom of 10th
    noplay_skips.update({'DET192406120': [13,14]})  # no visiting team starting lineup listed
    noplay_skips.update({'PHI192409021': [63]})  # Phillies outfielders about to change positions
    noplay_skips.update({'PHI193007232': [141]})  # comoa101 changed positions to CF
    noplay_skips.update({'CHN193405110': [64]})  # ejection
    noplay_skips.update({'CLE193909242': [15]})  # no visiting team starting lineup listed
    noplay_skips.update({'PIT194309262': [69]})  # scorecard stops
    noplay_skips.update({'NY1194707112': [14,15]})  # no visiting team starting lineup listed
    noplay_skips.update({'SLA195204202': [108]})  # multiple fielders changed positions
    noplay_skips.update({'COL199809080': [6]})     # no starters listed for home team
    noplay_skips.update({'CHA201309090': [6]})  # no visiting team starting lineup listed        
    explained = False
    mystery_noplay_dict = {}
#     if (type(thisrow['comments'] == float)):
#         print('comment. is float')
#     else:
#         print(thisrow['comments'], ': ', type(thisrow['comments']))
    
    # is it a comment?
    if (isinstance(noplay_df['comments'].loc[0], str)):   
        if (printme):
            print('--COMMENT--')
            print('\t{0:}'.format(noplay_df['comments'].loc[0]))
            print('--/COMMENT--')
        explained = True
        #state['comments'] = thisrow['comments']
#        state['killswitch'] = True
    if (isinstance(noplay_df['comments'].loc[1], str)):   
        if (printme):
            print('--COMMENT IN NEXT ROW--')
            print('\t{0:}'.format(noplay_df['comments'].loc[0]))
            print('--/COMMENT IN NEXT ROW--')            
        explained = True
    if (noplay_df['batterID'].loc[0] != noplay_df['batterID'].loc[1]):   
        if (printme):
            print('-- replacing batter: {0:} replaces {1:} --'.format(noplay_df['batterID'].loc[1], noplay_df['batterID'].loc[0]))
        state['batterID'] = noplay_df['batterID'].loc[1]
        explained = True
    if (noplay_df['pitcherID'].loc[0] != noplay_df['pitcherID'].loc[1]):
        if (printme):
            print('-- replacing pitcher: {0:} replaces {1:} --'.format(noplay_df['pitcherID'].loc[1], noplay_df['pitcherID'].loc[0]))
        state['pitcherID'] = noplay_df.loc[1]['pitcherID']
        explained = True
    for x in ['field_C_playerID', 'field_1B_playerID', 'field_2B_playerID', 'field_SS_playerID', 'field_3B_playerID', 'field_LF_playerID', 'field_CF_playerID', 'field_RF_playerID']:
        if (noplay_df[x].loc[0] != noplay_df[x].loc[1]):
            if (printme):
                print('**** REPLACING {0:}: {1:} replaces {2:}'.format(x, noplay_df[x].loc[1], noplay_df[x].loc[0]))
            explained = True
    if (noplay_df['gameID'].loc[0] in list(noplay_skips.keys())): # special cases when we previously parsed players staying in
        if (noplay_df['event_in_game'].loc[0] in noplay_skips[noplay_df['gameID'].loc[0]]):
            if (printme):
                print('SKIP gameID = {0:}, event {1:.0f}'.format(noplay_df['gameID'].loc[0], noplay_df['event_in_game'].loc[0]))
            explained = True
    if (not(explained)):
        for x in ['pitcherID', 'field_C_playerID', 'field_1B_playerID', 'field_2B_playerID', 'field_SS_playerID', 'field_3B_playerID', 'field_LF_playerID', 'field_CF_playerID', 'field_RF_playerID']:
            if (noplay_df[x].loc[0] != noplay_df[x].loc[-1]):
                if (printme):
                    print('**** Leaving in a previous field sub at {0:}: {1:} stays in to replace {2:}'.format(x, noplay_df[x].loc[0], noplay_df[x].loc[-1]))
                explained = True
    if (isinstance(noplay_df['pinch_runner_playerID'].loc[0], str)):        
        if ((int(noplay_df['gameID'].loc[0][3:7]) >= 2020) & (noplay_df['inning'].loc[0] >= 10) & ('starts inning at' in str(noplay_df['comments'].loc[0]))):
            courtesy_base = int(str(noplay_df['comments'].loc[0])[str(noplay_df['comments'].loc[0]).find('starts inning at')+17:str(noplay_df['comments'].loc[0]).find('starts inning at')+18])
            if (printme):
                print('@@@@@@@@@@@ COURTESY RUNNER AT {0:} @@@@@@@@@@@'.format(ordinaltg(courtesy_base)))
            state['onbase'][courtesy_base] = noplay_df['pinch_runner_playerID'].loc[0]
            if (printme):
                print('new onbase state: ',state['onbase'])
        else:
            for b in [3,2,1]:
                if (state['onbase'][b] == noplay_df['pinch_runner_replaced_playerID'].loc[0]):
                    state['onbase'][b] = noplay_df['pinch_runner_playerID'].loc[0]
                    break
            if (printme):
                print('**** PINCH RUNNER: {0:} replaces {1:} at {2:}'.format(noplay_df['pinch_runner_playerID'].loc[0], noplay_df['pinch_runner_replaced_playerID'].loc[0], ordinaltg(b)))
        explained = True
    #if (noplay_df[['gameID', 'event_in_game']].isin(special_case_skips_df.set_index(['gameID', 'event_in_game']).index.tolist())):
    
    if (not(explained)):
        print('-----------start np-------------\n')
        
        #print(thisrow)
        print('Previous row:')
        pprint(noplay_df.loc[-1])
        print('This row:')
        pprint(noplay_df.loc[0])
        print('///////// state ///////////')
        pprint(state)
        print('/ ///////// state ///////////')
        print('Next row:')
        pprint(noplay_df.loc[1])
        print('/-----------end np-------------\n')
        state['killswitch'] = True
    return state
print('ok')

ok


In [81]:
s = time.time()

start_at_game_number = 0
pfe = 2334345

skip100 = True
printme = False
courtesy_runner_half_inning = -1

non_batter_re = re.compile('^(SB|CS|PO|BK|WP|PB|OA)')
sb_re = re.compile('^(SB(\d|H);*)+')
caught_stealing_re = re.compile('CS(\d|H)')
#pickoff_re = re.compile('(PO\d\(*E*\d*\))+')
#pickoff_re = re.compile('(PO\d\(\d*E+\d*\))+')
pickoff_re = re.compile('(PO\d\;*)+')

hit_re = re.compile('^(S(?!B)|D|T|HR)(\d|GR)*')
onbase_nohit_re = re.compile('^(\d|FL|FC)*(W|IW|HP|C|E|FC)')
fielder_out_re = re.compile('^\d+.*')

base_updater_atom_re = re.compile('\((\d|B|H)\)')
#base_updater_atom_re = re.compile('(\d)+\((\d|B|H)\)')
mp_re = re.compile('(?<!N)(D|T)P')

print('creating a new dataframe to hold what we calculate here...')
new_things_df = pandas.DataFrame(data=None, columns=new_columns, index=events_df.index)
max_event_id = -1

# NOTE: we HAVE TO keep all the NPs because pitches occurred on all of them!
print('\nparsing plays...')

gamelist = events_df['gameID'].unique().tolist()
print('Found {0:,.0f} events from {1:,.0f} games-with-events!'.format(len(events_df), len(gamelist)))
print('\n')

for this_game_number in range(start_at_game_number, len(gamelist)):
#for this_game_number in [517,671]:
#for this_game_number in range(start_at_game_number, start_at_game_number+1):
    thisgameID = gamelist[this_game_number]
        
    if (skip100):
        if (np.mod(this_game_number, 100) == 0):
            print('Reading game {0:.0f} of {1:.0f}: {2:}...'.format(this_game_number, len(gamelist), thisgameID))
    else:
        print('Initializing game {0:.0f} of {1:.0f}: {2:}'.format(this_game_number, len(gamelist), thisgameID))
    #print(games_df.loc[gamelist[this_game_number]][['visitor', 'home']])
#    print('\n')
    state = {}
    state['score'] = [0,0]
    state['nOuts'] = 0
    state['onbase'] = {1: '', 2: '', 3: ''}
    state['manual_batter_advance'] = False
    state['batterID'] = ''
    state['vh'] = -1
    state['killswitch'] = False
    already_said_ended_early = False

    this_game_innings = events_df[events_df['gameID'] == thisgameID]['inning'].unique()              
    for this_inning in this_game_innings:
        for vh in [0,1]:
            state['vh'] = vh
            state['nOuts'] = 0
#            state['comments'] = np.nan
            for i in range(1,3+1):
                state['onbase'][i] = ''
            if (vh == 0):
                inningside = 'top'
            else:
                inningside = 'bottom'
            if (printme):
                print('====================================================')
                print('  {0:} of the {1:} has {2:.0f} plays...'.format(inningside.title(), ordinaltg(this_inning), len(events_df[(events_df['gameID'] == thisgameID) & (events_df['inning'] == this_inning) & (events_df['visitor_or_home'] == vh)])))
                print('====================================================')

            this_half_inning_events_df = events_df[
                    (events_df['gameID'] == thisgameID) 
                    & (events_df['inning'] == this_inning) 
                    & (events_df['visitor_or_home'] == vh)
            ]

            for ix, thisrow in this_half_inning_events_df.iterrows():
                new_things_df.loc[ix, ['vis_score', 'home_score', 'outs', 'runner_1b', 'runner_2b','runner_3b']
                                 ] = [state['score'][0], state['score'][1], state['nOuts'], state['onbase'][1], state['onbase'][2], state['onbase'][3]]
                new_things_df.loc[ix, ['sh_flag', 'sf_flag']] = [False, False]
                state['batterID'] = thisrow['batterID']
                state['manual_batter_advance'] = False
                
                if (thisrow['event_in_game'] >= pfe):
                    printme = True
                    print_batterID_at_end = True
                if (printme):
                    if (thisrow['theplay'] != 'NP'):
                        print('     ***')
                        print('     Event {0:.0f} starting state: V {1:.0f} H {2:.0f}, {3:.0f} outs, onbase = {4:}, {5:} at bat, {6:} pitching'.format(thisrow['event_in_game'], state['score'][0], state['score'][1], state['nOuts'], state['onbase'], thisrow['batterID'], thisrow['pitcherID']))
                if (printme):
                    print('\tEvent {0:.0f}: {1:}'.format(thisrow['event_in_game'], thisrow['play_orig']))

                has_baserunning = False
            
                if (thisrow['baserunning'] != ''):
                    has_baserunning = True

                if (thisrow['theplay'][0:2] == 'NP'):
                    if (printme):
                        print('------Event {0:.0f} (ix = {1:.0f}): NP ------------------'.format(thisrow['event_in_game'], ix))
                    print_batterID_at_end = True
                    noplay_columns = ['which', 'ix', 'gameID', 'event_in_game', 'inning', 'batterID', 'pitcherID', 'comments']
                    noplay_columns += ['field_C_playerID', 'field_1B_playerID', 'field_2B_playerID', 'field_SS_playerID', 'field_3B_playerID']
                    noplay_columns += ['field_LF_playerID', 'field_CF_playerID', 'field_RF_playerID', 'pinch_runner_playerID', 'pinch_runner_replaced_playerID']
                    
                    noplay_df = pandas.DataFrame(data=None, columns=noplay_columns)
                    noplay_df = noplay_df.set_index('which')
                    noplay_df.loc[0, :] = events_df.loc[ix][[x for x in events_df.columns if x in [y for y in noplay_df.columns if y != 'ix']]]
                    noplay_df.loc[0, 'ix'] = ix
                    # Is this the first event of this half-inning?
                    if (ix == this_half_inning_events_df.index.min()):
                        if (printme):
                            print('---- first event of inning {0:.0f}.{1:.0f} ----'.format(this_inning,vh))
                        if (this_inning == 1):
                            noplay_df.loc[-1, :] = pandas.Series(dtype=object)
                        else:
                            most_recent_eventID = events_df[(events_df['gameID'] == thisgameID) & (events_df['inning'] == this_inning - 1) & (events_df['visitor_or_home'] == vh)].index.max()
                            noplay_df.loc[-1, :] = events_df[[x for x in events_df.columns if x in [y for y in noplay_df.columns if y != 'ix']]].loc[most_recent_eventID]
                            noplay_df.loc[-1, 'ix'] = most_recent_eventID
                            
                            #noplay_df.loc[-1, :] = events_df.loc[(events_df['gameID'] == thisgameID) & (events_df['inning'] == this_inning - 1) & (events_df['visitor_or_home'] == vh)][[x for x in events_df.columns if x in noplay_df.columns]]
                        if (printme):
                            print('\t---- most recent eventID is {0:.0f} ----'.format(most_recent_eventID))
                    else:
                        noplay_df.loc[-1, :] = events_df.loc[ix-1][[x for x in events_df.columns if x in [y for y in noplay_df.columns if y != 'ix']]]
                        noplay_df.loc[-1, 'ix'] = ix-1
                        
                    # Is this the last event of this half-inning?
                    if (ix == this_half_inning_events_df.index.max()):
                        noplay_df.loc[1, :] = pandas.Series(dtype=object)
                    else:
                        noplay_df.loc[1, :] = events_df.loc[ix+1][[x for x in events_df.columns if x in [y for y in noplay_df.columns if y != 'ix']]]
                        noplay_df.loc[1, 'ix'] = ix+1
                    noplay_df = noplay_df.sort_index()
                    
                    ## Send thisrow plus previous and/or next row
                    if (printme):
                        #state = noplay(noplay_df, state, printme)
                        print("\t\tstate = noplay(noplay_df, state, {0:.0f})".format(printme))
                        print('\tthisrow has ix = {0:.0f}, followed by ix = {1:.0f} and preceded by ix = {2:.0f}'.format(noplay_df['ix'].loc[0], noplay_df['ix'].loc[1], noplay_df['ix'].loc[-1]))                    
                    state = noplay(noplay_df, state, printme)
                    
                    if (printme):
                        print('------------- /NP ------------------')

                elif (re.search(non_batter_re, thisrow['theplay']) != None):
                    print_batterID_at_end = True
                    thematch = re.match(non_batter_re, thisrow['theplay'])
#                     if (printme):
#                         print('Non batter event: {0:} |{1:}|'.format(thematch, thisrow['theplay']))
                    if (re.search(sb_re, thisrow['theplay']) != None):
                        sbmatch = re.match(sb_re, thisrow['theplay'])
#                         if (printme):
#                             print('Stolen base: {0:} |{1:}|'.format(sbmatch, thisrow['theplay']))
                        if (has_baserunning):
                            state = steal_bases(thisrow['theplay'], state, printme, thisrow['baserunning'])
                        else:
                            state = steal_bases(thisrow['theplay'], state, printme)
                    elif ('POCS' in thisrow['theplay']): 
#                         if (printme):
#                             print('Pickoff plus caught stealing: {0:} |{1:}|'.format(thematch, thisrow['theplay']))
                        if (has_baserunning):
                            state = catch_stealing(thisrow['theplay'][thisrow['theplay'].find('CS'):], state, printme, thisrow['baserunning'])
                        else:
                            state = catch_stealing(thisrow['theplay'][thisrow['theplay'].find('CS'):], state, printme)
                    elif ('CS' in thisrow['theplay']): 
#                         if (printme):
#                             print('Caught stealing: {0:} |{1:}|'.format(thematch, thisrow['theplay']))
                        if (has_baserunning):
                            state = catch_stealing(thisrow['theplay'], state, printme, thisrow['baserunning'])
                        else:
                            state = catch_stealing(thisrow['theplay'], state, printme)
                    elif (re.search(pickoff_re, thisrow['theplay']) != None):
                        pomatch = re.search(pickoff_re, thisrow['theplay'])
#                         if (printme):
#                             print('Pickoff: {0:} |{1:}|'.format(thisrow['theplay'][pomatch.start():pomatch.end()], thisrow['theplay']))
                        if (has_baserunning):
                            state = pickoff(thisrow['theplay'], state, printme, thisrow['baserunning'])
                        else:
                            state = pickoff(thisrow['theplay'], state, printme)
                        
                    ### Events that advance baserunners but never batter
                    elif (('BK' in thisrow['theplay']) or ('WP' in thisrow['theplay']) or ('PB' in thisrow['theplay']) or ('OA' in thisrow['theplay'])):
#                         if (printme):
#                             print('Event that advances baserunners but not batter: {0:} |{1:}|'.format(thematch, thisrow['theplay']))
                        state = baserunning(thisrow['baserunning'], state, printme)

                elif (re.search(hit_re, thisrow['theplay']) != None):   ### Next, events that get batter on base
                    thematch = re.match(hit_re, thisrow['theplay'])
#                     if (printme):
#                         print('\t\tHit: {0:} |{1:}|'.format(thematch, thisrow['theplay']))
                    if (has_baserunning):
                        state = baserunning(thisrow['baserunning'], state, printme)
                    if (thisrow['theplay'][0] == 'S'):
                        if not(state['manual_batter_advance']):
                            state['onbase'][1] = thisrow['batterID']
                    elif (thisrow['theplay'][0] == 'D'):
                        if not(state['manual_batter_advance']):
                            state['onbase'][2] = thisrow['batterID']
                    elif (thisrow['theplay'][0] == 'T'):
                        if not(state['manual_batter_advance']):
                            state['onbase'][3] = thisrow['batterID']
                    elif (thisrow['theplay'][0:2] == 'HR'):
                        if not(state['manual_batter_advance']):
                            state['score'][vh] += 1
                        
                elif (re.search(onbase_nohit_re, thisrow['theplay']) != None):   ### Next, other events that get batter on base
#                     if (printme):
#                         if ('+' in thisrow['theplay']):
#                             print('\t\tEvent {0:.0f}: On base with no hit plus extra: {1:}'.format(thisrow['event_in_game'], thisrow['theplay']))
#                         else:
#                             print('\t\tOn base with no hit!')
                    if ('W+SB' in thisrow['theplay']):
                        if (has_baserunning):
                            state = steal_bases(thisrow['theplay'][thisrow['theplay'].find('SB'):], state, printme, thisrow['baserunning'])
                        else:
                            state = steal_bases(thisrow['theplay'][thisrow['theplay'].find('SB'):], state, printme)
                    elif ('W+CS' in thisrow['theplay']):
                        if (has_baserunning):
                            state = catch_stealing(thisrow['theplay'][thisrow['theplay'].find('CS'):], state, printme, thisrow['baserunning'])
                        else:
                            state = catch_stealing(thisrow['theplay'][thisrow['theplay'].find('CS'):], state, printme)
                    elif ('W+POCS' in thisrow['theplay']):
                        pass
                        # SFN201407300 event 64, just do a special case
                    elif ('W+PO' in thisrow['theplay']):
                        if (has_baserunning):
                            state = pickoff(thisrow['theplay'][thisrow['theplay'].find('PO'):], state, printme, thisrow['baserunning'])
                        else:
                            state = pickoff(thisrow['theplay'][thisrow['theplay'].find('PO'):], state, printme)
                    else:
                        if (has_baserunning):  # advance bases if they weren't advanced in a + event
                            state = baserunning(thisrow['baserunning'], state, printme)
                    if (not(state['manual_batter_advance'])):
                        state['onbase'][1] = state['batterID'] # regardless of events given in +, batter goes to 1st
                            
                elif (thisrow['theplay'][0] == 'K'): ### Start with events that get the batter out
#                     if (printme):
#                         if ('+' in thisrow['theplay']):
#                             print('\t\tEvent {0:.0f}: Strikeout plus extra: {1:}'.format(thisrow['event_in_game'], thisrow['theplay']))
#                         else:
#                             print('\t\tStrikeout!')
                    if ('+' in thisrow['theplay']):
                        if ('SB' in thisrow['theplay']):
                            state = steal_bases(thisrow['theplay'][thisrow['theplay'].find('SB'):], state, printme, thisrow['baserunning'])
                            if (not(state['manual_batter_advance'])):
                                state['nOuts'] += 1    # batter still out
                        elif ('CS' in thisrow['theplay']):
                            state = catch_stealing(thisrow['theplay'][thisrow['theplay'].find('CS'):], state, printme, thisrow['baserunning'])
                            if (not(state['manual_batter_advance'])):
                                state['nOuts'] += 1    # batter still out
                        elif ('PO' in thisrow['theplay']):
                            state = pickoff(thisrow['theplay'][2:], state, printme, thisrow['baserunning'])
                            if not(state['manual_batter_advance']):
                                state['nOuts'] += 1    # batter still out
                        elif (('DI' in thisrow['theplay']) | ('OA' in thisrow['theplay'])):
                            state = baserunning(thisrow['baserunning'], state, printme)
                            if (not(state['manual_batter_advance'])):
                                state['nOuts'] += 1    # batter still out                                
                        elif ('E' in thisrow['theplay']):
                            if (has_baserunning):
                                state = baserunning(thisrow['baserunning'], state, printme)
                            #else:
                            state['nOuts'] += 1
                        elif ('WP' in thisrow['theplay']):
                            state = baserunning(thisrow['baserunning'], state, printme)
                            if (not(state['manual_batter_advance'])):
                                state['nOuts'] += 1    # batter still out
                        elif ('PB' in thisrow['theplay']):
                            state = baserunning(thisrow['baserunning'], state, printme)
                            if (not(state['manual_batter_advance'])):
                                state['nOuts'] += 1
#                             if (not(state['manual_batter_advance'])): # batter still out, unlesss... explicit batter baserunning OR
#                                 if ('K23' in thisrow['theplay']):  # ...K23 (C to 1B after dropped 3rd strike)
#                                     state['nOuts'] += 1
#                                 else:
#                                     state['onbase'][1] = state['batterID']
                        if (state['nOuts'] > 3):
                            state['nOuts'] = 3
                    else:
                        if (has_baserunning):
                            state = baserunning(thisrow['baserunning'], state, printme)
                        if (not(state['manual_batter_advance'])):
                            state['nOuts'] += 1
                    
#                     if (not(state['manual_batter_advance'])):
#                         state['nOuts'] += 1
                        
                elif (re.search(fielder_out_re, thisrow['theplay'])):
                    if (re.search(base_updater_atom_re, thisrow['theplay']) == None):
#                         if (printme):
#                             print('\t\tFielder out: {0:}'.format(thisrow['theplay']))
                        state['nOuts'] += 1
                        if (has_baserunning):
                            state = baserunning(thisrow['baserunning'], state, printme)
                    else:
#                         if (printme):
#                             print('\t\tFielder out with shenanigans: {0:}'.format(thisrow['theplay']))
                        involves_batter = False
                        batter_implied = False
                        base_updater_iter = re.finditer(base_updater_atom_re, thisrow['theplay'])
                        nBaseUpdaters = 0
                        for this_base_updater in base_updater_iter:
                            nBaseUpdaters += 1
                            from_base_str = thisrow['theplay'][this_base_updater.start()+1:this_base_updater.end()-1]
                            if (from_base_str == 'B'):
#                                 if (printme):
#                                     print('\t{0:}: batter out'.format(this_base_updater))
                                involves_batter = True
                            else:
                                from_base = int(thisrow['theplay'][this_base_updater.start()+1:this_base_updater.end()-1])
#                                 if (printme):
#                                     print('\t{0:}: runner from {1:} out'.format(this_base_updater, ordinaltg(from_base)))

                                state['onbase'][from_base] = ''
                                state['nOuts'] += 1
                        if (re.search(mp_re, thisrow['theplay']) != None):
                            mpmatch = re.search(mp_re, thisrow['theplay'])
#                             if (printme):
#                                 print('MULTIPLE PLAY: {0:} |{1:}|, {2:.0f} base updaters'.format(thisrow['theplay'][mpmatch.start():mpmatch.end()], thisrow['theplay'], nBaseUpdaters))
                            if (((thisrow['theplay'][mpmatch.start():mpmatch.end()] == 'DP') and (nBaseUpdaters < 2)) or ((thisrow['theplay'][mpmatch.start():mpmatch.end()] == 'TP') and (nBaseUpdaters < 3))):
                                batter_implied = True
                            #state['killswitch'] = True
                        if (has_baserunning):
                            state = baserunning(thisrow['baserunning'], state, printme)
                        if (not(state['manual_batter_advance'])):
                            if (involves_batter or batter_implied):
                                state['nOuts'] += 1
                            else:
                                state['onbase'][1] = state['batterID']
                        if (state['nOuts'] > 3):
                            state['nOuts'] = 3
                else:
                    print('UNKNOWN EVENT: {0:} | {1:}'.format(thisrow['theplay'], thisrow['play_orig']))
                    state['killswitch'] = True
                    
                ## is this a SH?
                if ('/SH' in thisrow['play_orig']):
                    if (printme):
                        print('^^^^^^^^^^^^^^^SAC HIT!^^^^^^^^^^^^^^')
                    new_things_df.loc[ix, 'sh_flag'] = True
                if ('/SF' in thisrow['play_orig']):
                    if (printme):
                        print('~~~~~~~~~~~~~SAC FLY!~~~~~~~~~~~~~~~~')
                    new_things_df.loc[ix, 'sf_flag'] = True
                
                if (state['nOuts'] > 3):  # if it appears there are more than 3 outs, there are 3 outs
                    state['nOuts'] = 3

                ### SPECIAL CASES!
                if (thisrow['gameID'] in list(special_case_events.keys())):
                    if (thisrow['event_in_game'] in special_case_events[thisrow['gameID']]):
                        if (printme):
                            print('fixing special case event...')
                        state = fix_special_case_event(thisrow['gameID'], thisrow['event_in_game'], state)
#                     if (printme):
#                         if (print_batterID_at_end):
#                             print('     Event {0:.0f} ending state: V {1:.0f} H {2:.0f}, {3:.0f} outs, onbase = {4:}, {5:} at bat'.format(thisrow['event_in_game'], state['score'][0], state['score'][1], state['nOuts'], state['onbase'], thisrow['batterID']))
#                         else:
#                             print('     Event {0:.0f} ending state: V {1:.0f} H {2:.0f}, {3:.0f} outs, onbase = {4:}'.format(thisrow['event_in_game'], state['score'][0], state['score'][1], state['nOuts'], state['onbase']))
#                         print('     ***')
#                         print('\n')

            # checks: each half-inning must end with 3 outs, and each inning must end with scores matching game log
            if (state['nOuts'] != 3):
                if (thisrow['gameID'] in early_end_game_list):
                    if (not(already_said_ended_early)):
                        print('\tGAME NUMBER {0:.0f} ENDED EARLY: {1:}'.format(this_game_number, thisrow['gameID']))
                        #print('\n')
                        already_said_ended_early = True
                else:
                    if not((this_inning == np.max(this_game_innings)) & (vh == 1)):
                        print('DISCREPANCY: {0:} of the {1:} has {2:0d} outs!'.format(inningside.title(), ordinaltg(this_inning), state['nOuts']))
                        state['killswitch'] = True
                        max_event_id = ix
            skips = 0
            sco = 0
            if (not(already_said_ended_early) & (games_df.loc[thisgameID]['has_linescores'])):
                if (vh == 0):
                #    print(games_df.loc[thisgameID]['visitor_linescore'])
                    if (thisgameID == 'PHI192907062'):
                        if (this_inning == 9):
                            sco = double_digit_scores_in_multitple_innings(games_df, thisgameID, this_inning, vh, True)
                        else:
                            sco = double_digit_scores_in_multitple_innings(games_df, thisgameID, this_inning, vh)
                    else:   # every other game
                        for j in range(0, this_inning):
                            if (games_df.loc[thisgameID]['visitor_linescore'][j] == '('):
                                visitor_linescore_raw = games_df.loc[thisgameID]['visitor_linescore']
                                skips = visitor_linescore_raw[j:].find(')')
                                sco += int(visitor_linescore_raw[visitor_linescore_raw.find('(')+1:visitor_linescore_raw.find(')')])
                            else:
                                sco += int(games_df.loc[thisgameID]['visitor_linescore'][j+skips])
                    if (state['score'][0] != sco):
                        print('SCORE DISCREPANCY: {0:} of the {1:}: {2:} has {3:0d} and they should have {4:0d} <linescore: {5:}>'.format(inningside, ordinaltg(this_inning), games_df.loc[thisgameID]['visitor'], state['score'][0], sco, games_df.loc[thisgameID]['visitor_linescore']))
                        state['killswitch'] = True
                        max_event_id = ix
                else:
                #    print(games_df.loc[thisgameID]['home_linescore'])
                    if (thisgameID in ['CHN192208250', 'BRO194307100']):
                        if (this_inning == 9):
                            sco = double_digit_scores_in_multitple_innings(games_df, thisgameID, this_inning, vh, True)
                        else:
                            sco = double_digit_scores_in_multitple_innings(games_df, thisgameID, this_inning, vh)
                    else:  # every other game
                        for j in range(0, this_inning):
                            if (games_df.loc[thisgameID]['home_linescore'][j+skips] != 'x'):
                                if (games_df.loc[thisgameID]['home_linescore'][j] == '('):
                                    home_linescore_raw = games_df.loc[thisgameID]['home_linescore']
                                    skips = home_linescore_raw[j:].find(')')
                                    sco += int(home_linescore_raw[home_linescore_raw.find('(')+1:home_linescore_raw.find(')')])
                                else:
                                    sco += int(games_df.loc[thisgameID]['home_linescore'][j+skips])
                    if (state['score'][1] != sco):
                        print('SCORE DISCREPANCY: {0:} of the {1:}: {2:} has {3:0d} and they should have {4:0d} <linescore: {5:}>'.format(inningside, ordinaltg(this_inning), games_df.loc[thisgameID]['home'], state['score'][1], sco, games_df.loc[thisgameID]['home_linescore']))
                        state['killswitch'] = True
                        max_event_id = ix            
            if (state['killswitch']):
                break
        if (state['killswitch']):
            break
    if (state['killswitch']):
        break
    else:
        max_event_id = ix
# if (killswitch):
#     print('killswitch', state['killswixtch'])
events_contexts_df = pandas.concat((events_df.loc[events_df.index <= max_event_id], new_things_df.loc[new_things_df.index <= max_event_id]), axis=1)
e = time.time()
g += e-s
print('\n')

print('Processed {0:,.0f} events in {1:.0f} minutes {2:.0f} seconds!'.format(len(events_contexts_df), np.floor((e-s)/60), (e-s)%60))

creating a new dataframe to hold what we calculate here...

parsing plays...
Found 214,038 events from 2,430 games-with-events!


Reading game 0 of 2430: ANA202204070...
Reading game 100 of 2430: BAL202205190...
	GAME NUMBER 159 ENDED EARLY: BAL202210030
Reading game 200 of 2430: BOS202207060...
Reading game 300 of 2430: CHA202208150...
Reading game 400 of 2430: CLE202210010...
Reading game 500 of 2430: HOU202205190...
Reading game 600 of 2430: KCA202206120...
Reading game 700 of 2430: MIN202208030...
Reading game 800 of 2430: NYA202209100...
Reading game 900 of 2430: SEA202205050...
Reading game 1000 of 2430: TBA202206030...
Reading game 1100 of 2430: TEX202208010...
Reading game 1200 of 2430: TOR202208310...
Reading game 1300 of 2430: ATL202204110...
Reading game 1400 of 2430: CHN202205302...
Reading game 1500 of 2430: CIN202207072...
Reading game 1600 of 2430: COL202208130...
Reading game 1700 of 2430: LAN202210050...
	Fixing a special case in game number 1785 (MIL202204170) at even

In [82]:
s = time.time()
events_contexts_df = pandas.concat((events_df, new_things_df), axis=1)

events_contexts_df = events_contexts_df[final_column_order]
events_contexts_df.to_csv('{0:0d}rs.csv'.format(thisyear))
e = time.time()
g += e-s
print('Processed {0:,.0f} events in {1:.0f} minutes {2:.0f} seconds!'.format(len(new_things_df), np.floor(g/60), g%60))

print('\nTotal time: {0:.0f} minutes {1:.0f} seconds!'.format(np.floor(g/60), g%60))

print(events_contexts_df.index.max()) 

Processed 214,038 events in 39 minutes 30 seconds!

Total time: 39 minutes 30 seconds!
16107577


In [83]:
FileLink(r'{0:}rs.csv'.format(thisyear))
# events_df[
#     (events_df['gameID'] == 'PHI191907240') 
#     & ((events_df['event_in_game'] > 92) & (events_df['event_in_game'] < 103)
#       )
# ][['gameID', 'event_in_game', 'batterID', 'pitcherID', 'badj', 'ladj', 'pinch_runner_playerID', 'play_orig', 'comments']]

# # #['gameID', 'event_in_game', 'batterID', 'pitcherID', 'badj', 'ladj', 'pinch_runner_playerID', 'play_orig', 'comments']]
# # #[['event_in_game', 'play_orig', 'field_C_playerID', 'field_1B_playerID', 'field_2B_playerID', 'field_3B_playerID', 'field_SS_playerID', 'field_LF_playerID', 'field_CF_playerID', 'field_RF_playerID']]


/kaggle/working/2022rs.csv

In [84]:
# events_df[
#     (events_df['gameID'] == 'CHN197807192') 
#     #& (events_df['theplay'] == 'NP')  # 13, 24, 30, 31...
#     & (events_df['inning'] == 2) & (events_df['visitor_or_home'] == 1)
# ][['event_in_game', 'inning', 'visitor_or_home', 'theplay', 'batterID', 'pitcherID']]

In [85]:
#HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [86]:
#events_df[(events_df['gameID'] == 'PHA192706222') & ((events_df['event_in_game'] > 39) & (events_df['event_in_game'] < 49))][['field_LF_playerID']]